In [ ]:
import os
import time
import pandas as pd
from plotnine import (
    aes,
    element_text,
    facet_wrap,
    geom_bar,
    geom_col,
    ggplot,
    ggsave,
    guides,
    labs,
    scale_fill_gradient,
    scale_x_date,
    theme,
)
from mizani.formatters import date_format
from typing import Optional

def preprocess_data(df):
    '''
    Load the dataframe
    Do some preprocessing of the input data, such as fixing account allocations
    '''

    # df = pd.read_csv('../dfout_all.csv', dtype={15: str})

    #Fix jiaowa account info to ferrier rather than sbs
    mask = (df['User'] == 'jiaowa') & (df['Account'] == 'sbs')
    df.loc[mask, 'Account'] = 'ferrier'

    # Mapping of old account names to new ones
    account_mapping = {
        'scpslab206': 'scps', 
        'scpslab306': 'scps',
        'spacejam': 'scps',
        'phys414': 'scps',
        'students': 'scps',
        'cad': 'admin',
        'root': 'admin'
    }

    # Replace account names
    df['Account'] = df['Account'].replace(account_mapping)

    # currently we use a fix usd to nzd exchange rate to calculate aws cost
    usd_to_nzd = 1.62
    df.aws_cost = df.aws_cost * usd_to_nzd

    # set dates as datetime and create columns for month and year
    df['Start'] = pd.to_datetime(df['Start'])
    df['Submit'] = pd.to_datetime(df['Submit'])
    df['End'] = pd.to_datetime(df['End'])
    df['Year'] = df['Start'].dt.year
    df['Month'] = df['Start'].dt.month

    return df

def generate_plot(df: pd.DataFrame, x_column: str, title: str, subtitle: str, filename: str, width: Optional[int] = 20) -> None:
    if x_column == 'Year':
        width = 0.7  # A value of 0.7 is commonly used when you have yearly data.
    elif x_column == 'YearMonth':
        width = 20  # This width can be adjusted based on how wide you want the bars to be.

    plot = (
        ggplot(df, aes(x=x_column, y='UniqueUsers', fill='UniqueUsers'))
        + geom_bar(stat='identity', width=width)
        + scale_fill_gradient(low="blue", high="red")
        + labs(x='Date', y='Unique Users', title=title, subtitle=subtitle, fill='UniqueUsers')
        + theme(axis_text_x=element_text(angle=45, hjust=1),  # rotate x-axis labels 45 degrees
                plot_title=element_text(hjust=0.5),  # center title
                plot_subtitle=element_text(hjust=0.5))  # center subtitle
        + guides(fill=False)  # remove color bar
    )

    # Save the plot
    # ggsave(plot, filename=filename, format='png', dpi=300)
    return plot


def plot_unique_users_per_month(df):
    
    # Group by 'Account', 'Year', 'Month' and 'User', then count unique 'User'
    unique_users = df.groupby(['Account', 'Year', 'Month', 'User']).size().reset_index().rename(columns={0:'count'})
    print(unique_users.head())

    # Now group by 'Account', 'Year' and 'Month' and count unique 'User'
    unique_users_per_month = unique_users.groupby(['Account', 'Year', 'Month']).size().reset_index().rename(columns={0:'UniqueUsers'})
    
    # Convert 'Year' and 'Month' to integer, then to string, combine them, and convert to datetime
    unique_users_per_month['YearMonth'] = pd.to_datetime(unique_users_per_month['Year'].astype(int).astype(str) + '-' + unique_users_per_month['Month'].astype(int).astype(str))
   
    # Capitalize 'Account'
    unique_users_per_month['Account'] = unique_users_per_month['Account'].str.upper()

    accounts = unique_users_per_month['Account'].unique()

    # Create the directory if it doesn't already exist
    if not os.path.exists('plots/monthly_users'):
        os.makedirs('plots/monthly_users')

    for account in accounts:
        account_data = unique_users_per_month[unique_users_per_month['Account'] == account]
        generate_plot(account_data, 'YearMonth', 'Rāpoi', f'Unique {account} Users Per Month', f'plots/monthly_users/{account}_users_per_month.png')
        
    # Produce the total unique users per month
    start_time = time.time()
    # For the total unique users per month, group the original DataFrame by Year and Month and sum the unique users
    total_users_per_month = unique_users_per_month.groupby(['YearMonth'])['UniqueUsers'].sum().reset_index()
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Creating total unique users took:', elapsed_time, 'seconds')

    generate_plot(total_users_per_month, 'YearMonth', 'Rāpoi', 'Total Unique Users Per Month', 'plots/monthly_users/total_users_per_month.png')

def plot_unique_users_per_year(df):
    unique_users_per_year = df.groupby(['Account', 'Year', 'User']).size().reset_index().rename(columns={0:'count'})

    # Now group by 'Account' and 'Year' and count unique 'User'
    unique_users_per_year = unique_users_per_year.groupby(['Account', 'Year']).size().reset_index().rename(columns={0:'UniqueUsers'})
    accounts = unique_users_per_year['Account'].unique()

    # Create the directory if it doesn't already exist
    if not os.path.exists('plots/yearly_users'):
        os.makedirs('plots/yearly_users/')

    for account in accounts:
        account_data = unique_users_per_year[unique_users_per_year['Account'] == account]
        generate_plot(account_data, 'Year', 'Rāpoi', f'Unique {account} Users Per Year', f'plots/yearly_users/{account}_users_per_year.png')
    

    # Produce the total unique users per year
    start_time = time.time()
    # For the total unique users per year, group the original DataFrame by Year and Month and sum the unique users
    total_users_per_year = unique_users_per_year.groupby(['Year'])['UniqueUsers'].sum().reset_index()
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Creating total unique users took:', elapsed_time, 'seconds')

    # For total users
    generate_plot(total_users_per_year, 'Year', 'Rāpoi', 'Total Unique Users Per Year', 'plots/yearly_users/total_users_per_year.png')



def plot_costs_per_year(df):
    # Group by 'Account' and 'Year' and sum 'aws_cost' and 'nesi_cost'
    cost_per_year = df.groupby(['Account', 'Year']).agg({'aws_cost': 'sum', 'nesi_cost': 'sum'}).reset_index()

    # Convert 'Year' to integer, then to string, and convert to datetime
    cost_per_year['Year'] = pd.to_datetime(cost_per_year['Year'].astype(int).astype(str))

    

    # Capitalize 'Account'
    cost_per_year['Account'] = cost_per_year['Account'].str.upper()

    accounts = cost_per_year['Account'].unique()

    # Ensure the directories exist
    os.makedirs('plots/yearly_costs/aws/', exist_ok=True)
    os.makedirs('plots/yearly_costs/nesi/', exist_ok=True)

    for account in accounts:
        account_data = cost_per_year[cost_per_year['Account'] == account]

        for cost_type in ['aws_cost', 'nesi_cost']:
            cost_title = 'AWS cost' if cost_type == 'aws_cost' else 'NeSi cost'
            cost_subtitle = 'Based on 2020 best matched instance for given core count' if cost_type == 'aws_cost' else ' '
            save_folder = 'plots/yearly_costs/aws/' if cost_type == 'aws_cost' else 'plots/yearly_costs/nesi/'

            plot = (
                ggplot(account_data, aes(x='Year', y=cost_type, fill=cost_type))
                + geom_col()  # using geom_col instead of geom_bar with stat='identity'
                + scale_fill_gradient(low = "blue", high = "red")
                + labs(x='Year', y='Cost', title=f'{cost_title} for {account} Per Year', subtitle=cost_subtitle, fill=cost_type)
                + theme(axis_text_x = element_text(angle = 45, hjust = 1),  # rotate x-axis labels 45 degrees
                        plot_title=element_text(hjust=0.5),  # center title
                        plot_subtitle=element_text(hjust=0.5))  # center subtitle
                + guides(fill=False)  # remove color bar
                + scale_x_date(date_breaks='1 year', labels=date_format('%Y'))  # set x-axis breaks and labels
            )
            
            print(plot)

            # Save the plot
            # plot.save(f"{save_folder}{account}_{cost_type}.png")


def plot_costs_per_month(df):
    # Group by 'Account', 'Year' and 'Month' and sum 'aws_cost' and 'nesi_cost'
    cost_per_month = df.groupby(['Account', 'Year', 'Month']).agg({'aws_cost': 'sum', 'nesi_cost': 'sum'}).reset_index()

    # Convert 'Year' and 'Month' to integer, then to string, combine them, and convert to datetime
    cost_per_month['YearMonth'] = pd.to_datetime(cost_per_month['Year'].astype(int).astype(str) + '-' + cost_per_month['Month'].astype(int).astype(str))

    accounts = cost_per_month['Account'].unique()

    # Capitalize 'Account'
    cost_per_month['Account'] = cost_per_month['Account'].str.upper()

    # Ensure the directories exist
    os.makedirs('plots/monthly_costs/aws/', exist_ok=True)
    os.makedirs('plots/monthly_costs/nesi/', exist_ok=True)

    for account in accounts:
        account_data = cost_per_month[cost_per_month['Account'] == account]

        for cost_type in ['aws_cost', 'nesi_cost']:
            cost_title = 'AWS cost' if cost_type == 'aws_cost' else 'NeSi cost'
            cost_subtitle = 'Based on 2020 best matched instance for given core count' if cost_type == 'aws_cost' else ' '
            save_folder = 'plots/monthly_costs/aws/' if cost_type == 'aws_cost' else 'plots/monthly_costs/nesi/'

            plot = (
                ggplot(account_data, aes(x='YearMonth', y=cost_type, fill=cost_type))
                + geom_bar(stat='identity', width=20)  # adjust the width as needed
                + scale_fill_gradient(low = "blue", high = "red")
                + labs(x='Date', y='Cost', title=f'{cost_title} for {account} Per Month', subtitle=cost_subtitle, fill=cost_type)
                + theme(axis_text_x = element_text(angle = 45, hjust = 1),  # rotate x-axis labels 45 degrees
                        plot_title=element_text(hjust=0.5),  # center title
                        plot_subtitle=element_text(hjust=0.5))  # center subtitle
                + guides(fill=False)  # remove color bar
            )
            
            print(plot)

            # Save the plot
        # plot.save(f"{save_folder}{account}_{cost_type}.png")

############################################################
# Plot wait times for jobs to start from submit time with y axis as time in seconds and x axis as the number of jobs
import pandas as pd
from plotnine import (
    ggplot,
    aes,
    geom_bar,
    geom_col, # geom_col is suitable for pre-summarized data
    scale_fill_gradient,
    labs,
    theme,
    element_text,
    guides,
)
import os # Import os for directory creation

def plot_submit_start_time(df: pd.DataFrame):
    """
    Orchestrates plotting of:
    1. Total wait time per month for each account (jobs >= 4 hours wait time).
    2. Total wait time for each unique job (jobs >= 4 hours wait time).

    Args:
        df (pd.DataFrame): DataFrame containing 'Start', 'Submit', 'Account', and 'JobID' columns.
    """

    # Create a copy to avoid modifying the original DataFrame passed into the function
    df_copy = df.copy()

    # --- Plotting Wait Time Per Account Per Month ---

    # Ensure 'Start' and 'Submit' columns are datetime objects
    df_copy['Start'] = pd.to_datetime(df_copy['Start'], errors='coerce')
    df_copy['Submit'] = pd.to_datetime(df_copy['Submit'], errors='coerce')

    # Drop rows where 'Start' or 'Submit' could not be converted to datetime
    df_copy.dropna(subset=['Start', 'Submit'], inplace=True)

    # Calculate the difference between 'Start' and 'Submit' in seconds
    df_copy['WaitTime'] = (df_copy['Start'] - df_copy['Submit']).dt.total_seconds()

    # Only keep rows where 'WaitTime' is greater than or equal to 4 hours (14400 seconds)
    df_filtered_accounts = df_copy[df_copy['WaitTime'] >= 14400]

    if df_filtered_accounts.empty:
        print("No data meets the criteria (WaitTime >= 4 hours) for per-account plotting.")
    else:
        # Convert 'WaitTime' from seconds to hours for better readability on the plot
        df_filtered_accounts['WaitTime'] = df_filtered_accounts['WaitTime'] / 3600

        # Extract Year and Month from the 'Submit' time
        df_filtered_accounts['Year'] = df_filtered_accounts['Submit'].dt.year
        df_filtered_accounts['Month'] = df_filtered_accounts['Submit'].dt.month

        # Group by 'Account', 'Year', and 'Month' and sum the 'WaitTime'
        wait_time_per_month = df_filtered_accounts.groupby(['Account', 'Year', 'Month']).agg(
            WaitTime=('WaitTime', 'sum')
        ).reset_index()

        # Create 'YearMonth' column as a datetime object for chronological plotting
        wait_time_per_month['YearMonth'] = pd.to_datetime(
            wait_time_per_month['Year'].astype(str)
            + '-'
            + wait_time_per_month['Month'].astype(str)
            + '-01'
        )

        # Get a list of unique accounts to iterate through and generate a plot for each
        accounts = wait_time_per_month['Account'].unique()

        # Create the directory for saving plots if it doesn't exist
        plots_dir_accounts = "plots/wait_times_per_account"
        os.makedirs(plots_dir_accounts, exist_ok=True)

        # Loop through each unique account to create and save a plot
        for account in accounts:
            account_data = wait_time_per_month[
                wait_time_per_month['Account'] == account
            ].copy()

            plot_account = (
                ggplot(account_data, aes(x='YearMonth', y='WaitTime', fill='WaitTime'))
                + geom_bar(stat='identity', width=20)
                + scale_fill_gradient(low="blue", high="red")
                + labs(
                    x='Date',
                    y='Wait Time (hours)',
                    title=f'Wait Time for {account} Per Month',
                    subtitle='Total Wait Time Per Month (Jobs >= 4 hours Wait Time)',
                    fill='WaitTime'
                )
                + theme(
                    axis_text_x=element_text(angle=45, hjust=1),
                    plot_title=element_text(hjust=0.5),
                    plot_subtitle=element_text(hjust=0.5)
                )
                + guides(fill=False)
            )

            print(f"Generating plot for account: {account}")
            print(plot_account)
            plot_filename_account = os.path.join(plots_dir_accounts, f"{account}_wait_time_per_month.png")
            # plot_account.save(plot_filename_account)
            print(f"Saved plot: {plot_filename_account}")


    # --- Plotting Total Wait Time Per Job ---

    # We reuse the df_copy that has already been converted to datetime and dropped NaT
    # Calculate the difference between 'Start' and 'Submit' in seconds
    # (re-calculate if df_filtered_accounts modified df_copy, but here it's on a copy)
    df_copy['WaitTime'] = (df_copy['Start'] - df_copy['Submit']).dt.total_seconds()

    # Only keep rows where 'WaitTime' is greater than or equal to 4 hours (14400 seconds)
    df_filtered_jobs = df_copy[df_copy['WaitTime'] >= 14400]

    if df_filtered_jobs.empty:
        print("No data meets the criteria (WaitTime >= 4 hours) to plot total wait time per job.")
        return

    # Convert 'WaitTime' from seconds to hours for better readability
    df_filtered_jobs['WaitTime'] = df_filtered_jobs['WaitTime'] / 3600

    # Group by 'JobID' and sum the 'WaitTime'
    total_wait_time_per_job = df_filtered_jobs.groupby('JobID').agg(
        TotalWaitTime=('WaitTime', 'sum')
    ).reset_index()

    if total_wait_time_per_job.empty:
        print("No job data after aggregation to plot total wait time per job.")
        return

    # --- IMPORTANT FIX: Order the 'JobID' categories explicitly ---
    # Sort the DataFrame by TotalWaitTime in ascending order
    total_wait_time_per_job = total_wait_time_per_job.sort_values(
        by='TotalWaitTime', ascending=True
    )
    # Convert 'JobID' column to an ordered categorical type based on the sorted order.
    # plotnine will then respect this order on the x-axis, achieving the 'reorder' effect.
    total_wait_time_per_job['JobID'] = pd.Categorical(
        total_wait_time_per_job['JobID'],
        categories=total_wait_time_per_job['JobID'], # Use the current order as the categories
        ordered=True
    )

    # Create the directory for saving plots if it doesn't exist
    plots_dir_jobs = "plots/total_job_wait_times"
    os.makedirs(plots_dir_jobs, exist_ok=True)

    # Create the plot using plotnine
    plot_jobs = (
        ggplot(total_wait_time_per_job, aes(x='JobID', y='TotalWaitTime', fill='TotalWaitTime'))
        + geom_col(width=0.7)
        + scale_fill_gradient(low="blue", high="red")
        + labs(
            x='Job ID',
            y='Total Wait Time (hours)',
            title='Total Wait Time Per Job',
            subtitle='Sum of Wait Times (Jobs >= 4 hours Wait Time)',
            fill='TotalWaitTime'
        )
        + theme(
            axis_text_x=element_text(angle=90, hjust=1),
            plot_title=element_text(hjust=0.5),
            plot_subtitle=element_text(hjust=0.5)
        )
        + guides(fill=False)
    )

    print("Generating plot for total wait time per job:")
    print(plot_jobs)
    plot_filename_jobs = os.path.join(plots_dir_jobs, "total_wait_time_per_job.png")
    # plot_jobs.save(plot_filename_jobs)
    print(f"Saved plot: {plot_filename_jobs}")



#########################


##################################


import pandas as pd
import numpy as np
from plotnine import (
    ggplot,
    aes,
    geom_col,
    scale_fill_gradient,
    labs,
    theme,
    element_text,
    guides,
)
import os
import concurrent.futures
import traceback

def _process_df_chunk(chunk_df):
    """
    Helper function to process a chunk of the DataFrame in parallel.
    Performs datetime conversion, wait time calculation, and initial filtering.

    Args:
        chunk_df (pd.DataFrame): A chunk of the original DataFrame.

    Returns:
        pd.DataFrame: Processed chunk with 'WaitTime' and relevant columns, or an empty DataFrame if no data meets criteria.
    """
    try:
        # Explicitly create a deep copy to prevent SettingWithCopyWarning
        chunk_df = chunk_df.copy(deep=True)

        # Ensure 'Start' and 'Submit' columns are datetime objects
        chunk_df['Start'] = pd.to_datetime(chunk_df['Start'], errors='coerce')
        chunk_df['Submit'] = pd.to_datetime(chunk_df['Submit'], errors='coerce')

        # Drop rows where 'Start' or 'Submit' could not be converted to datetime (i.e., they are NaT)
        chunk_df.dropna(subset=['Start', 'Submit'], inplace=True)

        # Calculate the difference between 'Start' and 'Submit' in seconds
        chunk_df['WaitTime'] = (chunk_df['Start'] - chunk_df['Submit']).dt.total_seconds()

        # Only keep rows where 'WaitTime' is greater than or equal to 4 hours (14400 seconds)
        chunk_df = chunk_df[chunk_df['WaitTime'] >= 14400]

        # Convert 'WaitTime' from seconds to hours for better readability
        chunk_df['WaitTime'] = chunk_df['WaitTime'] / 3600

        # Select only the necessary columns ('JobID' and 'WaitTime')
        # This reduces memory usage when returning processed chunks
        return chunk_df[['JobID', 'WaitTime']]

    except Exception as e:
        print(f"Error processing DataFrame chunk: {e}")
        traceback.print_exc()
        return pd.DataFrame() # Return empty DataFrame on error


def plot_total_wait_time_per_jobid_multiprocessed(df, num_chunks=None, plot_dpi=150):
    """
    Plots the total wait time for each unique JobID, using multiprocessing
    for faster data preprocessing and aggregation.

    Args:
        df (pd.DataFrame): DataFrame containing 'Start', 'Submit', and 'JobID' columns.
        num_chunks (int, optional): Number of chunks to split the DataFrame into for
                                    parallel processing. If None, defaults to os.cpu_count().
        plot_dpi (int, optional): Resolution of the saved plot image in dots per inch.
                                  Lower values (e.g., 96, 150) result in smaller files and faster saving.
                                  Defaults to 150.
    """

    print("Starting data preprocessing with multiprocessing...")

    if num_chunks is None:
        num_chunks = os.cpu_count() # Use all available CPU cores by default

    # Split the DataFrame into chunks for parallel processing
    # Using numpy.array_split for more even distribution, especially for smaller DataFrames
    df_chunks = [chunk for chunk in np.array_split(df, num_chunks) if not chunk.empty]

    processed_chunks = []
    # Use ProcessPoolExecutor to parallelize chunk processing
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_chunks) as executor:
        futures = [executor.submit(_process_df_chunk, chunk) for chunk in df_chunks]

        for i, future in enumerate(concurrent.futures.as_completed(futures)):
            try:
                processed_chunk = future.result()
                if not processed_chunk.empty:
                    processed_chunks.append(processed_chunk)
                print(f"Finished processing chunk {i+1}/{len(df_chunks)}.")
            except Exception as exc:
                print(f'A chunk processing generated an exception: {exc}')
                traceback.print_exc()

    # Concatenate all processed chunks back into a single DataFrame
    if not processed_chunks:
        print("No data remained after multiprocessing and filtering. No plot will be generated.")
        return

    combined_df = pd.concat(processed_chunks).reset_index(drop=True)

    # If no data remains after filtering in any chunk, print a message and exit
    if combined_df.empty:
        print("No data meets the criteria (WaitTime >= 4 hours) after initial filtering and combining. No plot will be generated.")
        return

    print("Finished parallel data preprocessing. Starting aggregation...")

    # Group by 'JobID' and sum the 'WaitTime'
    total_wait_time_per_jobid = combined_df.groupby('JobID').agg(
        TotalWaitTime=('WaitTime', 'sum')
    ).reset_index()

    # If total_wait_time_per_jobid is empty after aggregation, print a message and exit
    if total_wait_time_per_jobid.empty:
        print("No JobID data after aggregation to plot total wait time per JobID.")
        return

    # Order the 'JobID' categories explicitly by 'TotalWaitTime' for plotting
    total_wait_time_per_jobid = total_wait_time_per_jobid.sort_values(
        by='TotalWaitTime', ascending=True
    )
    total_wait_time_per_jobid['JobID'] = pd.Categorical(
        total_wait_time_per_jobid['JobID'],
        categories=total_wait_time_per_jobid['JobID'],
        ordered=True
    )

    # Create the directory for saving plots if it doesn't exist
    plots_dir = "plots/total_jobid_wait_times"
    os.makedirs(plots_dir, exist_ok=True)

    print(f"Generating plot with DPI: {plot_dpi}...")
    # Create the plot using plotnine (this part remains single-threaded as it's one plot)
    plot = (
        ggplot(total_wait_time_per_jobid, aes(x='JobID', y='TotalWaitTime', fill='TotalWaitTime'))
        + geom_col(width=0.7)
        + scale_fill_gradient(low="blue", high="red")
        + labs(
            x='JobID',
            y='Total Wait Time (hours)',
            title='Total Wait Time Per Unique JobID',
            subtitle='Sum of Wait Times (>= 4 hours) for Each JobID',
            fill='TotalWaitTime'
        )
        + theme(
            axis_text_x=element_text(angle=90, hjust=1),
            plot_title=element_text(hjust=0.5),
            plot_subtitle=element_text(hjust=0.5)
        )
        + guides(fill=False)
    )

    # Save the generated plot to a file, using the specified DPI
    plot_filename = os.path.join(plots_dir, "total_wait_time_per_jobid.png")
    # plot.save(plot_filename, dpi=plot_dpi) # Added dpi parameter
    print(f"Successfully saved plot: {plot_filename}")

#########################


#########################

'''
Each unique user and wait time for their jobs 

'''
import pandas as pd
import numpy as np
from plotnine import (
    ggplot,
    aes,
    geom_col,
    scale_fill_gradient,
    labs,
    theme,
    element_text,
    guides,
)
import os
import concurrent.futures
import traceback

def _process_df_chunk(chunk_df):
    """
    Helper function to process a chunk of the DataFrame in parallel.
    Performs datetime conversion, wait time calculation, and initial filtering.

    Args:
        chunk_df (pd.DataFrame): A chunk of the original DataFrame.

    Returns:
        pd.DataFrame: Processed chunk with 'Account', 'JobID', 'WaitTime',
                      or an empty DataFrame if no data meets criteria.
    """
    try:
        # Explicitly create a deep copy to prevent SettingWithCopyWarning
        chunk_df = chunk_df.copy(deep=True)

        # Ensure 'Start' and 'Submit' columns are datetime objects
        chunk_df['Start'] = pd.to_datetime(chunk_df['Start'], errors='coerce')
        chunk_df['Submit'] = pd.to_datetime(chunk_df['Submit'], errors='coerce')

        # Drop rows where 'Start' or 'Submit' could not be converted to datetime (i.e., they are NaT)
        chunk_df.dropna(subset=['Start', 'Submit'], inplace=True)

        # Calculate the difference between 'Start' and 'Submit' in seconds
        chunk_df['WaitTime'] = (chunk_df['Start'] - chunk_df['Submit']).dt.total_seconds()

        # Only keep rows where 'WaitTime' is greater than or equal to 4 hours (14400 seconds)
        chunk_df = chunk_df[chunk_df['WaitTime'] >= 14400]

        # Convert 'WaitTime' from seconds to hours for better readability
        chunk_df['WaitTime'] = chunk_df['WaitTime'] / 3600

        # Select only the necessary columns ('Account', 'JobID', and 'WaitTime')
        return chunk_df[['Account', 'JobID', 'WaitTime']]

    except Exception as e:
        print(f"Error processing DataFrame chunk: {e}")
        traceback.print_exc()
        return pd.DataFrame() # Return empty DataFrame on error

def _plot_single_user_jobs(user_data_for_plot, account_name, base_plots_dir, plot_dpi):
    """
    Helper function to plot and save a single user's job wait time data.
    This function will be executed in a separate process.

    Args:
        user_data_for_plot (pd.DataFrame): DataFrame containing wait time data for a single user's jobs.
        account_name (str): The account name for the current plot.
        base_plots_dir (str): The base directory where plots should be saved.
        plot_dpi (int): Resolution of the saved plot image in dots per inch.
    """
    try:
        if user_data_for_plot.empty:
            print(f"No valid data for Account: {account_name}. Skipping plot.")
            return

        # Ensure JobID is ordered by TotalWaitTime for consistent plotting
        user_data_for_plot = user_data_for_plot.sort_values(
            by='TotalWaitTime', ascending=True
        )
        user_data_for_plot['JobID'] = pd.Categorical(
            user_data_for_plot['JobID'],
            categories=user_data_for_plot['JobID'],
            ordered=True
        )

        # Define a specific directory for each account to keep plots organized
        account_plots_dir = os.path.join(base_plots_dir, str(account_name))
        os.makedirs(account_plots_dir, exist_ok=True)

        # Create the plot using plotnine
        plot = (
            ggplot(user_data_for_plot, aes(x='JobID', y='TotalWaitTime', fill='TotalWaitTime'))
            + geom_col(width=0.7)
            + scale_fill_gradient(low="blue", high="red")
            + labs(
                x='JobID',
                y='Total Wait Time (hours)',
                title=f'Total Wait Time for Jobs by User: {account_name}',
                subtitle='Sum of Wait Times (>= 4 hours) for Each JobID',
                fill='TotalWaitTime'
            )
            + theme(
                axis_text_x=element_text(angle=90, hjust=1), # Rotate x-axis labels
                plot_title=element_text(hjust=0.5),
                plot_subtitle=element_text(hjust=0.5)
            )
            + guides(fill=False)
        )

        # Sanitize account name for filename to avoid issues with special characters
        sanitized_account_name = (
            str(account_name)
            .replace('/', '_')
            .replace('\\', '_')
            .replace(':', '_')
            .replace('*', '_')
            .replace('?', '_')
            .replace('"', '_')
            .replace('<', '_')
            .replace('>', '_')
            .replace('|', '_')
        )
        plot_filename = os.path.join(account_plots_dir, f"{sanitized_account_name}_jobs_wait_time.png")
        # plot.save(plot_filename, dpi=plot_dpi)
        print(f"Successfully saved plot for User: {account_name} to {plot_filename}")

    except Exception as e:
        print(f"Error plotting for User: {account_name}: {e}")
        traceback.print_exc()

def plot_jobs_per_user_wait_times_multiprocessed(df, num_chunks=None, plot_dpi=150):
    """
    Plots the total wait time for each JobID, grouped by unique user (Account),
    using multiprocessing for faster data preprocessing and plot generation.

    Args:
        df (pd.DataFrame): DataFrame containing 'Start', 'Submit', 'Account', and 'JobID' columns.
        num_chunks (int, optional): Number of chunks to split the DataFrame into for
                                    parallel processing. If None, defaults to os.cpu_count().
        plot_dpi (int, optional): Resolution of the saved plot image in dots per inch.
                                  Defaults to 150.
    """

    print("Starting initial data preprocessing with multiprocessing...")

    if num_chunks is None:
        num_chunks = os.cpu_count()

    df_chunks = [chunk for chunk in np.array_split(df, num_chunks) if not chunk.empty]

    processed_chunks = []
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_chunks) as executor:
        futures = [executor.submit(_process_df_chunk, chunk) for chunk in df_chunks]

        for i, future in enumerate(concurrent.futures.as_completed(futures)):
            try:
                processed_chunk = future.result()
                if not processed_chunk.empty:
                    processed_chunks.append(processed_chunk)
                print(f"Finished processing chunk {i+1}/{len(df_chunks)}.")
            except Exception as exc:
                print(f'A chunk processing generated an exception: {exc}')
                traceback.print_exc()

    if not processed_chunks:
        print("No data remained after multiprocessing and initial filtering. No plots will be generated.")
        return

    combined_df = pd.concat(processed_chunks).reset_index(drop=True)

    if combined_df.empty:
        print("No data meets the criteria (WaitTime >= 4 hours) after initial filtering and combining. No plots will be generated.")
        return

    print("Finished parallel data preprocessing. Starting user-wise aggregation...")

    # Group by 'Account' and 'JobID' to get total wait time per job for each user
    total_wait_time_per_user_job = combined_df.groupby(['Account', 'JobID']).agg(
        TotalWaitTime=('WaitTime', 'sum')
    ).reset_index()

    if total_wait_time_per_user_job.empty:
        print("No job data after user-wise aggregation. No plots will be generated.")
        return

    # Create the base directory for saving plots if it doesn't exist
    base_plots_dir = "plots/user_job_wait_times" # New directory for user-specific plots
    os.makedirs(base_plots_dir, exist_ok=True)

    # Get unique accounts to iterate through
    unique_accounts = total_wait_time_per_user_job['Account'].unique()
    print(f"Found {len(unique_accounts)} unique users to plot.")

    print("Generating plots for each user in parallel...")
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = []
        for account in unique_accounts:
            # Filter the aggregated data for the current user
            user_data = total_wait_time_per_user_job[
                total_wait_time_per_user_job['Account'] == account
            ].copy() # .copy() is important to avoid SettingWithCopyWarning in worker processes

            futures.append(
                executor.submit(
                    _plot_single_user_jobs, user_data, account, base_plots_dir, plot_dpi
                )
            )

        for future in concurrent.futures.as_completed(futures):
            try:
                future.result() # Re-raise any exceptions from worker processes
            except Exception as exc:
                print(f'An error occurred during plotting a user\'s jobs: {exc}')
                traceback.print_exc()

    print("All user job plots finished generating.")





#########################










def plot_all_slurm():
    '''
    Preprocess and then plot all the raapoi user and estimated cost data.
    '''
    df = pd.read_csv('raapoi_metrics/raapoi_data.csv', dtype={15: str})
    df = preprocess_data(df)

    plot_unique_users_per_month(df)
    plot_unique_users_per_year(df)
    plot_costs_per_year(df)
    # plot_costs_per_month(df) # debug
    # plot_submit_start_time(df) # optimize
    # plot_total_wait_time_per_jobid_multiprocessed(df, num_chunks=os.cpu_count())
    # plot_jobs_per_user_wait_times_multiprocessed(df, num_chunks=os.cpu_count(), plot_dpi=150)
    
plot_all_slurm()

In [ ]:
import os
import time
import pandas as pd
from plotnine import (
    aes,
    element_text,
    facet_wrap,
    geom_bar,
    geom_col,
    ggplot,
    ggsave,
    guides,
    labs,
    scale_fill_gradient,
    scale_x_date,
    theme,
)
from mizani.formatters import date_format
from typing import Optional, List
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def memfix(inmem):
    if pd.isnull(inmem):
        outmem = 0
    else:
        outmem = float(str(inmem).strip('M'))
    return outmem 

def preprocess_data(df):
    '''
    Load the dataframe
    Do some preprocessing of the input data, such as fixing account allocations
    '''

    # df = pd.read_csv('../dfout_all.csv', dtype={15: str})

    #Fix jiaowa account info to ferrier rather than sbs
    mask = (df['User'] == 'jiaowa') & (df['Account'] == 'sbs')
    df.loc[mask, 'Account'] = 'ferrier'

    # Mapping of old account names to new ones
    account_mapping = {
        'scpslab206': 'scps', 
        'scpslab306': 'scps',
        'spacejam': 'scps',
        'phys414': 'scps',
        'students': 'scps',
        'cad': 'admin',
        'root': 'admin'
    }

    # Replace account names
    df['Account'] = df['Account'].replace(account_mapping)

    # currently we use a fix usd to nzd exchange rate to calculate aws cost
    usd_to_nzd = 1.62
    df.aws_cost = df.aws_cost * usd_to_nzd

    # set dates as datetime and create columns for month and year
    df['Start'] = pd.to_datetime(df['Start'])
    df['Submit'] = pd.to_datetime(df['Submit'])
    df['End'] = pd.to_datetime(df['End'])
    df['Year'] = df['Start'].dt.year
    df['Month'] = df['Start'].dt.month
    df['Wait'] = df['Start'] - df ['Submit']
    
    # memfix
    df['ReqMem'] = df['ReqMem'].apply(memfix)

    return df

'''
Preprocess and then plot all the raapoi user and estimated cost data.
'''
df = pd.read_csv('raapoi_metrics/raapoi_data.csv', dtype={15: str})
df = preprocess_data(df)

# Create a common plot function with consistent aesthetics
def common_plot(df: pd.DataFrame, x: str, y: str, title: str, xlabel: str, ylabel: str, color: str = 'blue', kind: str = 'line', ax: Optional[plt.Axes] = None, ylim: Optional[tuple] = None):
    '''
    Common plotting function for consistent aesthetics.
    Can plot on a specific axes if 'ax' is provided, otherwise creates a new figure.
    '''
    if ax is None:
        plt.figure(figsize=(12, 6))
        current_ax = plt.gca()
    else:
        current_ax = ax

    if kind == 'line':
        sns.lineplot(data=df, x=x, y=y, color=color, ax=current_ax)
    # elif kind == 'bar':
    #     sns.barplot(data=df, x=x, y=y, color=color, ax=current_ax)
    elif kind == 'bar':
        barplot_container = sns.barplot(data=df, x=x, y=y, color=color, ax=current_ax)
        show_bar_labels = True
        bar_label_fmt = '{:.0f}'  # Format for bar labels
        if show_bar_labels:
            # Check if bar_label is available (Matplotlib 3.4+)
            if hasattr(current_ax, 'bar_label'):
                current_ax.bar_label(barplot_container.containers[0], fmt=bar_label_fmt, padding=3, rotation=90)
            else:
                # Fallback for older Matplotlib versions (less ideal positioning)
                for p in current_ax.patches:
                    current_ax.text(p.get_x() + p.get_width() / 2.,
                                    p.get_height(),
                                    f'{p.get_height():.2f}',
                                    ha='center', va='bottom', fontsize=9, color='black')



    current_ax.set_title(title)
    current_ax.set_xlabel(xlabel)
    current_ax.set_ylabel(ylabel)
    current_ax.tick_params(axis='x', rotation=90)
    # current_ax.set_ylim(0, 200)

    # Apply y-limit if provided
    if ylim is not None:
        current_ax.set_ylim(ylim[0], ylim[1])
        
    if ax is None:
        plt.tight_layout()
        plt.show()
        


In [ ]:
# Aggregate data for jobs per partition
jobs_per_partition = df['Partition'].value_counts().reset_index()
jobs_per_partition.columns = ['Partition', 'JobCount']

# Calculate total number of jobs
total_jobs = jobs_per_partition['JobCount'].sum()

# Calculate percentage of jobs for each partition
jobs_per_partition['Percentage'] = (jobs_per_partition['JobCount'] / total_jobs) * 100

# Display the percentage distribution in a table
print("Percentage Distribution of Jobs per Partition:")
print(jobs_per_partition.sort_values(by='Percentage', ascending=False).to_string(index=False))

# Plotting percentage distribution as a pie chart
plt.figure(figsize=(10, 8))
plt.pie(jobs_per_partition['Percentage'], labels=jobs_per_partition['Partition'], autopct='%1.1f%%', startangle=140, pctdistance=0.85)
plt.title('Percentage Distribution of Jobs per Partition')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.tight_layout()
plt.show()


In [ ]:
def plot_jobs_over_time_per_partition(df: pd.DataFrame):
    """
    Plots the number of jobs for each partition over time.
    The time unit is derived from the 'Start' column (Year-Month).

    Args:
        df (pd.DataFrame): The input DataFrame containing job data.
                          Must have 'Start' (datetime) and 'Partition' columns.
    """
    # Ensure 'Start' column is datetime, crucial for time-based plotting
    if not pd.api.types.is_datetime64_any_dtype(df['Start']):
        print("Warning: 'Start' column is not datetime. Attempting conversion.")
        df['Start'] = pd.to_datetime(df['Start'], errors='coerce')
        df.dropna(subset=['Start'], inplace=True) # Drop rows where conversion failed

    # Create a 'YearMonth' column for chronological ordering
    df['YearMonth'] = df['Start'].dt.to_period('M').astype(str)

    # Aggregate data: count jobs per partition per YearMonth
    jobs_over_time = df.groupby(['YearMonth', 'Partition']).size().reset_index(name='JobCount')

    # Create the plot using seaborn.
    # hue='Partition' will create a separate line for each partition.
    plt.figure(figsize=(15, 8)) # Increased figure size for better readability
    sns.lineplot(
        data=jobs_over_time,
        x='YearMonth',
        y='JobCount',
        hue='Partition',
        marker='o', # Add markers to points
        ax=plt.gca()
    )

    plt.title('Number of Jobs per Partition Over Time')
    plt.xlabel('Time (Year-Month)')
    plt.ylabel('Number of Jobs')
    plt.xticks(rotation=90) # Rotate x-axis labels for better readability
    plt.grid(True, linestyle='--', alpha=0.7) # Add a grid for easier reading
    plt.legend(title='Partition', bbox_to_anchor=(1.05, 1), loc='upper left') # Place legend outside plot area
    plt.tight_layout()
    plt.show()
    # plt.savefig('number_of_jobs_per_partition_over_time.png')
    plt.close() # Close the figure to free memory
    print("Plot 'number_of_jobs_per_partition_over_time.png' saved successfully.")

plot_jobs_over_time_per_partition(df)

In [ ]:
from typing import Optional, List

def plot_jobs_over_time_per_partition(df: pd.DataFrame, partitions: Optional[List[str]] = None):
    """
    Plots the number of jobs for specified partitions over time as a bar chart,
    using the common_plot function for consistent aesthetics.
    The time unit is derived from the 'Start' column (Year-Month).

    Args:
        df (pd.DataFrame): The input DataFrame containing job data.
                          Must have 'Start' (datetime) and 'Partition' columns.
        partitions (Optional[List[str]]): A list of partition names to plot.
                                          If None, all partitions will be plotted.
    """
    # Ensure 'Start' column is datetime, crucial for time-based plotting
    if not pd.api.types.is_datetime64_any_dtype(df['Start']):
        print("Warning: 'Start' column is not datetime. Attempting conversion.")
        df['Start'] = pd.to_datetime(df['Start'], errors='coerce')
        df.dropna(subset=['Start'], inplace=True) # Drop rows where conversion failed

    # Filter by specified partitions if the argument is provided
    if partitions:
        df_filtered = df[df['Partition'].isin(partitions)].copy()
        if df_filtered.empty:
            print(f"No data found for the specified partitions: {', '.join(partitions)}")
            return
    else:
        df_filtered = df.copy()


    # Create a 'YearMonth' column for chronological ordering
    df_filtered['YearMonth'] = df_filtered['Start'].dt.to_period('M').astype(str)

    # Aggregate data: count jobs per partition per YearMonth
    jobs_over_time = df_filtered.groupby(['YearMonth', 'Partition']).size().reset_index(name='JobCount')

    # Create a combined 'YearMonth-Partition' for x-axis labels
    jobs_over_time['TimePartition'] = jobs_over_time['YearMonth'] + ' - ' + jobs_over_time['Partition']

    title_suffix = f" for {', '.join(partitions)}" if partitions else " for All Partitions"
    plot_title = f'Number of Jobs per Partition Over Time (Bar Chart){title_suffix}'
    file_name = f'number_of_jobs_per_partition_over_time_bar_chart{title_suffix.replace(" ", "_").replace(",", "").lower()}.png'

    common_plot(
        df=jobs_over_time,
        x='TimePartition',
        y='JobCount',
        title=plot_title,
        xlabel='Time (Year-Month) - Partition',
        ylabel='Number of Jobs',
        color='skyblue', # Default color for now, common_plot doesn't support hue directly
        kind='bar'
    )
    print(f"Plot '{file_name}' saved successfully.")
    
    
# Call the function
plot_jobs_over_time_per_partition(df, partitions=['parallel'])


In [ ]:
# print header and first row

# print 1st row as list
print("DataFrame Header:")
print(df.columns.tolist())
print("First Row as List:")
# Convert the first row to a list and print it
print(df.head(1).values.tolist()[0])


In [ ]:
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Optional, Dict

def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Load the dataframe
    Do some preprocessing of the input data, such as fixing account allocations
    '''
    # Make a copy to avoid SettingWithCopyWarning
    df = df.copy()

    # Fix jiaowa account info to ferrier rather than sbs
    mask = (df['User'] == 'jiaowa') & (df['Account'] == 'sbs')
    df.loc[mask, 'Account'] = 'ferrier'

    # Mapping of old account names to new ones
    account_mapping = {
        'scpslab206': 'scps',
        'scpslab306': 'scps',
        'spacejam': 'scps',
        'phys414': 'scps',
        'students': 'scps',
        'cad': 'admin',
        'root': 'admin'
    }

    # Replace account names
    df['Account'] = df['Account'].replace(account_mapping)

    # Currently we use a fixed USD to NZD exchange rate to calculate AWS cost
    usd_to_nzd = 1.62
    df['aws_cost'] = df['aws_cost'] * usd_to_nzd

    # Set dates as datetime and create columns for month and year
    # Ensure columns exist before accessing .dt
    if 'Start' in df.columns:
        df['Start'] = pd.to_datetime(df['Start'])
        df['Year'] = df['Start'].dt.year
        df['Month'] = df['Start'].dt.month
    if 'Submit' in df.columns:
        df['Submit'] = pd.to_datetime(df['Submit'])
    if 'End' in df.columns:
        df['End'] = pd.to_datetime(df['End'])

    return df

def generate_plot(df: pd.DataFrame, x_column: str, title: str, subtitle: str, filename: str, width: Optional[float] = None) -> plt.Figure:
    """
    Generates a bar plot using Matplotlib/Seaborn.

    Args:
        df (pd.DataFrame): DataFrame containing the data to plot.
        x_column (str): Name of the column to use for the x-axis (e.g., 'YearMonth').
        title (str): Main title of the plot.
        subtitle (str): Subtitle of the plot.
        filename (str): Desired filename for saving the plot (not used for display).
        width (Optional[float]): Placeholder for bar width; Matplotlib handles this more
                                 automatically or through `width` parameter in `plt.bar`.

    Returns:
        plt.Figure: The Matplotlib Figure object containing the plot.
    """
    # Determine figure size based on number of unique x-axis values for better readability
    num_x_values = len(df[x_column].unique())
    fig_width = max(8, num_x_values * 0.5) # Min width 8, scale up with more bars
    fig_height = 6

    fig, ax = plt.subplots(figsize=(fig_width, fig_height))

    # Use Seaborn for a nice bar plot
    # The 'UniqueUsers' column will automatically determine bar height and color intensity
    # using a sequential colormap.
    sns.barplot(x=x_column, y='UniqueUsers', data=df, color='skyblue', ax=ax) # Reds_d gives red gradient

    # Add labels and titles
    ax.set_xlabel('Date')
    ax.set_ylabel('Unique Users')
    ax.set_title(f"{title}\n{subtitle}", loc='center', wrap=True) # Combined title and subtitle, centered

    # Rotate x-axis labels for better readability if they are dates or long strings
    if df[x_column].dtype == '<M8[ns]': # Check if x_column is datetime
        fig.autofmt_xdate(rotation=45, ha='right') # Automatically format and rotate date labels
    else:
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right') # General rotation for other types
        
    # Set y-axis limits: from 0 to slightly above the maximum 'UniqueUsers' for padding
    max_users = df['UniqueUsers'].max()
    ax.set_ylim(0, max_users * 1.10) # 10% padding above max value

    # Improve layout to prevent labels from overlapping
    plt.tight_layout()

    # Saving the plot is optional here, as the primary goal is display.
    # If you still want to save, uncomment the line below.
    # plt.savefig(filename, dpi=300, bbox_inches='tight')

    # Close the plot to prevent it from being displayed twice if plt.show() is called later
    # plt.close(fig) # Commenting out plt.close() as we want to return the figure for display

    return fig


def plot_unique_users_per_month(df: pd.DataFrame) -> Dict[str, plt.Figure]:
    """
    Generates and saves monthly unique user plots for each account and a total.

    Args:
        df (pd.DataFrame): The input DataFrame containing 'Account', 'Year', 'Month', and 'User' columns.

    Returns:
        Dict[str, plt.Figure]: A dictionary where keys are plot names (e.g., 'Account A Monthly Users')
                               and values are the generated Matplotlib Figure objects.
                               These objects can then be displayed in an interactive environment.
    """
    all_plots = {} # Dictionary to store all generated Matplotlib Figure objects

    # Group by 'Account', 'Year', 'Month' and 'User', then count unique 'User'
    # This step ensures each user within an account for a given month is counted only once.
    unique_users = df.groupby(['Account', 'Year', 'Month', 'User'], dropna=False).size().reset_index().rename(columns={0:'count'})
    print("Head of unique_users after initial grouping:")
    print(unique_users.head())

    # Now group by 'Account', 'Year' and 'Month' and count unique 'User'
    # This gives us the total unique users per account per month.
    unique_users_per_month = unique_users.groupby(['Account', 'Year', 'Month'], dropna=False).size().reset_index().rename(columns={0:'UniqueUsers'})

    # Convert 'Year' and 'Month' to integer, then to string, combine them, and convert to datetime
    # This creates a proper datetime object for plotting on the x-axis.
    # Handle potential NaN values from grouping (e.g., if 'Year' or 'Month' were missing)
    unique_users_per_month = unique_users_per_month.dropna(subset=['Year', 'Month']) # Drop rows where Year/Month might be NaN after grouping
    unique_users_per_month['YearMonth'] = pd.to_datetime(
        unique_users_per_month['Year'].astype(int).astype(str) + '-' +
        unique_users_per_month['Month'].astype(int).astype(str)
    )

    # Capitalize 'Account' names for consistent labeling and file naming
    unique_users_per_month['Account'] = unique_users_per_month['Account'].str.upper()

    accounts = unique_users_per_month['Account'].unique()

    # Create the directory for plots if it doesn't already exist
    plot_dir = 'plots/monthly_users'
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)
        print(f"Created directory: {plot_dir}")

    # Generate and store plots for each individual account
    print("\nGenerating plots for individual accounts...")
    for account in accounts:
        account_data = unique_users_per_month[unique_users_per_month['Account'] == account].copy() # Use .copy() to avoid SettingWithCopyWarning
        plot_title_str = f'Unique {account} Users Per Month'
        file_path = os.path.join(plot_dir, f'{account.replace(" ", "_").lower()}_users_per_month.png')

        # Call generate_plot and store the returned Matplotlib Figure object
        current_fig = generate_plot(
            df=account_data,
            x_column='YearMonth',
            title='Rāpoi', # Assuming 'Rāpoi' is a constant title or placeholder
            subtitle=plot_title_str,
            filename=file_path
        )
        all_plots[plot_title_str] = current_fig
        print(f"Generated plot for {account}: {file_path}")

    # Produce the total unique users per month plot
    start_time = time.time()

    # For the total unique users per month, group the aggregated unique_users_per_month
    # DataFrame by YearMonth and sum the 'UniqueUsers' across all accounts.
    total_users_per_month = unique_users_per_month.groupby(['YearMonth'])['UniqueUsers'].sum().reset_index()

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'\nCreating total unique users data took: {elapsed_time:.4f} seconds')

    total_plot_title_str = 'Total Unique Users Per Month'
    total_file_path = os.path.join(plot_dir, 'total_users_per_month.png')
    total_fig = generate_plot(
        df=total_users_per_month,
        x_column='YearMonth',
        title='Rāpoi', # Assuming 'Rāpoi' is a constant title or placeholder
        subtitle=total_plot_title_str,
        filename=total_file_path
    )
    all_plots[total_plot_title_str] = total_fig
    print(f"Generated total unique users plot: {total_file_path}")

    return all_plots

def plot_all_slurm():
    '''
    Preprocess and then plot all the raapoi user and estimated cost data.
    '''
    # Load your data
    # IMPORTANT: Replace 'data.csv' with the actual path to your CSV file.
    # If running in the same directory as the script, 'data.csv' is fine.
    # If it's a different path (e.g., from the provided example '/nfs/scratch/duggalro/ex_python/raapoi_metrics/raapoi_data.csv'), use that.
    try:
        df = pd.read_csv('/nfs/scratch/duggalro/ex_python/raapoi_metrics/raapoi_data.csv', dtype={15: str})
        print("Loaded data.csv")
    except FileNotFoundError:
        print("data.csv not found. Please ensure the CSV file is in the correct directory or update the path.")
        return # Exit if data isn't found

    df = preprocess_data(df)

    # Get the dictionary of generated plots
    generated_plots = plot_unique_users_per_month(df)

    # Iterate through the plots and display each one
    print("\n--- Displaying all generated plots ---")
    for plot_name, plot_fig in generated_plots.items():
        print(f"\n--- Displaying: {plot_name} ---")
        # Display the figure in the Jupyter Notebook
        # plt.show() is often not strictly necessary in a notebook if the figure is the last output,
        # but it can ensure proper rendering, especially if you have multiple figures.
        # plt.show() also closes the figure automatically after display.
        # If you want to keep the figure open for further manipulation, you can omit plt.show()
        # and simply have 'plot_fig' as the last line in a cell.
        plt.show(plot_fig) # Pass the figure object to show

# Call the main function to run the process
plot_all_slurm()


In [ ]:
def plot_unique_users_per_year(df: pd.DataFrame) -> Dict[str, plt.Figure]:
    """
    Generates and saves yearly unique user plots for each account and a total.

    Args:
        df (pd.DataFrame): The input DataFrame containing 'Account', 'Year', and 'User' columns.

    Returns:
        Dict[str, plt.Figure]: A dictionary where keys are plot names (e.g., 'Account A Yearly Users')
                               and values are the generated Matplotlib Figure objects.
    """
    all_plots_yearly = {}

    unique_users_per_year_initial = df.groupby(['Account', 'Year', 'User'], dropna=False).size().reset_index().rename(columns={0:'count'})

    # Now group by 'Account' and 'Year' and count unique 'User'
    unique_users_per_year = unique_users_per_year_initial.groupby(['Account', 'Year'], dropna=False).size().reset_index().rename(columns={0:'UniqueUsers'})
    
    # Capitalize 'Account'
    unique_users_per_year['Account'] = unique_users_per_year['Account'].str.upper()

    accounts = unique_users_per_year['Account'].unique()

    # Create the directory if it doesn't already exist
    plot_dir = 'plots/yearly_users'
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)
        print(f"Created directory: {plot_dir}")

    print("\nGenerating plots for individual accounts (yearly)...")
    for account in accounts:
        account_data = unique_users_per_year[unique_users_per_year['Account'] == account].copy()
        plot_title_str = f'Unique {account} Users Per Year'
        file_path = os.path.join(plot_dir, f'{account.replace(" ", "_").lower()}_users_per_year.png')

        current_fig = generate_plot(
            df=account_data,
            x_column='Year',
            title='Rāpoi',
            subtitle=plot_title_str,
            filename=file_path
        )
        all_plots_yearly[plot_title_str] = current_fig
        print(f"Generated yearly plot for {account}: {file_path}")

    # Produce the total unique users per year
    start_time = time.time()
    total_users_per_year = unique_users_per_year.groupby(['Year'])['UniqueUsers'].sum().reset_index()
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'Creating total unique users took: {elapsed_time:.4f} seconds')

    total_plot_title_str = 'Total Unique Users Per Year'
    total_file_path = os.path.join(plot_dir, 'total_users_per_year.png')
    total_fig = generate_plot(
        df=total_users_per_year,
        x_column='Year',
        title='Rāpoi',
        subtitle=total_plot_title_str,
        filename=total_file_path
    )
    all_plots_yearly[total_plot_title_str] = total_fig
    print(f"Generated total unique users yearly plot: {total_file_path}")

    return all_plots_yearly

# Get the dictionary of yearly generated plots
generated_yearly_plots = plot_unique_users_per_year(df)
# Iterate through all yearly plots and display each one
print("\n--- Displaying all generated YEARLY plots ---")
for plot_name, plot_fig in generated_yearly_plots.items():
    print(f"\n--- Displaying: {plot_name} ---")
    plt.show(plot_fig) # Pass the figure object to show


In [ ]:
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Optional, Dict

def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Load the dataframe
    Do some preprocessing of the input data, such as fixing account allocations
    '''
    # Make a copy to avoid SettingWithCopyWarning
    df = df.copy()

    # Fix jiaowa account info to ferrier rather than sbs
    mask = (df['User'] == 'jiaowa') & (df['Account'] == 'sbs')
    df.loc[mask, 'Account'] = 'ferrier'

    # Mapping of old account names to new ones
    account_mapping = {
        'scpslab206': 'scps',
        'scpslab306': 'scps',
        'spacejam': 'scps',
        'phys414': 'scps',
        'students': 'scps',
        'cad': 'admin',
        'root': 'admin'
    }

    # Replace account names
    df['Account'] = df['Account'].replace(account_mapping)

    # Currently we use a fixed USD to NZD exchange rate to calculate AWS cost
    usd_to_nzd = 1.62
    df['aws_cost'] = df['aws_cost'] * usd_to_nzd

    # Set dates as datetime and create columns for month and year
    # Ensure columns exist before accessing .dt
    if 'Start' in df.columns:
        df['Start'] = pd.to_datetime(df['Start'])
        df['Year'] = df['Start'].dt.year
        df['Month'] = df['Start'].dt.month
    if 'Submit' in df.columns:
        df['Submit'] = pd.to_datetime(df['Submit'])
    if 'End' in df.columns:
        df['End'] = pd.to_datetime(df['End'])

    return df

def generate_plot(df: pd.DataFrame, x_column: str, title: str, subtitle: str, filename: str, width: Optional[float] = None) -> plt.Figure:
    """
    Generates a bar plot using Matplotlib/Seaborn.

    Args:
        df (pd.DataFrame): DataFrame containing the data to plot.
        x_column (str): Name of the column to use for the x-axis (e.g., 'YearMonth').
        title (str): Main title of the plot (e.g., 'Rāpoi').
        subtitle (str): Subtitle of the plot (e.g., 'Unique Account A Users Per Month').
        filename (str): Desired filename for saving the plot (not used for display).
        width (Optional[float]): Placeholder for bar width; Matplotlib handles this more
                                 automatically or through `width` parameter in `plt.bar`.

    Returns:
        plt.Figure: The Matplotlib Figure object containing the plot.
    """
    # Use a fixed, decent figure size for consistent bar appearance
    # Seaborn will adjust bar widths to fit this figure size.
    fig_width = 10
    fig_height = 6

    fig, ax = plt.subplots(figsize=(fig_width, fig_height))

    # Use Seaborn for a nice bar plot with a single soft color
    sns.barplot(x=x_column, y='UniqueUsers', data=df, color='skyblue', ax=ax)

    # Set self-defining x-axis label
    x_label_map = {
        'YearMonth': 'Date (Year-Month)',
        'Year': 'Year'
    }
    ax.set_xlabel(x_label_map.get(x_column, 'Date')) # Default to 'Date' if x_column not in map
    ax.set_ylabel('Unique Users')

    # Create a more elaborative combined title
    main_title_prefix = f"{title} - Unique User Statistics"
    full_plot_title = f"{main_title_prefix}\n{subtitle}"
    ax.set_title(full_plot_title, loc='center', wrap=True, fontsize=14)

    # Rotate x-axis labels for better readability if they are dates or long strings
    if df[x_column].dtype == '<M8[ns]': # Check if x_column is datetime
        fig.autofmt_xdate(rotation=45, ha='right') # Automatically format and rotate date labels
    else:
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right') # General rotation for other types

    # Set y-axis limits: from 0 to slightly above the maximum 'UniqueUsers' for padding
    max_users = df['UniqueUsers'].max()
    ax.set_ylim(0, max_users * 1.10) # 10% padding above max value


    # Improve layout to prevent labels from overlapping
    plt.tight_layout()

    # Saving the plot is optional here, as the primary goal is display.
    # If you still want to save, uncomment the line below.
    # plt.savefig(filename, dpi=300, bbox_inches='tight')

    return fig


def plot_unique_users_per_month(df: pd.DataFrame) -> Dict[str, plt.Figure]:
    """
    Generates and saves monthly unique user plots for each account and a total.

    Args:
        df (pd.DataFrame): The input DataFrame containing 'Account', 'Year', 'Month', and 'User' columns.

    Returns:
        Dict[str, plt.Figure]: A dictionary where keys are plot names (e.g., 'Account A Monthly Users')
                               and values are the generated Matplotlib Figure objects.
                               These objects can then be displayed in an interactive environment.
    """
    all_plots = {} # Dictionary to store all generated Matplotlib Figure objects

    # Group by 'Account', 'Year', 'Month' and 'User', then count unique 'User'
    # This step ensures each user within an account for a given month is counted only once.
    unique_users = df.groupby(['Account', 'Year', 'Month', 'User'], dropna=False).size().reset_index().rename(columns={0:'count'})
    print("Head of unique_users after initial grouping:")
    print(unique_users.head())

    # Now group by 'Account', 'Year' and 'Month' and count unique 'User'
    # This gives us the total unique users per account per month.
    unique_users_per_month = unique_users.groupby(['Account', 'Year', 'Month'], dropna=False).size().reset_index().rename(columns={0:'UniqueUsers'})

    # Convert 'Year' and 'Month' to integer, then to string, combine them, and convert to datetime
    # This creates a proper datetime object for plotting on the x-axis.
    # Handle potential NaN values from grouping (e.g., if 'Year' or 'Month' were missing)
    unique_users_per_month = unique_users_per_month.dropna(subset=['Year', 'Month']) # Drop rows where Year/Month might be NaN after grouping
    unique_users_per_month['YearMonth'] = pd.to_datetime(
        unique_users_per_month['Year'].astype(int).astype(str) + '-' +
        unique_users_per_month['Month'].astype(int).astype(str)
    )

    # Capitalize 'Account' names for consistent labeling and file naming
    unique_users_per_month['Account'] = unique_users_per_month['Account'].str.upper()

    accounts = unique_users_per_month['Account'].unique()

    # Create the directory for plots if it doesn't already exist
    plot_dir = 'plots/monthly_users'
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)
        print(f"Created directory: {plot_dir}")

    # Generate and store plots for each individual account
    print("\nGenerating plots for individual accounts...")
    for account in accounts:
        account_data = unique_users_per_month[unique_users_per_month['Account'] == account].copy() # Use .copy() to avoid SettingWithCopyWarning
        plot_title_str = f'Unique {account} Users Per Month'
        file_path = os.path.join(plot_dir, f'{account.replace(" ", "_").lower()}_users_per_month.png')

        # Call generate_plot and store the returned Matplotlib Figure object
        current_fig = generate_plot(
            df=account_data,
            x_column='YearMonth',
            title='Rāpoi', # Assuming 'Rāpoi' is a constant title or placeholder
            subtitle=plot_title_str,
            filename=file_path
        )
        all_plots[plot_title_str] = current_fig
        print(f"Generated plot for {account}: {file_path}")

    # Produce the total unique users per month plot
    start_time = time.time()

    # For the total unique users per month, group the aggregated unique_users_per_month
    # DataFrame by YearMonth and sum the 'UniqueUsers' across all accounts.
    total_users_per_month = unique_users_per_month.groupby(['YearMonth'])['UniqueUsers'].sum().reset_index()

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'\nCreating total unique users took: {elapsed_time:.4f} seconds')

    total_plot_title_str = 'Total Unique Users Per Month'
    total_file_path = os.path.join(plot_dir, 'total_users_per_month.png')
    total_fig = generate_plot(
        df=total_users_per_month,
        x_column='YearMonth',
        title='Rāpoi', # Assuming 'Rāpoi' is a constant title or placeholder
        subtitle=total_plot_title_str,
        filename=total_file_path
    )
    all_plots[total_plot_title_str] = total_fig
    print(f"Generated total unique users plot: {total_file_path}")

    return all_plots

def plot_unique_users_per_year(df: pd.DataFrame) -> Dict[str, plt.Figure]:
    """
    Generates and saves yearly unique user plots for each account and a total.

    Args:
        df (pd.DataFrame): The input DataFrame containing 'Account', 'Year', and 'User' columns.

    Returns:
        Dict[str, plt.Figure]: A dictionary where keys are plot names (e.g., 'Account A Yearly Users')
                               and values are the generated Matplotlib Figure objects.
    """
    all_plots_yearly = {}

    unique_users_per_year_initial = df.groupby(['Account', 'Year', 'User'], dropna=False).size().reset_index().rename(columns={0:'count'})

    # Now group by 'Account' and 'Year' and count unique 'User'
    unique_users_per_year = unique_users_per_year_initial.groupby(['Account', 'Year'], dropna=False).size().reset_index().rename(columns={0:'UniqueUsers'})

    # Capitalize 'Account'
    unique_users_per_year['Account'] = unique_users_per_year['Account'].str.upper()

    accounts = unique_users_per_year['Account'].unique()

    # Create the directory if it doesn't already exist
    plot_dir = 'plots/yearly_users'
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)
        print(f"Created directory: {plot_dir}")

    print("\nGenerating plots for individual accounts (yearly)...")
    for account in accounts:
        account_data = unique_users_per_year[unique_users_per_year['Account'] == account].copy()
        plot_title_str = f'Unique {account} Users Per Year'
        file_path = os.path.join(plot_dir, f'{account.replace(" ", "_").lower()}_users_per_year.png')

        current_fig = generate_plot(
            df=account_data,
            x_column='Year',
            title='Rāpoi',
            subtitle=plot_title_str,
            filename=file_path
        )
        all_plots_yearly[plot_title_str] = current_fig
        print(f"Generated yearly plot for {account}: {file_path}")

    # Produce the total unique users per year
    start_time = time.time()
    total_users_per_year = unique_users_per_year.groupby(['Year'])['UniqueUsers'].sum().reset_index()
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'Creating total unique users took: {elapsed_time:.4f} seconds')

    total_plot_title_str = 'Total Unique Users Per Year'
    total_file_path = os.path.join(plot_dir, 'total_users_per_year.png')
    total_fig = generate_plot(
        df=total_users_per_year,
        x_column='Year',
        title='Rāpoi',
        subtitle=total_plot_title_str,
        filename=total_file_path
    )
    all_plots_yearly[total_plot_title_str] = total_fig
    print(f"Generated total unique users yearly plot: {total_file_path}")

    return all_plots_yearly


def plot_all_slurm():
    '''
    Preprocess and then plot all the raapoi user and estimated cost data.
    '''
    # Load your data
    # IMPORTANT: Replace 'data.csv' with the actual path to your CSV file.
    # If running in the same directory as the script, 'data.csv' is fine.
    # If it's a different path (e.g., from the provided example '/nfs/scratch/duggalro/ex_python/raapoi_metrics/raapoi_data.csv'), use that.
    try:
        df = pd.read_csv('/nfs/scratch/duggalro/ex_python/raapoi_metrics/raapoi_data.csv', dtype={15: str})
        print("Loaded data.csv")
    except FileNotFoundError:
        print("data.csv not found. Please ensure the CSV file is in the correct directory or update the path.")
        return # Exit if data isn't found

    df = preprocess_data(df)

    # Get the dictionary of monthly generated plots
    generated_monthly_plots = plot_unique_users_per_month(df)

    # Get the dictionary of yearly generated plots
    generated_yearly_plots = plot_unique_users_per_year(df)

    # Iterate through all monthly plots and display each one
    print("\n--- Displaying all generated MONTHLY plots ---")
    for plot_name, plot_fig in generated_monthly_plots.items():
        print(f"\n--- Displaying: {plot_name} ---")
        plt.show(plot_fig) # Pass the figure object to show

    # Iterate through all yearly plots and display each one
    print("\n--- Displaying all generated YEARLY plots ---")
    for plot_name, plot_fig in generated_yearly_plots.items():
        print(f"\n--- Displaying: {plot_name} ---")
        plt.show(plot_fig) # Pass the figure object to show


# Call the main function to run the process
plot_all_slurm()


In [ ]:


def plot_unique_jobs_per_account_by_year(df: pd.DataFrame):
    '''
    Plots the number of unique jobs per account by year,
    specifically for jobs with wait time >= 4 hours and count < 100.
    Each year gets a subplot.
    '''
    # Ensure 'Wait' column is timedelta type.
    if 'Wait' in df.columns and not pd.api.types.is_timedelta64_dtype(df['Wait']):
        try:
            df['Wait'] = pd.to_timedelta(df['Wait'])
        except Exception as e:
            print(f"Warning: Could not convert 'Wait' column to timedelta. Error: {e}")
            print("Skipping unique jobs per account analysis.")
            return

    if 'Wait' not in df.columns:
        print("The 'Wait' column is not available or could not be processed for unique jobs per account analysis.")
        return

    # Filter for jobs with wait time >= 4 hours (14400 seconds)
    df_long_wait = df[df['Wait'].dt.total_seconds() >= 14400].copy()

    # Group by Year and Account, then count unique JobIDs
    # 'JobID' is assumed to be the unique job identifier
    if 'JobID' not in df_long_wait.columns:
        print("Error: 'JobID' column not found. Cannot count unique jobs.")
        return

    unique_jobs_per_account_year = df_long_wait.groupby(['Year', 'Account'])['JobID'].nunique().reset_index(name='UniqueJobCount')

    # Filter for counts less than 400
    # unique_jobs_per_account_year_filtered = unique_jobs_per_account_year[unique_jobs_per_account_year['UniqueJobCount'] < 400]

    # Get unique years for subplots
    # years = sorted(unique_jobs_per_account_year_filtered['Year'].unique())
    years= sorted(unique_jobs_per_account_year['Year'].unique())

    if not years:
        print("No data available after filtering for unique jobs with wait time >= 4 hours and count < 400.")
        return

    # Determine grid size for subplots
    n_years = len(years)
    cols = 2  # Max 2 columns for subplots
    rows = (n_years + cols - 1) // cols # Calculate rows needed

    fig, axes = plt.subplots(rows, cols, figsize=(15, 6 * rows), squeeze=False)
    axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

    for i, year in enumerate(years):
        # data_for_year = unique_jobs_per_account_year_filtered[unique_jobs_per_account_year_filtered['Year'] == year]
        data_for_year = unique_jobs_per_account_year[unique_jobs_per_account_year['Year'] == year]
        if not data_for_year.empty:
            common_plot(data_for_year, 'Account', 'UniqueJobCount',
                        f'Jobs (Wait >= 4hr) in {int(year)}',
                        'Account', 'Number of Jobs', color='teal', kind='bar', ax=axes[i])
            # yscale is log
            axes[i].set_yscale('log')
        else:
            # Hide empty subplots
            axes[i].set_visible(False)
            # 

    # Hide any unused subplots if n_years is not a perfect multiple of cols
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    # super title
    plt.suptitle('Number of Unique Jobs per Account by Year (Wait >= 4hr)', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Create and plot number of unique jobs per account by year
plot_unique_jobs_per_account_by_year(df)

In [ ]:
df.columns

In [ ]:
def plot_unique_jobs_per_account_all_years(df: pd.DataFrame):
    '''
    Plots the number of unique jobs per account for each year in separate subplots.
    Each subplot visualizes the distribution of unique job counts across different
    accounts for a specific year. This function does not filter based on 'Wait' time
    or job count thresholds, showing all unique jobs.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data,
                         expected to have 'Year', 'Account', and 'JobID' columns.
    '''
    # Basic data validation
    if 'JobID' not in df.columns:
        print("Error: 'JobID' column not found. Cannot count unique jobs.")
        return
    if 'Year' not in df.columns:
        print("Error: 'Year' column not found. Please ensure data preprocessing has been run (df['Start'] converted to datetime).")
        return
    if 'Account' not in df.columns:
        print("Error: 'Account' column not found. Please ensure data preprocessing has been run.")
        return

    # Group by Year and Account, then count unique JobIDs
    # This captures the total unique jobs for each account in each year
    unique_jobs_per_account_year = df.groupby(['Year', 'Account'])['JobID'].nunique().reset_index(name='UniqueJobCount')

    # Get a sorted list of unique years present in the data for subplot creation
    years = sorted(unique_jobs_per_account_year['Year'].unique())

    if not years:
        print("No data available to plot unique jobs per account by year after grouping.")
        return

    # Determine the layout for subplots: 2 columns, rows calculated based on number of years
    n_years = len(years)
    cols = 2
    rows = (n_years + cols - 1) // cols

    # Create the figure and subplots
    fig, axes = plt.subplots(rows, cols, figsize=(15, 6 * rows), squeeze=False)
    axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

    # Iterate through each year and create a subplot
    for i, year in enumerate(years):
        # Filter data for the current year
        data_for_year = unique_jobs_per_account_year[unique_jobs_per_account_year['Year'] == year]

        if not data_for_year.empty:
            # Use common_plot to draw a bar plot for the current year
            common_plot(data_for_year, 'Account', 'UniqueJobCount',
                        f'Unique Jobs by Account in {int(year)}', # Title for each subplot
                        'Account', 'Number of Unique Jobs',
                        color='steelblue', # Consistent color for these plots
                        kind='bar',
                        ax=axes[i]) # Pass the specific axes for the subplot
            # yscale is log
            axes[i].set_yscale('log')
        else:
            # If a year has no data after filtering (unlikely with this grouping, but good practice),
            # hide its subplot to avoid empty frames.
            axes[i].set_visible(False)

    # Hide any remaining unused subplots if the total number of years doesn't fill the grid perfectly
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    
    # Adjust layout to prevent overlapping titles/labels
    plt.tight_layout()
    plt.show()
    
plot_unique_jobs_per_account_all_years(df)

In [ ]:
def plot_unique_users_per_month_all_years(df: pd.DataFrame):
    '''
    Plots the number of unique users per month for each year in separate subplots.
    Each subplot visualizes the distribution of unique user counts across different
    months for a specific year.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data,
                         expected to have 'Year', 'Month', and 'User' columns.
    '''
    # Basic data validation
    if 'User' not in df.columns:
        print("Error: 'User' column not found. Cannot count unique users.")
        return
    if 'Year' not in df.columns:
        print("Error: 'Year' column not found. Please ensure data preprocessing has been run (df['Start'] converted to datetime).")
        return
    if 'Month' not in df.columns:
        print("Error: 'Month' column not found. Please ensure data preprocessing has been run (df['Start'] converted to datetime).")
        return

    # Group by Year and Month, then count unique Users
    # This captures the total unique users for each month in each year
    unique_users_per_month_year = df.groupby(['Year', 'Month'])['User'].nunique().reset_index(name='UniqueUserCount')

    # Ensure Month is treated as a category or sorted correctly for plotting
    # You might want to map month numbers to names for better readability on plots
    month_names = {
        1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
        7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
    }
    unique_users_per_month_year['MonthName'] = unique_users_per_month_year['Month'].map(month_names)

    # Get a sorted list of unique years present in the data for subplot creation
    years = sorted(unique_users_per_month_year['Year'].unique())

    if not years:
        print("No data available to plot unique users per month by year after grouping.")
        return

    # Determine the layout for subplots: 2 columns, rows calculated based on number of years
    n_years = len(years)
    cols = 2
    rows = (n_years + cols - 1) // cols

    # Create the figure and subplots
    fig, axes = plt.subplots(rows, cols, figsize=(15, 6 * rows), squeeze=False)
    axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

    # Iterate through each year and create a subplot
    for i, year in enumerate(years):
        # Filter data for the current year
        # Sort by Month to ensure chronological order on the x-axis
        data_for_year = unique_users_per_month_year[unique_users_per_month_year['Year'] == year].sort_values(by='Month')

        if not data_for_year.empty:
            # Use common_plot to draw a bar plot for the current year
            common_plot(data_for_year, 'MonthName', 'UniqueUserCount',
                        f'Unique Users by Month in {int(year)}', # Title for each subplot
                        'Month', 'Number of Unique Users',
                        color='mediumseagreen', # A distinct consistent color for these plots
                        kind='bar',
                        ax=axes[i]) # Pass the specific axes for the subplot
        else:
            # If a year has no data after filtering, hide its subplot
            axes[i].set_visible(False)

    # Hide any remaining unused subplots if the total number of years doesn't fill the grid perfectly
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    # Adjust layout to prevent overlapping titles/labels
    plt.tight_layout()
    plt.show()
    
plot_unique_users_per_month_all_years(df)

In [ ]:
def plot_jobs_by_hour_of_day(df: pd.DataFrame):
    '''
    Plots the number of job submissions by hour of the day across all years.
    This helps identify peak usage hours for the system.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data,
                         expected to have a 'Submit' (datetime) column.
    '''
    # Basic data validation
    if 'Submit' not in df.columns:
        print("Error: 'Submit' column not found. Please ensure data preprocessing has been run.")
        return

    # Extract the hour from the 'Submit' datetime column
    df['HourOfDay'] = df['Submit'].dt.hour

    # Group by HourOfDay and count the number of jobs
    jobs_by_hour = df.groupby('HourOfDay').size().reset_index(name='JobCount')

    # Ensure all 24 hours are present, filling missing hours with 0 job count
    all_hours = pd.DataFrame({'HourOfDay': range(24)})
    jobs_by_hour = pd.merge(all_hours, jobs_by_hour, on='HourOfDay', how='left').fillna(0)

    # Plot the data using common_plot
    plt.figure(figsize=(12, 6)) # Create a single figure for this plot
    common_plot(jobs_by_hour, 'HourOfDay', 'JobCount',
                'Job Submissions by Hour of Day (All Years)',
                'Hour of Day (0-23)', 'Number of Jobs',
                color='purple', # A distinct color for this plot
                kind='bar')

    # Adjust x-axis ticks to show all hours clearly
    plt.xticks(range(0, 24))
    plt.tight_layout()
    plt.show()
    
plot_jobs_by_hour_of_day(df)

In [ ]:
def plot_unique_users_per_month(df: pd.DataFrame):
    """
    Plots the total number of unique users per month using a filled line plot.

    Args:
        df (pd.DataFrame): The input DataFrame containing job data.
                          Must have 'Start' (datetime) and 'User' columns.
    """
    # Ensure 'Start' column is datetime
    if not pd.api.types.is_datetime64_any_dtype(df['Start']):
        print("Warning: 'Start' column is not datetime. Attempting conversion.")
        df['Start'] = pd.to_datetime(df['Start'], errors='coerce')
        df.dropna(subset=['Start'], inplace=True)

    # Create a 'YearMonth' column for aggregation
    df['YearMonth'] = df['Start'].dt.to_period('M')

    # Group by YearMonth and count unique users
    unique_users_per_month = df.groupby('YearMonth')['User'].nunique().reset_index(name='UniqueUserCount')

    # Convert YearMonth back to string for plotting
    unique_users_per_month['YearMonth'] = unique_users_per_month['YearMonth'].astype(str)

    plt.figure(figsize=(12, 6))
    ax = plt.gca()

    # Use seaborn lineplot and then fill_between for the filled effect
    sns.lineplot(
        data=unique_users_per_month,
        x='YearMonth',
        y='UniqueUserCount',
        marker='o',
        color='teal',
        ax=ax
    )
    # Fill the area under the line
    ax.fill_between(
        unique_users_per_month['YearMonth'],
        unique_users_per_month['UniqueUserCount'],
        color='teal',
        alpha=0.3
    )

    ax.set_title('Total Unique Users Per Month')
    ax.set_xlabel('Time (Year-Month)')
    ax.set_ylabel('Number of Unique Users')
    ax.tick_params(axis='x', rotation=90)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    # plt.savefig('total_unique_users_per_month.png')
    plt.close()
    print("Plot 'total_unique_users_per_month.png' saved successfully.")
    

plot_unique_users_per_month(df)

In [ ]:
def plot_unique_users_per_account_over_time(df: pd.DataFrame, weeks: int = 1, accounts: Optional[List[str]] = None):
    '''
    Plots the total number of unique users per account over time in subplots.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have 'Start' (datetime), 'User', and 'Account' columns.
    - weeks (int): The number of past weeks for which to plot the data. Defaults to 1.
    - accounts (Optional[List[str]]): A list of account names to plot.
                                          If None, unique users for all *unique* accounts will be plotted
                                          in separate subplots.
    '''
    # Filter df for the specified number of weeks
    filter_date = pd.Timestamp.now() - pd.DateOffset(weeks=weeks)
    df_filtered_time = df[df['Start'] >= filter_date].copy()

    # Ensure 'Start' column is datetime
    if not pd.api.types.is_datetime64_any_dtype(df_filtered_time['Start']):
        print("Warning: 'Start' column is not datetime. Attempting conversion.")
        df_filtered_time['Start'] = pd.to_datetime(df_filtered_time['Start'], errors='coerce')
        df_filtered_time.dropna(subset=['Start'], inplace=True) # Drop rows where conversion failed

    # Determine which accounts to plot
    if accounts:
        accounts_to_plot = [a for a in accounts if a in df_filtered_time['Account'].unique()]
        if not accounts_to_plot:
            print(f"No data found for the specified accounts: {', '.join(accounts)} in the last {weeks} weeks after time filtering.")
            return
    else:
        accounts_to_plot = df_filtered_time['Account'].unique().tolist()
        if not accounts_to_plot:
            print(f"No accounts found in the data for the last {weeks} weeks.")
            return

    # Create a 'YearMonth' column for chronological ordering
    df_filtered_time['YearMonth'] = df_filtered_time['Start'].dt.to_period('M').astype(str)

    # Set up subplots
    num_accounts = len(accounts_to_plot)
    if num_accounts == 0:
        print("No accounts to plot.")
        return

    n_cols = min(3, num_accounts) # Max 3 columns
    n_rows = (num_accounts + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 8, n_rows * 6), squeeze=False)
    axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

    # Iterate through each account and create a subplot
    for i, account_name in enumerate(accounts_to_plot):
        ax = axes[i] # Get the current subplot axis

        # Filter data for the current account
        df_account = df_filtered_time[df_filtered_time['Account'] == account_name].copy()

        if df_account.empty:
            ax.set_title(f'No Data for {account_name}')
            ax.set_xlabel('Time (Year-Month)')
            ax.set_ylabel('Number of Unique Users')
            ax.tick_params(axis='x', rotation=90)#, ha='right')
            ax.text(0.5, 0.5, 'No data', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
            continue

        # Group by YearMonth and count unique users for this account
        unique_users_account = df_account.groupby('YearMonth')['User'].nunique().reset_index(name='UniqueUserCount')

        # Plot unique users per month for the current account
        sns.lineplot(data=unique_users_account, x='YearMonth', y='UniqueUserCount',
                     marker='o', color='purple', ax=ax)

        ax.set_title(f'Unique Users for {account_name}', fontsize=12)
        ax.set_xlabel('Time (Year-Month)', fontsize=10)
        ax.set_ylabel('Number of Unique Users', fontsize=10)

        # Rotate x-axis labels
        ax.tick_params(axis='x', rotation=90)#, ha='right')
        ax.tick_params(axis='y', labelsize=8)

        # Add a grid
        ax.grid(True, linestyle='--', alpha=0.7)

        # Set the y-axis limit
        max_y_account = unique_users_account['UniqueUserCount'].max()
        ax.set_ylim(bottom=0, top=max_y_account * 1.1 if max_y_account > 0 else 1) # Ensure min 1 if no users

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # Overall title for the figure
    overall_title_suffix = f" for selected accounts: {', '.join(accounts_to_plot)}" if accounts else " for All Accounts"
    fig.suptitle(f'Total Unique Users Per Account Over Time{overall_title_suffix}', fontsize=18, y=1.02)

    plt.tight_layout(rect=[0, 0, 1, 0.98])
    file_name = f'unique_users_per_account_over_time{overall_title_suffix.replace(" ", "_").replace(",", "").lower()}.png'
    # plt.savefig(file_name)
    plt.show()
    plt.close()
    print(f"Plot '{file_name}' saved successfully.")
    
plot_unique_users_per_account_over_time(df, weeks=104)

In [ ]:
def plot_unique_users_per_account_over_time(df: pd.DataFrame, weeks: int = 1, accounts: Optional[List[str]] = None):
    '''
    Plots the total number of unique users per account over time in subplots, using bar plots.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have 'Start' (datetime), 'User', and 'Account' columns.
    - weeks (int): The number of past weeks for which to plot the data. Defaults to 1.
    - accounts (Optional[List[str]]): A list of account names to plot.
                                          If None, unique users for all *unique* accounts will be plotted
                                          in separate subplots.
    '''
    # Filter df for the specified number of weeks
    filter_date = pd.Timestamp.now() - pd.DateOffset(weeks=weeks)
    df_filtered_time = df[df['Start'] >= filter_date].copy()

    # Ensure 'Start' column is datetime
    if not pd.api.types.is_datetime64_any_dtype(df_filtered_time['Start']):
        print("Warning: 'Start' column is not datetime. Attempting conversion.")
        df_filtered_time['Start'] = pd.to_datetime(df_filtered_time['Start'], errors='coerce')
        df_filtered_time.dropna(subset=['Start'], inplace=True) # Drop rows where conversion failed

    # Determine which accounts to plot
    if accounts:
        accounts_to_plot = [a for a in accounts if a in df_filtered_time['Account'].unique()]
        if not accounts_to_plot:
            print(f"No data found for the specified accounts: {', '.join(accounts)} in the last {weeks} weeks after time filtering.")
            return
    else:
        accounts_to_plot = df_filtered_time['Account'].unique().tolist()
        if not accounts_to_plot:
            print(f"No accounts found in the data for the last {weeks} weeks.")
            return

    # Create a 'YearMonth' column for chronological ordering
    df_filtered_time['YearMonth'] = df_filtered_time['Start'].dt.to_period('M').astype(str)

    # Set up subplots
    num_accounts = len(accounts_to_plot)
    if num_accounts == 0:
        print("No accounts to plot.")
        return

    n_cols = min(3, num_accounts) # Max 3 columns
    n_rows = (num_accounts + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 8, n_rows * 6), squeeze=False)
    axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

    # Iterate through each account and create a subplot
    for i, account_name in enumerate(accounts_to_plot):
        ax = axes[i] # Get the current subplot axis

        # Filter data for the current account
        df_account = df_filtered_time[df_filtered_time['Account'] == account_name].copy()

        if df_account.empty:
            ax.set_title(f'No Data for {account_name}')
            ax.set_xlabel('Time (Year-Month)')
            ax.set_ylabel('Number of Unique Users')
            ax.tick_params(axis='x', rotation=90)#, ha='right')
            ax.text(0.5, 0.5, 'No data', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
            continue

        # Group by YearMonth and count unique users for this account
        unique_users_account = df_account.groupby('YearMonth')['User'].nunique().reset_index(name='UniqueUserCount')

        # Plot unique users per month for the current account using common_plot
        # common_plot will handle labels for kind='bar'
        common_plot(
            df=unique_users_account,
            x='YearMonth',
            y='UniqueUserCount',
            title=f'Unique Users for {account_name}',
            xlabel='Time (Year-Month)',
            ylabel='Number of Unique Users',
            color='purple',
            kind='bar', # Changed to bar plot
            ax=ax
        )
        # Manually adjust x-axis ticks rotation if common_plot defaults are not sufficient for bars
        ax.tick_params(axis='x', rotation=90)#, ha='right')


        # Set the y-axis limit
        max_y_account = unique_users_account['UniqueUserCount'].max()
        # Adjusted ylim to ensure it's set correctly after common_plot, as common_plot only sets it if ax is None
        ax.set_ylim(bottom=0, top=max_y_account * 1.1 if max_y_account > 0 else 1) # Ensure min 1 if no users
        ax.set_ylim(bottom=0, top=25)


    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # Overall title for the figure
    overall_title_suffix = f" for selected accounts: {', '.join(accounts_to_plot)}" if accounts else " for All Accounts"
    fig.suptitle(f'Total Unique Users Per Account Over Time (Bar Chart){overall_title_suffix}', fontsize=18, y=1.02) # Updated title for bar chart

    plt.tight_layout(rect=[0, 0, 1, 0.98])
    file_name = f'unique_users_per_account_over_time_bar_chart{overall_title_suffix.replace(" ", "_").replace(",", "").lower()}.png' # Updated file name
    # plt.savefig(file_name)
    plt.show()
    plt.close()
    print(f"Plot '{file_name}' saved successfully.")


plot_unique_users_per_account_over_time(df, weeks=104)

In [ ]:
def plot_unique_users_per_account_over_time(df: pd.DataFrame, weeks: int = 1, accounts: Optional[List[str]] = None):
    '''
    Plots the total number of unique users per account over time in subplots, using bar plots.
    Creates new figures for every 3x3 grid of subplots.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have 'Start' (datetime), 'User', and 'Account' columns.
    - weeks (int): The number of past weeks for which to plot the data. Defaults to 1.
    - accounts (Optional[List[str]]): A list of account names to plot.
                                          If None, unique users for all *unique* accounts will be plotted
                                          in separate subplots.
    '''
    # Filter df for the specified number of weeks
    filter_date = pd.Timestamp.now() - pd.DateOffset(weeks=weeks)
    df_filtered_time = df[df['Start'] >= filter_date].copy()

    # Ensure 'Start' column is datetime
    if not pd.api.types.is_datetime64_any_dtype(df_filtered_time['Start']):
        print("Warning: 'Start' column is not datetime. Attempting conversion.")
        df_filtered_time['Start'] = pd.to_datetime(df_filtered_time['Start'], errors='coerce')
        df_filtered_time.dropna(subset=['Start'], inplace=True) # Drop rows where conversion failed

    # Determine which accounts to plot
    if accounts:
        accounts_to_plot = [a for a in accounts if a in df_filtered_time['Account'].unique()]
        if not accounts_to_plot:
            print(f"No data found for the specified accounts: {', '.join(accounts)} in the last {weeks} weeks after time filtering.")
            return
    else:
        accounts_to_plot = df_filtered_time['Account'].unique().tolist()
        if not accounts_to_plot:
            print(f"No accounts found in the data for the last {weeks} weeks.")
            return

    # Create a 'YearMonth' column for chronological ordering
    df_filtered_time['YearMonth'] = df_filtered_time['Start'].dt.to_period('M').astype(str)

    # Fixed subplot grid size
    PLOT_COLS = 3
    PLOT_ROWS = 3
    SUBPLOTS_PER_FIGURE = PLOT_COLS * PLOT_ROWS

    # Iterate through accounts in chunks and create a new figure for each chunk
    for chunk_idx in range(0, len(accounts_to_plot), SUBPLOTS_PER_FIGURE):
        accounts_chunk = accounts_to_plot[chunk_idx : chunk_idx + SUBPLOTS_PER_FIGURE]
        num_accounts_in_chunk = len(accounts_chunk)

        fig, axes = plt.subplots(PLOT_ROWS, PLOT_COLS, figsize=(PLOT_COLS * 8, PLOT_ROWS * 6), squeeze=False)
        axes = axes.flatten()

        for i, account_name in enumerate(accounts_chunk):
            ax = axes[i]

            # Filter data for the current account
            df_account = df_filtered_time[df_filtered_time['Account'] == account_name].copy()

            if df_account.empty:
                ax.set_title(f'No Data for {account_name}')
                ax.set_xlabel('Time (Year-Month)')
                ax.set_ylabel('Number of Unique Users')
                ax.tick_params(axis='x', rotation=90)#, ha='right')
                ax.text(0.5, 0.5, 'No data', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
                continue

            # Group by YearMonth and count unique users for this account
            unique_users_account = df_account.groupby('YearMonth')['User'].nunique().reset_index(name='UniqueUserCount')

            # Plot unique users per month for the current account using common_plot
            common_plot(
                df=unique_users_account,
                x='YearMonth',
                y='UniqueUserCount',
                title=f'Unique Users for {account_name}',
                xlabel='Time (Year-Month)',
                ylabel='Number of Unique Users',
                color='purple',
                kind='bar', # Changed to bar plot
                ax=ax
            )
            # Manually adjust x-axis ticks rotation if common_plot defaults are not sufficient for bars
            ax.tick_params(axis='x', rotation=90)#, ha='right')

            # Set the y-axis limit
            max_y_account = unique_users_account['UniqueUserCount'].max()
            ax.set_ylim(bottom=0, top=max_y_account * 1.1 if max_y_account > 0 else 1)
            ax.set_ylim(bottom=0, top=25)  # Adjusted ylim to ensure it's set correctly after common_plot, as common_plot only sets it if ax is None

        # Hide any unused subplots in the current figure
        for j in range(num_accounts_in_chunk, SUBPLOTS_PER_FIGURE):
            fig.delaxes(axes[j])

        # Overall title for the current figure
        overall_title_suffix = f" for selected accounts (Part {chunk_idx // SUBPLOTS_PER_FIGURE + 1})" if accounts else f" for All Accounts (Part {chunk_idx // SUBPLOTS_PER_FIGURE + 1})"
        fig.suptitle(f'Total Unique Users Per Account Over Time (Bar Chart){overall_title_suffix}', fontsize=18, y=1.02)

        plt.tight_layout(rect=[0, 0, 1, 0.98])
        file_name = f'unique_users_per_account_over_time_bar_chart_part{chunk_idx // SUBPLOTS_PER_FIGURE + 1}{overall_title_suffix.replace(" ", "_").replace(",", "").lower().replace("(", "").replace(")", "")}.png'
        # plt.savefig(file_name)
        plt.show()
        plt.close()
        print(f"Plot '{file_name}' saved successfully.")


plot_unique_users_per_account_over_time(df, weeks=104)

In [ ]:
def plot_yearly_total_cost(df):
    """
    Plots the total AWS and NeSI costs per year across all accounts.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have 'Year', 'aws_cost', and 'nesi_cost' columns.
    """
    # Group by 'Year' and sum 'aws_cost' and 'nesi_cost'
    yearly_cost = df.groupby('Year').agg({'aws_cost': 'sum', 'nesi_cost': 'sum'}).reset_index()

    # Ensure 'Year' is treated as a category for proper bar plotting
    yearly_cost['Year'] = yearly_cost['Year'].astype(str)

    # Ensure the output directory exists
    os.makedirs('plots/yearly_costs/', exist_ok=True)

    for cost_type in ['aws_cost', 'nesi_cost']:
        cost_title_prefix = 'AWS Total Cost' if cost_type == 'aws_cost' else 'NeSI Total Cost'
        cost_subtitle_text = 'Based on 2020 best matched instance for given core count' if cost_type == 'aws_cost' else ''
        save_folder = 'plots/yearly_costs/'

        # Filter out years with no data for the current cost type to avoid empty plots
        years_with_data = yearly_cost[yearly_cost[cost_type] > 0]['Year'].unique().tolist()
        if not years_with_data:
            print(f"No {cost_title_prefix} data found for any year.")
            continue
        
        # Print the yearly total cost table
        print("\n--- Yearly Total Costs ---")
        print(yearly_cost[['Year', 'aws_cost', 'nesi_cost']].sort_values(by='Year').to_string())
        print("--------------------------\n")

        fig, ax = plt.subplots(figsize=(6, 4))

        common_plot(
            df=yearly_cost[yearly_cost[cost_type] > 0], # Only plot years with data
            x='Year',
            y=cost_type,
            title=f'',
            xlabel='Year',
            ylabel='Total Cost (NZD)',
            color='purple', # Choose a different color for total yearly plots
            kind='bar',
            ax=ax,
            # ylim=(1000000, 4000000)  # Set y-axis limit to start from 0, auto scale top
        )
        # Ensure y-axis starts from 0 and scales appropriately
        max_cost = yearly_cost[cost_type].max()
        ax.set_ylim(bottom=0, top=max_cost * 1.1 if max_cost > 0 else 1)
        # Set y-axis ticks
        # ax.set_yticks(range(0, int(3e6) + 1, int(1e6)))
        # yscale is log
        # ax.set_yscale('log')
        # Overall title for the current figure (suptitle)
        overall_title = f'{cost_title_prefix} Per Year\n{cost_subtitle_text}'
        fig.suptitle(overall_title)#, fontsize=11, y=0.02, va='bottom', ha='center') # Moved to bottom

        # Adjust layout to make space for the suptitle at the bottom
        plt.tight_layout(rect=[0, 0.08, 1, 1]) # Adjusted bottom margin (0.08 is an example, may need tweaking)
        
        
        plot_file_name = f"{save_folder}total_yearly_{cost_type}.png"
        # plt.savefig(plot_file_name)
        plt.tight_layout()
        plt.show()
        plt.close(fig)
        
        print(f"Plot '{plot_file_name}' saved successfully.")
        
        
plot_yearly_total_cost(df)

In [ ]:
def plot_costs_per_month(df):
    """
    Plots AWS and NeSI costs per month for each account, using a 3x3 subplot grid
    and creating new figures as needed to accommodate all accounts.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have 'Start' (datetime), 'Account', 'aws_cost', and 'nesi_cost' columns.
    """
    # Group by 'Account', 'Year' and 'Month' and sum 'aws_cost' and 'nesi_cost'
    cost_per_month = df.groupby(['Account', 'Year', 'Month']).agg({'aws_cost': 'sum', 'nesi_cost': 'sum'}).reset_index()

    # Convert 'Year' and 'Month' to integer, then to string, combine them, and convert to datetime
    # This creates a sortable chronological order for the x-axis
    cost_per_month['YearMonth'] = pd.to_datetime(
        cost_per_month['Year'].astype(int).astype(str) + '-' +
        cost_per_month['Month'].astype(int).astype(str)
    )

    # Get unique accounts for plotting
    accounts = cost_per_month['Account'].unique()

    # Capitalize 'Account' names for consistent display on plots
    cost_per_month['Account'] = cost_per_month['Account'].str.upper()

    # Ensure the output directories exist
    os.makedirs('plots/monthly_costs/aws/', exist_ok=True)
    os.makedirs('plots/monthly_costs/nesi/', exist_ok=True)

    # Fixed subplot grid size
    PLOT_COLS = 3
    PLOT_ROWS = 3
    SUBPLOTS_PER_FIGURE = PLOT_COLS * PLOT_ROWS

    for cost_type in ['aws_cost', 'nesi_cost']:
        cost_title_prefix = 'AWS Cost' if cost_type == 'aws_cost' else 'NeSI Cost'
        cost_subtitle_text = 'Based on 2020 best matched instance for given core count' if cost_type == 'aws_cost' else ''
        save_folder = 'plots/monthly_costs/aws/' if cost_type == 'aws_cost' else 'plots/monthly_costs/nesi/'

        # Filter out accounts with no data for the current cost type to avoid empty plots
        accounts_with_data = cost_per_month[cost_per_month[cost_type] > 0]['Account'].unique().tolist()
        if not accounts_with_data:
            print(f"No {cost_title_prefix} data found for any account.")
            continue

        # Iterate through accounts in chunks and create a new figure for each chunk
        for chunk_idx in range(0, len(accounts_with_data), SUBPLOTS_PER_FIGURE):
            accounts_chunk = accounts_with_data[chunk_idx : chunk_idx + SUBPLOTS_PER_FIGURE]
            num_accounts_in_chunk = len(accounts_chunk)

            fig, axes = plt.subplots(PLOT_ROWS, PLOT_COLS, figsize=(PLOT_COLS * 8, PLOT_ROWS * 6), squeeze=False)
            axes = axes.flatten()

            for i, account_name in enumerate(accounts_chunk):
                ax = axes[i]

                account_data = cost_per_month[cost_per_month['Account'] == account_name].copy()

                if account_data.empty or account_data[cost_type].sum() == 0:
                    ax.set_title(f'No {cost_title_prefix} for {account_name}')
                    ax.set_xlabel('Date')
                    ax.set_ylabel('Cost')
                    ax.tick_params(axis='x', rotation=90)#, ha='right')
                    ax.text(0.5, 0.5, 'No data', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
                    continue

                # Use common_plot for consistency
                common_plot(
                    df=account_data,
                    x='YearMonth',
                    y=cost_type,
                    title=f'{cost_title_prefix} for {account_name} Per Month',
                    xlabel='Date',
                    ylabel='Cost',
                    color='blue', # Default color for cost plots
                    kind='bar',
                    ax=ax
                )
                # Manually adjust x-axis ticks rotation if common_plot defaults are not sufficient for bars
                ax.tick_params(axis='x', rotation=90)#, ha='right')

                # Ensure y-axis starts from 0 and scales appropriately
                max_cost = account_data[cost_type].max()
                ax.set_ylim(bottom=0, top=max_cost * 1.1 if max_cost > 0 else 1)


            # Hide any unused subplots in the current figure
            for j in range(num_accounts_in_chunk, SUBPLOTS_PER_FIGURE):
                fig.delaxes(axes[j])

            # Overall title for the current figure
            overall_title_suffix = f" (Part {chunk_idx // SUBPLOTS_PER_FIGURE + 1})"
            fig.suptitle(f'Monthly {cost_title_prefix} Per Account{overall_title_suffix}\n{cost_subtitle_text}', fontsize=18, y=1.02)

            plt.tight_layout(rect=[0, 0, 1, 0.98]) # Adjust rect to make space for the suptitle
            plot_file_name = f"{save_folder}{cost_type}_{chunk_idx // SUBPLOTS_PER_FIGURE + 1}.png"
            # plt.savefig(plot_file_name)
            plt.show()
            plt.close(fig) # Close the figure to free memory
            print(f"Plot '{plot_file_name}' saved successfully.")
            
            
plot_costs_per_month(df)

In [ ]:
def print_total_costs_per_account(df: pd.DataFrame):
    """
    Calculates and prints a table showing the total AWS and NeSI costs for each account.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have 'Account', 'aws_cost', and 'nesi_cost' columns.
    """
    # Group by 'Account' and sum 'aws_cost' and 'nesi_cost'
    total_costs = df.groupby('Account').agg(
        Total_AWS_Cost=('aws_cost', 'sum'),
        Total_NeSI_Cost=('nesi_cost', 'sum')
    ).reset_index()

    # Capitalize 'Account' names for consistent display
    total_costs['Account'] = total_costs['Account'].str.upper()

    print("\nTotal AWS and NeSI Costs per Account:")
    # Format the cost columns to 2 decimal places for better readability
    total_costs['Total_AWS_Cost'] = total_costs['Total_AWS_Cost'].map('{:.2f}'.format)
    total_costs['Total_NeSI_Cost'] = total_costs['Total_NeSI_Cost'].map('{:.2f}'.format)
    print(total_costs.to_string(index=False))
    
print_total_costs_per_account(df)

In [ ]:
def print_total_costs_per_account(df: pd.DataFrame):
    """
    Calculates and prints tables showing the total AWS and NeSI costs for each account,
    broken down by year.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have 'Account', 'Year', 'aws_cost', and 'nesi_cost' columns.
    """
    # Get unique years from the DataFrame and drop NaN values
    unique_years = sorted(df['Year'].dropna().unique())

    if not unique_years:
        print("No year data found to calculate costs.")
        return

    for year in unique_years:
        print(f"\n--- Total AWS and NeSI Costs per Account for Year: {int(year)} ---")
        
        # Filter data for the current year
        df_year = df[df['Year'] == year].copy()

        if df_year.empty:
            print(f"No data available for year {int(year)}.")
            continue

        # Group by 'Account' and sum 'aws_cost' and 'nesi_cost' for the current year
        total_costs_year = df_year.groupby('Account').agg(
            Total_AWS_Cost=('aws_cost', 'sum'),
            Total_NeSI_Cost=('nesi_cost', 'sum')
        ).reset_index()

        # Capitalize 'Account' names for consistent display
        total_costs_year['Account'] = total_costs_year['Account'].str.upper()

        # Format the cost columns to 2 decimal places for better readability
        total_costs_year['Total_AWS_Cost'] = total_costs_year['Total_AWS_Cost'].map('{:.2f}'.format)
        total_costs_year['Total_NeSI_Cost'] = total_costs_year['Total_NeSI_Cost'].map('{:.2f}'.format)
        print(total_costs_year.to_string(index=False))
        
        
print_total_costs_per_account(df)


In [ ]:
def plot_capacity_daily(df, weeks=1):
    '''
    Plot the total available and requested capacity daily.
    Total configured capacity is 5535 + 14335 + 1004= 20874 GB

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have 'Start' (datetime), 'ReqMem', and 'User' columns.
    - weeks (int): The number of past weeks for which to plot the data. Defaults to 1.
    '''
    # Filter df for the specified number of weeks
    filter_date = pd.Timestamp.now() - pd.DateOffset(weeks=weeks)
    df_filtered = df[df['Start'] >= filter_date].copy()

    # Ensure ReqMem is in GB (assuming it's initially in MB as per memfix)
    # This check prevents re-conversion if 'ReqMem' is already in GB
    if 'ReqMem' in df_filtered.columns and df_filtered['ReqMem'].max() > 2048:
        df_filtered['ReqMem_GB'] = df_filtered['ReqMem'] / 1024
    else:
        df_filtered['ReqMem_GB'] = df_filtered['ReqMem'] # Assume it's already GB if max is small

    print(f"Plotting capacity for the last {weeks} weeks")
    print(f"Total Jobs in the last {weeks} weeks: {len(df_filtered)}")

    # Group by date and calculate total requested capacity and total available capacity
    daily_capacity = df_filtered.groupby(df_filtered['Start'].dt.date).agg(
        TotalRequestedCapacity=('ReqMem_GB', 'sum'),
        TotalAvailableCapacity=('User', lambda x: 2048 * x.nunique()) # Correctly calculates available capacity based on unique users
    ).reset_index()

    # Convert 'Start' column in daily_capacity back to datetime objects for plotting
    daily_capacity['Start'] = pd.to_datetime(daily_capacity['Start'])

    print(daily_capacity.head())

    # Plotting using matplotlib and seaborn for multiple lines on one plot
    fig, current_ax = plt.subplots(figsize=(14, 7)) # Create figure and get axis object directly

    # Plot requested capacity
    sns.lineplot(data=daily_capacity, x='Start', y='TotalRequestedCapacity',
                 label='Total Requested Capacity (GB)', color='dodgerblue', linewidth=2, ax=current_ax)

    # Plot available capacity
    # sns.lineplot(data=daily_capacity, x='Start', y='TotalAvailableCapacity',
    #              label='Total Available Capacity (GB)', color='orange', linewidth=2, ax=current_ax)

    # Plot a horizontal line for the total configured capacity (20874 GB)
    current_ax.axhline(y=20874, color='red', linestyle='--', label='Total Configured Capacity (Configured: 20874 GB)', linewidth=2)

    # Set title and labels with consistent formatting using the axis object
    current_ax.set_title('Daily Requested vs Configured Memory Capacity', fontsize=16)
    current_ax.set_xlabel('Date', fontsize=12)
    current_ax.set_ylabel('Capacity (GB)', fontsize=12)

    # Rotate x-axis labels for better readability of dates using tick_params
    current_ax.tick_params(axis='x', rotation=0) # Removed 'ha' keyword
    current_ax.tick_params(axis='y', labelsize=10) # Consistent tick label font size for y-axis

    # Add a grid for easier reading of values
    current_ax.grid(True, linestyle='--', alpha=0.7)
    # Set the y-axis limit
    # current_ax.set_ylim(bottom=0, top=7.5e5) # Set lower limit to 0 and upper limit to 21000 GB
    
    # Place legend below the plot
    current_ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), # Adjusted bbox_to_anchor for below plot
                      ncol=1, # One column for legend items
                      fontsize=10,
                      frameon=False) # Remove frame around legend for cleaner look

    # Place legend outside the plot area if it overlaps, or in a less obstructive spot
    # current_ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=10)
    

    # Adjust layout to prevent overlapping elements
    # plt.tight_layout(rect=[0, 0.1, 0.1, 1]) # Adjust rect to make space for the legend

    # Display the plot
    plt.show()
    
plot_capacity_daily(df, weeks=104)

In [ ]:
def plot_capacity_daily(df, weeks=1):
    '''
    Plot the total available and requested capacity daily.
    Total configured capacity is 5535 + 14335 + 1004= 20874 GB

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have 'Start' (datetime), 'ReqMem', and 'User' columns.
    - weeks (int): The number of past weeks for which to plot the data. Defaults to 1.
    '''
    # Filter df for the specified number of weeks
    filter_date = pd.Timestamp.now() - pd.DateOffset(weeks=weeks)
    df_filtered = df[df['Start'] >= filter_date].copy()

    # Ensure ReqMem is in GB (assuming it's initially in MB as per memfix)
    # This check prevents re-conversion if 'ReqMem' is already in GB
    if 'ReqMem' in df_filtered.columns and df_filtered['ReqMem'].max() > 2048:
        df_filtered['ReqMem_GB'] = df_filtered['ReqMem'] / 1024
    else:
        df_filtered['ReqMem_GB'] = df_filtered['ReqMem'] # Assume it's already GB if max is small

    print(f"Plotting capacity for the last {weeks} weeks")
    print(f"Total Jobs in the last {weeks} weeks: {len(df_filtered)}")

    # Group by date and calculate total requested capacity and total available capacity
    daily_capacity = df_filtered.groupby(df_filtered['Start'].dt.date).agg(
        TotalRequestedCapacity=('ReqMem_GB', 'sum'),
        TotalAvailableCapacity=('User', lambda x: 20874) # Corrected: total configured capacity is a fixed number
    ).reset_index()

    # Convert 'Start' column in daily_capacity back to datetime objects for plotting
    daily_capacity.rename(columns={'index': 'Start'}, inplace=True) # Rename 'index' to 'Start' for consistency
    daily_capacity['Start'] = pd.to_datetime(daily_capacity['Start'])

    # Filter out data points where TotalRequestedCapacity is more than 1e6
    original_rows = len(daily_capacity)
    requested_capacity_threshold = 1e6 # 1e6 GB = 1 TB
    daily_capacity = daily_capacity[daily_capacity['TotalRequestedCapacity'] <= requested_capacity_threshold].copy()
    filtered_rows = len(daily_capacity)

    if original_rows > filtered_rows:
        print(f"Filtered out {original_rows - filtered_rows} days where TotalRequestedCapacity exceeded {requested_capacity_threshold / 1e6:.0f} TB.")
    else:
        print("No days filtered based on TotalRequestedCapacity threshold.")


    print(daily_capacity.head())

    # Plotting using matplotlib and seaborn for multiple lines on one plot
    fig, current_ax = plt.subplots(figsize=(14, 7)) # Create figure and get axis object directly

    # Plot requested capacity
    sns.lineplot(data=daily_capacity, x='Start', y='TotalRequestedCapacity',
                 label='Total Requested Capacity (GB)', color='dodgerblue', linewidth=2, ax=current_ax)

    # Plot available capacity
    sns.lineplot(data=daily_capacity, x='Start', y='TotalAvailableCapacity',
                 label='Total Available Capacity (GB)', color='orange', linewidth=2, ax=current_ax)

    # Plot a horizontal line for the total configured capacity (20874 GB)
    total_configured_capacity = 20874 # Defined as a variable
    current_ax.axhline(y=total_configured_capacity, color='red', linestyle='--', label=f'Total Configured Capacity ({total_configured_capacity} GB)', linewidth=2)

    # Add text label for the horizontal line
    # Position the text just above the line, at the start of the x-axis, with appropriate alignment
    text_x_position = daily_capacity['Start'].min() # Position at the start of the data range
    text_y_position = total_configured_capacity * 1.01 # Slightly above the line
    current_ax.text(text_x_position, text_y_position, f'Configured Capacity: {total_configured_capacity} GB',
                    color='red', fontsize=10, verticalalignment='bottom', horizontalalignment='left',
                    bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', boxstyle='round,pad=0.2'))


    # Set title and labels with consistent formatting using the axis object
    current_ax.set_title('Daily Requested vs Configured Memory Capacity', fontsize=16)
    current_ax.set_xlabel('Date', fontsize=12)
    current_ax.set_ylabel('Capacity (GB)', fontsize=12)

    # Rotate x-axis labels for better readability of dates using tick_params
    current_ax.tick_params(axis='x', rotation=0)#, ha='right') # Rotate and align for dates
    current_ax.tick_params(axis='y', labelsize=10) # Consistent tick label font size for y-axis

    # Add a grid for easier reading of values
    current_ax.grid(True, linestyle='--', alpha=0.7)
    # Set the y-axis limit - ensure it's dynamic but also accounts for the configured capacity
    max_y = max(daily_capacity['TotalRequestedCapacity'].max(), total_configured_capacity)
    current_ax.set_ylim(bottom=0, top=max_y * 1.1) # Set lower limit to 0 and upper limit slightly above max value

    # Place legend below the plot
    current_ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), # Adjusted bbox_to_anchor for below plot
                      ncol=1, # One column for legend items
                      fontsize=10,
                      frameon=False) # Remove frame around legend for cleaner look

    # Adjust layout to prevent overlapping elements
    plt.tight_layout(rect=[0, 0.1, 1, 1]) # Adjust rect to make space for the legend

    # Display the plot
    plt.show()
    
plot_capacity_daily(df, weeks=104) # Call the function with 104 weeks (2 years) of data

In [ ]:
def plot_capacity_daily(df: pd.DataFrame, weeks: int = 1, partitions: Optional[List[str]] = None, requested_memory_threshold: float = 2e5):
    '''
    Plot the total available and requested capacity daily for selected partitions in subplots.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have 'Start' (datetime), 'ReqMem', and 'User' columns.
    - weeks (int): The number of past weeks for which to plot the data. Defaults to 1.
    - partitions (Optional[List[str]]): A list of partition names to plot.
                                          If None, requested capacity for all *unique* partitions will be plotted
                                          in separate subplots.
    - requested_memory_threshold (float): The maximum allowed TotalRequestedCapacity (in GB) for a day.
                                        Days exceeding this will be dropped. Defaults to 1e6 GB (1 TB).
    '''
    # Define configured capacity for each known partition
    # These values are based on the sum provided in the previous prompt: 5535 + 14335 + 1004 = 20874 GB
    # Assuming the partitions are 'partition1', 'partition2', 'partition3' or similar.
    # You might need to adjust these names based on your actual data's 'Partition' column values.
    partition_configured_capacity = {
        'parallel': 502*28+250,   # Example: Assuming 'cpu' partition has this capacity
        'gpu': 502*3,    # Example: Assuming 'gpu' partition has this capacity
        'longrun': 502*2,  # Example: Assuming 'short' partition has this capacity
        'quicktest': 502*4,
        'bigmem': 1006*4,
        # Add other partitions and their capacities as needed.
        # A fallback for unknown partitions could be added, e.g., 0 or the total system capacity.
    }
    # Fallback for partitions not explicitly defined, or a sensible default
    default_configured_capacity = 20874 # Using the sum as a generic fallback if a partition isn't mapped

    # Filter df for the specified number of weeks
    filter_date = pd.Timestamp.now() - pd.DateOffset(weeks=weeks)
    df_filtered_time = df[df['Start'] >= filter_date].copy()

    # Ensure ReqMem is in GB (assuming it's initially in MB as per memfix)
    df_filtered_time['ReqMem_GB'] = df_filtered_time['ReqMem'] / 1024

    # Determine which partitions to plot
    if partitions:
        partitions_to_plot = [p for p in partitions if p in df_filtered_time['Partition'].unique()]
        if not partitions_to_plot:
            print(f"No data found for the specified partitions: {', '.join(partitions)} in the last {weeks} weeks after time filtering.")
            return
    else:
        partitions_to_plot = df_filtered_time['Partition'].unique().tolist()
        if not partitions_to_plot:
            print(f"No partitions found in the data for the last {weeks} weeks.")
            return

    # Set up subplots
    num_partitions = len(partitions_to_plot)
    if num_partitions == 0:
        print("No partitions to plot.")
        return

    # Calculate rows and columns for subplot grid, aiming for a squarish layout
    n_cols = min(3, num_partitions) # Max 3 columns
    n_rows = (num_partitions + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 8, n_rows * 6), squeeze=False)
    axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

    # Iterate through each partition and create a subplot
    for i, partition_name in enumerate(partitions_to_plot):
        ax = axes[i] # Get the current subplot axis

        # Get the configured capacity for the current partition, with a fallback
        current_configured_capacity = partition_configured_capacity.get(partition_name, default_configured_capacity)

        # Filter data for the current partition
        df_partition = df_filtered_time[df_filtered_time['Partition'] == partition_name].copy()

        if df_partition.empty:
            ax.set_title(f'No Data for {partition_name}')
            ax.set_xlabel('Date')
            ax.set_ylabel('Capacity (GB)')
            ax.tick_params(axis='x', rotation=45, ha='right')
            ax.text(0.5, 0.5, 'No data', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
            continue

        # Group by date for the current partition
        daily_capacity_requested_partition = df_partition.groupby(df_partition['Start'].dt.date)['ReqMem_GB'].sum().reset_index(name='TotalRequestedCapacity')
        daily_capacity_requested_partition.rename(columns={'Start': 'Date'}, inplace=True)
        daily_capacity_requested_partition['Date'] = pd.to_datetime(daily_capacity_requested_partition['Date'])

        # Filter out data points where TotalRequestedCapacity is more than the threshold for the current partition
        original_rows_part = len(daily_capacity_requested_partition)
        daily_capacity_requested_partition = daily_capacity_requested_partition[daily_capacity_requested_partition['TotalRequestedCapacity'] <= requested_memory_threshold].copy()
        filtered_rows_part = len(daily_capacity_requested_partition)

        if original_rows_part > filtered_rows_part:
            print(f"Filtered out {original_rows_part - filtered_rows_part} days for partition '{partition_name}' where TotalRequestedCapacity exceeded {requested_memory_threshold / 1e3:.0f} GB ({requested_memory_threshold / 1e6:.0f} TB).")

        if daily_capacity_requested_partition.empty:
            ax.set_title(f'No Plottable Data for {partition_name}')
            ax.set_xlabel('Date')
            ax.set_ylabel('Capacity (GB)')
            ax.tick_params(axis='x', rotation=45, ha='right')
            ax.text(0.5, 0.5, 'No plottable data', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
            continue

        # Plot requested capacity for the current partition
        sns.lineplot(data=daily_capacity_requested_partition, x='Date', y='TotalRequestedCapacity',
                     label=f'Requested Capacity ({partition_name} GB)', linewidth=2, ax=ax)

        # Plot a horizontal line for the partition's configured capacity
        ax.axhline(y=current_configured_capacity, color='red', linestyle='--', label=f'Configured Total ({current_configured_capacity} GB)', linewidth=2)

        # Add text label for the horizontal line (adjusted for subplot)
        text_x_position = daily_capacity_requested_partition['Date'].min()
        text_y_position = current_configured_capacity * 1.01
        ax.text(text_x_position, text_y_position, f'Configured: {current_configured_capacity} GB',
                color='red', fontsize=8, verticalalignment='bottom', horizontalalignment='left',
                bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', boxstyle='round,pad=0.2'))


        # Set subplot title and labels
        ax.set_title(f'Daily Capacity for {partition_name}', fontsize=12)
        ax.set_xlabel('Date', fontsize=10)
        ax.set_ylabel('Capacity (GB)', fontsize=10)

        # Rotate x-axis labels
        ax.tick_params(axis='x', rotation=0)#, ha='right')
        ax.tick_params(axis='y', labelsize=8)

        # Add a grid
        ax.grid(True, linestyle='--', alpha=0.7)

        # Set the y-axis limit
        max_y_partition = max(daily_capacity_requested_partition['TotalRequestedCapacity'].max(), current_configured_capacity)
        ax.set_ylim(bottom=0, top=max_y_partition * 1.1)

        # Place legend on each subplot
        ax.legend(loc='upper left', fontsize=8, frameon=False)

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # Overall title for the figure
    overall_title_suffix = f" for selected partitions: {', '.join(partitions_to_plot)}" if partitions else " for All Partitions"
    fig.suptitle(f'Daily Requested Memory Capacity Per Partition{overall_title_suffix}', fontsize=18, y=1.02) # Adjust y to prevent overlap

    plt.tight_layout(rect=[0, 0, 1, 0.98]) # Adjust rect to make space for the suptitle
    file_name = f'daily_requested_memory_capacity_per_partition{overall_title_suffix.replace(" ", "_").replace(",", "").lower()}.png'
    # plt.savefig(file_name)
    plt.show()
    plt.close()
    print(f"Plot '{file_name}' saved successfully.")

plot_capacity_daily(df, weeks=104) # Call the function with 104 weeks (2 years) of data

In [ ]:
def analyze_long_waits(df):
    '''
    Analyzes job wait times and plots a doughnut chart showing the
    proportion of jobs that waited more than 4 hours vs. less than or equal to 4 hours.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have a 'Wait' (timedelta) column.
    '''
    if df.empty:
        print("Cannot analyze long waits: DataFrame is empty.")
        return 0, 0

    # Define the 4-hour threshold
    four_hours = pd.Timedelta(hours=4)

    # Filter jobs where 'Wait' time is greater than 4 hours
    # Ensure 'Wait' column has valid timedelta objects and drop NaNs if any
    # Assuming 'Wait' column is already in a timedelta format from preprocess_data
    df_clean_wait = df.dropna(subset=['Wait'])

    long_wait_jobs = df_clean_wait[df_clean_wait['Wait'] > four_hours]
    num_long_waits = len(long_wait_jobs)
    total_jobs = len(df_clean_wait)

    if total_jobs == 0:
        percentage_long_waits = 0
    else:
        percentage_long_waits = (num_long_waits / total_jobs) * 100

    print(f"\n--- Job Wait Time Analysis ---")
    print(f"Total jobs analyzed: {total_jobs}")
    print(f"Jobs that waited more than 4 hours: {num_long_waits}")
    print(f"Percentage of jobs waiting > 4 hours: {percentage_long_waits:.2f}%")

    # Prepare data for doughnut chart
    # Use pd.cut to categorize wait times
    wait_categories = pd.cut(df_clean_wait['Wait'],
                             bins=[pd.Timedelta(seconds=0), four_hours, pd.Timedelta.max],
                             labels=['<= 4 Hours', '> 4 Hours'],
                             right=True,
                             include_lowest=True) # Ensure jobs with 0 wait time are included

    wait_counts = wait_categories.value_counts().sort_index()

    # Define colors for consistency
    colors = ['#66b3ff', '#ff9999'] # Light blue for short waits, light red for long waits
    # If the order of categories is not consistent, map colors explicitly
    # For example: colors = [colors[0] if label == '<= 4 Hours' else colors[1] for label in wait_counts.index]


    # Create a Doughnut chart
    fig, ax = plt.subplots(figsize=(5, 5)) # Use fig, ax for consistent plotting structure
    
    # Plot the pie chart wedges
    wedges, texts, autotexts = ax.pie(wait_counts,
                                      autopct='%1.1f%%',
                                      startangle=90,
                                      colors=colors,
                                      wedgeprops=dict(width=0.3, edgecolor='w')) # Make it a doughnut by setting wedge width

    # Draw a circle in the center to make it a doughnut chart
    centre_circle = plt.Circle((0,0), 0.70, fc='white')
    fig.gca().add_artist(centre_circle)

    # Set consistent font sizes for percentages
    for autotext in autotexts:
        autotext.set_fontsize(12)
        autotext.set_color('black') # Ensure text is visible

    # Add legend
    # Place legend below the plot, consistent with plot_capacity_daily
    ax.legend(wedges, wait_counts.index,
              title="Wait Time Category",
              loc="upper center",
              bbox_to_anchor=(0.5, -0.05), # Position below the plot
              ncol=len(wait_counts.index), # Arrange items in a single row
              fontsize=10,
              frameon=False) # No frame around the legend

    # Set title with consistent font size
    ax.set_title('Distribution of Job Wait Times', fontsize=12)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    # Adjust layout to make space for the legend below
    plt.tight_layout(rect=[0, 0.1, 0.85, 1]) # Adjust rect to make space for the legend

    # plt.savefig('job_wait_time_doughnut_chart.png') # Optional: uncomment to save

    plt.show()

    return num_long_waits, percentage_long_waits

analyze_long_waits(df)

In [ ]:
def analyze_long_waits(df):
    '''
    Analyzes job wait times and plots a doughnut chart showing the
    proportion of jobs in various wait time categories.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have a 'Wait' (timedelta) column.
    '''
    if df.empty:
        print("Cannot analyze long waits: DataFrame is empty.")
        return 0, 0

    # Define the new granular wait time thresholds
    bins = [
        pd.Timedelta(seconds=0),
        pd.Timedelta(minutes=30),
        pd.Timedelta(hours=2),
        pd.Timedelta(hours=4),
        pd.Timedelta(hours=12),
        pd.Timedelta.max # Represents infinity for the last bin
    ]
    labels = [
        '<30 mins',
        '>30 mins and <2 hours',
        '>2 hours and <4 hours',
        '>4 hours and <12 hours',
        '>12 hours'
    ]

    # Filter jobs where 'Wait' time is greater than 4 hours for the specific count
    four_hours = pd.Timedelta(hours=4)
    df_clean_wait = df.dropna(subset=['Wait'])

    long_wait_jobs = df_clean_wait[df_clean_wait['Wait'] > four_hours]
    num_long_waits = len(long_wait_jobs)
    total_jobs = len(df_clean_wait)

    if total_jobs == 0:
        percentage_long_waits = 0
    else:
        percentage_long_waits = (num_long_waits / total_jobs) * 100

    print(f"\n--- Job Wait Time Analysis ---")
    print(f"Total jobs analyzed: {total_jobs}")
    print(f"Jobs that waited more than 4 hours: {num_long_waits}")
    print(f"Percentage of jobs waiting > 4 hours: {percentage_long_waits:.2f}%")

    # Prepare data for doughnut chart using the new bins and labels
    wait_categories = pd.cut(df_clean_wait['Wait'],
                             bins=bins,
                             labels=labels,
                             right=False, # Use right=False to make intervals [lower, upper)
                             include_lowest=True)

    wait_counts = wait_categories.value_counts().sort_index()

    # Define colors for consistency (adjust for 5 categories)
    # Using a color map to ensure distinct colors for each segment
    colors = plt.cm.tab10.colors[:len(labels)] # Using tab10 colormap, taking first N colors


    # Create a Doughnut chart
    fig, ax = plt.subplots(figsize=(8, 8)) # Adjusted size for potentially more labels

    # Plot the pie chart wedges
    wedges, texts, autotexts = ax.pie(wait_counts,
                                      autopct='%1.1f%%',
                                      startangle=90,
                                      colors=colors,
                                      wedgeprops=dict(width=0.3, edgecolor='w')) # Make it a doughnut by setting wedge width

    # Draw a circle in the center to make it a doughnut chart
    centre_circle = plt.Circle((0,0), 0.70, fc='white')
    fig.gca().add_artist(centre_circle)

    # Set consistent font sizes for percentages
    for autotext in autotexts:
        autotext.set_fontsize(12)
        autotext.set_color('black') # Ensure text is visible

    # Add legend
    ax.legend(wedges, wait_counts.index,
              title="Wait Time Category",
              loc="upper center",
              bbox_to_anchor=(0.5, -0.05), # Position below the plot
              ncol=len(wait_counts.index), # Arrange items in a single row
              fontsize=10,
              frameon=False) # No frame around the legend

    # Set title with consistent font size
    ax.set_title('Distribution of Job Wait Times', fontsize=14)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    # Adjust layout to make space for the legend below
    plt.tight_layout(rect=[0, 0.1, 1, 1]) # Adjust rect to make space for the legend

    # plt.savefig('job_wait_time_doughnut_chart.png') # Optional: uncomment to save

    plt.show()

    return num_long_waits, percentage_long_waits

# --- NEW: Analysis and plot for granular wait times ---
analyze_long_waits(df)


In [ ]:
def plot_yearly_partition_long_wait_breakdown(df: pd.DataFrame, partition: str = None):
    import math
    """
    Plots the breakdown of jobs with wait times > 4 hours, partitioned by year
    and individual partition in subplots. Each subplot shows monthly stacked bars,
    where stacks represent different wait time categories. Legends are sorted by
    total job count for each wait category within that specific (partition, year) subplot.
    Custom y-axis limits are applied based on the year.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing job data with
                           'Start', 'Partition', 'JobID', and 'Wait' columns.
        partition (str, optional): If provided, plots only for this specific partition.
                                   Defaults to None, plotting all unique partitions.
    """

    if df.empty:
        print("Cannot plot long wait time breakdown: DataFrame is empty.")
        return

    # --- 1. Filter and Categorize Long Wait Jobs ---
    df_clean_wait = df.dropna(subset=['Wait', 'Start', 'Partition', 'JobID'])

    if df_clean_wait.empty:
        print("After dropping NaNs, DataFrame is empty for wait time analysis.")
        return

    four_hours = pd.Timedelta(hours=4)
    long_wait_jobs_df = df_clean_wait[df_clean_wait['Wait'] > four_hours].copy()

    if long_wait_jobs_df.empty:
        print("No jobs found with wait times greater than 4 hours to plot breakdown.")
        return

    # Define bins and labels for wait times > 4 hours
    wait_bins = [
        four_hours,
        pd.Timedelta(hours=12),
        pd.Timedelta(hours=24),
        pd.Timedelta(hours=48),
        long_wait_jobs_df['Wait'].max() + pd.Timedelta(seconds=1) # Ensure max value is included
    ]
    wait_labels = [
        '4-12 hours',
        '12-24 hours',
        '24-48 hours',
        '>48 hours'
    ]

    # Handle cases where max wait time might be less than 48 hours, reducing bins dynamically
    effective_bins = [b for b in wait_bins if b <= long_wait_jobs_df['Wait'].max() + pd.Timedelta(seconds=1)]
    effective_labels = wait_labels[:len(effective_bins) - 1]

    if not effective_labels: # If all long wait times are less than 12 hours (e.g., 4-8 hours)
        effective_bins = [four_hours, long_wait_jobs_df['Wait'].max() + pd.Timedelta(seconds=1)]
        effective_labels = ['4+ hours']


    long_wait_jobs_df['WaitCategory'] = pd.cut(long_wait_jobs_df['Wait'],
                                              bins=effective_bins,
                                              labels=effective_labels,
                                              right=True,
                                              include_lowest=True,
                                              ordered=True) # Ensure categories are ordered for sorting

    long_wait_jobs_df.dropna(subset=['WaitCategory'], inplace=True) # Drop jobs that didn't fit into a category

    if long_wait_jobs_df.empty:
        print("After categorizing wait times, no jobs remain for plotting.")
        return

    # --- 2. Preprocessing for Plotting (Year, Month, YearMonth) ---
    long_wait_jobs_df['Year'] = long_wait_jobs_df['Start'].dt.year
    long_wait_jobs_df['Month'] = long_wait_jobs_df['Start'].dt.month
    long_wait_jobs_df['YearMonth'] = long_wait_jobs_df['Start'].dt.to_period('M')

    # --- 3. Aggregate Data ---
    # Group by YearMonth, Partition, and WaitCategory, then count jobs
    # This will be the base data for all subplots
    # Add observed=False to suppress the FutureWarning
    monthly_partition_wait_counts_raw = long_wait_jobs_df.groupby(
        ['YearMonth', 'Partition', 'WaitCategory'], observed=False
    )['JobID'].count().unstack(fill_value=0) # Unstack WaitCategory to make them columns

    # --- 4. Identify Unique Partitions and Years ---
    if partition:
        # If a specific partition is provided, filter the long_wait_jobs_df BEFORE getting unique years
        long_wait_jobs_df_filtered_by_partition = long_wait_jobs_df[long_wait_jobs_df['Partition'] == partition].copy()
        if long_wait_jobs_df_filtered_by_partition.empty:
            print(f"No long wait jobs found for partition '{partition}'.")
            return
        unique_partitions_to_plot = [partition]
        unique_years = sorted(long_wait_jobs_df_filtered_by_partition['Year'].unique())
    else:
        # If no specific partition is provided, get unique partitions from the full DataFrame
        unique_partitions_to_plot = sorted(long_wait_jobs_df['Partition'].unique())
        unique_years = sorted(long_wait_jobs_df['Year'].unique())


    if not unique_partitions_to_plot or not unique_years:
        print("No unique partitions or years found after filtering long wait jobs.")
        return

    # --- 5. Subplot Setup ---
    num_subplots = len(unique_partitions_to_plot) * len(unique_years)
    ncols = min(3, max(1, num_subplots)) # Max 3 columns, at least 1
    nrows = math.ceil(num_subplots / ncols)

    # Adjust figsize for potential many subplots and legends
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(6 * ncols + 5, 6 * nrows), squeeze=False)
    axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

    # --- 6. Calculate overall max job count for consistent Y-axis limits (fallback if custom not applied) ---
    max_job_count = 0
    if not monthly_partition_wait_counts_raw.empty:
        # Sum across WaitCategory to get total jobs per YearMonth, Partition
        temp_summed = monthly_partition_wait_counts_raw.sum(axis=1)
        max_job_count = temp_summed.max()
    if max_job_count == 0:
        max_job_count = 1 # Avoid division by zero or empty plot range

    # --- 7. Iterate through Subplots (Partition by Year) ---
    plot_idx = 0
    for current_partition in unique_partitions_to_plot: # Use current_partition to avoid shadowing
        for year in unique_years:
            ax = axes[plot_idx]

            # Filter data for the current partition and year
            current_year_data = monthly_partition_wait_counts_raw[
                monthly_partition_wait_counts_raw.index.get_level_values('YearMonth').map(lambda x: x.year) == year
            ]
            
            # Ensure the partition exists in the current_year_data before attempting .xs()
            if current_partition not in current_year_data.index.get_level_values('Partition'):
                current_plot_data = pd.DataFrame() # Create an empty DataFrame if partition has no data for the year
            else:
                current_plot_data = current_year_data.xs(current_partition, level='Partition', drop_level=False)
                # Drop the 'Partition' level to make 'YearMonth' the sole index for plotting
                current_plot_data = current_plot_data.droplevel('Partition') 
                # If after droplevel, the dataframe became empty, then set to empty for correct handling
                if current_plot_data.empty:
                    current_plot_data = pd.DataFrame()


            if current_plot_data.empty:
                ax.set_title(f'No Long Waits: {current_partition} - {year}', fontsize=12)
                ax.axis('off') # Hide axes if no data
                plot_idx += 1
                continue

            # Convert 'YearMonth' PeriodIndex to string for x-axis labels
            current_plot_data.index = current_plot_data.index.to_timestamp().strftime('%b') # e.g., Jan, Feb

            # Calculate total jobs for each wait category for sorting the legend for THIS subplot
            wait_category_totals_subplot = current_plot_data.sum().sort_values(ascending=True)

            # Reindex columns to ensure consistent stacking and legend order based on totals
            categories_to_plot = wait_category_totals_subplot[wait_category_totals_subplot > 0].index
            current_plot_data = current_plot_data[categories_to_plot]

            # --- Plotting on the current subplot ---
            current_plot_data.plot(kind='bar', stacked=True, ax=ax, cmap='viridis', alpha=0.9)
            
            # --- Apply Subplot Specific Formatting ---
            ax.set_title(f'{current_partition} - {year}', fontsize=14, fontweight='bold')
            ax.set_xlabel('Month', fontsize=11)
            ax.set_ylabel('Number of Jobs', fontsize=11)
            
            # --- APPLY CUSTOM Y-AXIS LIMITS BASED ON YEAR (FIXED) ---
            if year == 2023: # Changed from "2023" to 2023
                ax.set_ylim(0, 7500)
            elif year == 2024: # Changed from "2024" to 2024
                ax.set_ylim(0, 20000)
            elif year == 2025: # Changed from "2025" to 2025
                ax.set_ylim(0, 750)
            else:
                # Fallback to dynamic limit if year is not specified
                ax.set_ylim(0, max_job_count * 1.1)

            ax.tick_params(axis='x', rotation=45, labelsize=9)
            ax.tick_params(axis='y', labelsize=9)
            ax.grid(True, linestyle='--', alpha=0.7, axis='y')

            # --- Update Legend for the current subplot ---
            legend_labels = []
            handles, labels = ax.get_legend_handles_labels()
            
            sorted_labels_for_legend = [lbl for lbl in wait_category_totals_subplot.index if lbl in labels]
            label_to_handle = dict(zip(labels, handles))
            sorted_handles = [label_to_handle[lbl] for lbl in sorted_labels_for_legend]


            for label in sorted_labels_for_legend:
                total_jobs = wait_category_totals_subplot.get(label, 0)
                legend_labels.append(f"{label} ({total_jobs:,})")

            ax.legend(sorted_handles, legend_labels, title='Wait Time Category (Total Jobs)', 
                      loc='upper left', bbox_to_anchor=(1.02, 1),
                      ncol=1,
                      fontsize=9,
                      frameon=False)
            
            plot_idx += 1
            
    # --- Clean up any unused subplots ---
    for j in range(plot_idx, len(axes)):
        fig.delaxes(axes[j])

    # --- Adjust overall layout ---
    fig.suptitle('Monthly Breakdown of Long Wait Times (>4h) by Partition and Year', 
                 fontsize=18, fontweight='bold', y=1.02)
    plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Adjust layout to make space for suptitle
    plt.show()






partition = 'gpu'
plot_yearly_partition_long_wait_breakdown(df, partition=partition)  # Call the function without a specific partition to plot all partitions

In [ ]:
def plot_long_wait_distribution(df: pd.DataFrame):
    """
    Filters jobs with wait times > 4 hours and plots their distribution across
    finer-grained wait time categories as a bar chart.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing job data with a 'Wait' column.
    """
    df_clean_wait = df.dropna(subset=['Wait'])
    four_hours = pd.Timedelta(hours=4)
    long_wait_jobs_df = df_clean_wait[df_clean_wait['Wait'] > four_hours].copy()

    if long_wait_jobs_df.empty:
        print("No jobs found with wait times greater than 4 hours to plot detailed distribution.")
        return

    # Define bins and labels for wait times > 4 hours
    bins = [
        four_hours,               # Start from >4 hours
        pd.Timedelta(hours=12),
        pd.Timedelta(hours=24),
        pd.Timedelta(hours=48),
        pd.Timedelta.max          # Up to the max wait time
    ]
    labels = [
        '4-12 hours',
        '12-24 hours',
        '24-48 hours',
        '>48 hours'
    ]

    long_wait_jobs_df['WaitCategory'] = pd.cut(long_wait_jobs_df['Wait'],
                                              bins=bins,
                                              labels=labels,
                                              right=True, # Include the right-most bin edge
                                              include_lowest=True) # Include values equal to the lower bound (4 hours)


    wait_counts = long_wait_jobs_df['WaitCategory'].value_counts().sort_index()
    # Convert series to DataFrame for common_plot
    plot_df = wait_counts.reset_index()
    plot_df.columns = ['WaitCategory', 'JobCount']
    
    # Ensure consistent order of categories for plotting
    plot_df['WaitCategory'] = pd.Categorical(plot_df['WaitCategory'], categories=labels, ordered=True)
    plot_df = plot_df.sort_values('WaitCategory')


    print("\n--- Distribution of Jobs with Wait Times > 4 Hours ---")
    common_plot(
        df=plot_df,
        x='WaitCategory',
        y='JobCount',
        title='Distribution of Jobs with Wait Times > 4 Hours',
        xlabel='Wait Time Category',
        ylabel='Number of Jobs',
        color='maroon',
        kind='bar',
        ylim=(0, plot_df['JobCount'].max() * 1.1)
    )
    
# --- NEW: Plotting distribution of jobs with wait times > 4 hours (Bar Chart) ---
plot_long_wait_distribution(df)

In [ ]:
def plot_long_wait_distribution_D(df: pd.DataFrame):
    """
    Filters jobs with wait times > 4 hours and plots their distribution across
    finer-grained wait time categories as a doughnut chart with percentages.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing job data with a 'Wait' column.
    """
    df_clean_wait = df.dropna(subset=['Wait'])
    four_hours = pd.Timedelta(hours=4)
    long_wait_jobs_df = df_clean_wait[df_clean_wait['Wait'] > four_hours].copy()

    if long_wait_jobs_df.empty:
        print("No jobs found with wait times greater than 4 hours to plot detailed distribution.")
        return

    # Define bins and labels for wait times > 4 hours
    bins = [
        four_hours,               # Start from >4 hours
        pd.Timedelta(hours=12),
        pd.Timedelta(hours=24),
        pd.Timedelta(hours=48),
        pd.Timedelta.max          # Up to the max wait time
    ]
    labels = [
        '4-12 hours',
        '12-24 hours',
        '24-48 hours',
        '>48 hours'
    ]

    long_wait_jobs_df['WaitCategory'] = pd.cut(long_wait_jobs_df['Wait'],
                                              bins=bins,
                                              labels=labels,
                                              right=True, # Include the right-most bin edge
                                              include_lowest=True) # Include values equal to the lower bound (4 hours)


    wait_counts = long_wait_jobs_df['WaitCategory'].value_counts().sort_index()

    # Define colors for the doughnut chart (adjust for 4 categories)
    colors = plt.cm.Set2.colors[:len(labels)] # Using plasma colormap for distinction


    print("\n--- Distribution of Jobs with Wait Times > 4 Hours (Doughnut Chart) ---")
    fig, ax = plt.subplots(figsize=(8, 8))

    wedges, texts, autotexts = ax.pie(wait_counts,
                                      autopct='%1.1f%%',
                                      startangle=90,
                                      colors=colors,
                                      wedgeprops=dict(width=0.3, edgecolor='w'))

    centre_circle = plt.Circle((0,0), 0.70, fc='white')
    fig.gca().add_artist(centre_circle)

    for autotext in autotexts:
        autotext.set_fontsize(12)
        autotext.set_color('black')

    ax.legend(wedges, wait_counts.index,
              title="Wait Time Category",
              loc="upper center",
              bbox_to_anchor=(0.5, -0.05),
              ncol=len(wait_counts.index),
              fontsize=10,
              frameon=False)

    ax.set_title('Distribution of Jobs with Wait Times > 4 Hours', fontsize=14)
    ax.axis('equal')

    plt.tight_layout(rect=[0, 0.1, 1, 1])
    plt.show()
    
plot_long_wait_distribution_D(df)

In [ ]:
def plot_capacity_cpus_daily(df, weeks=1):
    '''
    Plot the total available and requested CPU capacity daily.
    Total configured capacity is 7296 + 1280 = 9088 CPUs

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have 'Start' (datetime), 'AllocCPUS', and 'User' columns.
    - weeks (int): The number of past weeks for which to plot the data. Defaults to 1.
    '''
    # Filter df for the specified number of weeks
    filter_date = pd.Timestamp.now() - pd.DateOffset(weeks=weeks)
    df_filtered = df[df['Start'] >= filter_date].copy()

    print(f"Plotting CPU capacity for the last {weeks} weeks")
    print(f"Total Jobs in the last {weeks} weeks: {len(df_filtered)}")

    # Group by date and calculate total requested CPU capacity
    daily_cpu_capacity = df_filtered.groupby(df_filtered['Start'].dt.date).agg(
        TotalRequestedCPUs=('AllocCPUS', 'sum')
    ).reset_index()

    # Convert 'Start' column in daily_cpu_capacity back to datetime objects for plotting
    daily_cpu_capacity['Start'] = pd.to_datetime(daily_cpu_capacity['Start'])

    print(daily_cpu_capacity.head())

    # Plotting using matplotlib and seaborn for multiple lines on one plot
    fig, current_ax = plt.subplots(figsize=(12, 6))

    # Plot requested CPU capacity
    sns.lineplot(data=daily_cpu_capacity, x='Start', y='TotalRequestedCPUs',
                 label='Total Requested CPUs', color='dodgerblue', linewidth=2, ax=current_ax)

    # Total configured CPU capacity
    configured_capacity_cpus = 9088
    # Plot a horizontal line for the total configured CPU capacity
    current_ax.axhline(y=configured_capacity_cpus, color='red', linestyle='--', label=f'Total Configured CPUs ({configured_capacity_cpus})', linewidth=2)

    # # Add configured capacity number on the plot above the red horizontal line
    # x_min, x_max = current_ax.get_xlim()
    # y_text_pos = configured_capacity_cpus + (current_ax.get_ylim()[1] - configured_capacity_cpus) * 0.02
    # current_ax.text(x_min + (x_max - x_min) * 0.5, y_text_pos,
    #                 f'{configured_capacity_cpus} CPUs',
    #                 color='red', ha='center', va='bottom', fontsize=10, fontweight='bold')

    # Set title and labels with consistent formatting
    current_ax.set_title('Daily Requested vs Configured CPU Capacity', fontsize=16)
    current_ax.set_xlabel('Date', fontsize=12)
    current_ax.set_ylabel('Capacity (CPUs)', fontsize=12)

    # Rotate x-axis labels for better readability of dates
    current_ax.tick_params(axis='x', rotation=0)
    current_ax.tick_params(axis='y', labelsize=10)

    # Add a grid for easier reading of values
    current_ax.grid(True, linestyle='--', alpha=0.7)

    # Set the y-axis limit based on configured capacity
    # current_ax.set_ylim(bottom=0, top=configured_capacity_cpus * 1.05) # Set top slightly above max configured capacity

    # Place legend below the plot
    current_ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2),
                      ncol=1,
                      fontsize=10,
                      frameon=False)

    # Adjust layout to prevent overlapping elements, make space for the legend below
    # plt.tight_layout(rect=[0, 0.1, 1, 1])

    plt.show()
    
plot_capacity_cpus_daily(df, weeks=104) # Example for last 104 weeks (2 years)

In [ ]:
def plot_capacity_cpus_daily(df, weeks=1, requested_cpu_threshold: float = 200000):
    '''
    Plot the total available and requested CPU capacity daily.
    Total configured capacity is 7296 + 1280 = 9088 CPUs

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have 'Start' (datetime), 'AllocCPUS', and 'User' columns.
    - weeks (int): The number of past weeks for which to plot the data. Defaults to 1.
    - requested_cpu_threshold (float): The maximum allowed TotalRequestedCPUs for a day.
                                      Days exceeding this will be dropped.
    '''
    # Filter df for the specified number of weeks
    filter_date = pd.Timestamp.now() - pd.DateOffset(weeks=weeks)
    df_filtered = df[df['Start'] >= filter_date].copy()

    print(f"Plotting CPU capacity for the last {weeks} weeks")
    print(f"Total Jobs in the last {weeks} weeks: {len(df_filtered)}")

    # Group by date and calculate total requested CPU capacity
    daily_cpu_capacity = df_filtered.groupby(df_filtered['Start'].dt.date).agg(
        TotalRequestedCPUs=('AllocCPUS', 'sum')
    ).reset_index()

    # Convert 'Start' column in daily_cpu_capacity back to datetime objects for plotting
    daily_cpu_capacity['Start'] = pd.to_datetime(daily_cpu_capacity['Start'])

    # Filter out data points where TotalRequestedCPUs is more than the threshold
    original_rows = len(daily_cpu_capacity)
    daily_cpu_capacity = daily_cpu_capacity[daily_cpu_capacity['TotalRequestedCPUs'] <= requested_cpu_threshold].copy()
    filtered_rows = len(daily_cpu_capacity)

    if original_rows > filtered_rows:
        print(f"Filtered out {original_rows - filtered_rows} days where TotalRequestedCPUs exceeded {requested_cpu_threshold:.0f} CPUs.")
    else:
        print("No days filtered based on TotalRequestedCPUs threshold.")

    print(daily_cpu_capacity.head())

    # Plotting using matplotlib and seaborn for multiple lines on one plot
    fig, current_ax = plt.subplots(figsize=(12, 6))

    # Plot requested CPU capacity
    sns.lineplot(data=daily_cpu_capacity, x='Start', y='TotalRequestedCPUs',
                 label='Total Requested CPUs', color='dodgerblue', linewidth=2, ax=current_ax)

    # Total configured CPU capacity
    configured_capacity_cpus = 9088
    # Plot a horizontal line for the total configured CPU capacity
    current_ax.axhline(y=configured_capacity_cpus, color='red', linestyle='--', label=f'Total Configured CPUs ({configured_capacity_cpus})', linewidth=2)

    # Add text label for the horizontal line
    text_x_position_cpu = daily_cpu_capacity['Start'].min()
    text_y_position_cpu = configured_capacity_cpus * 1.01
    current_ax.text(text_x_position_cpu, text_y_position_cpu, f'Configured Capacity: {configured_capacity_cpus} CPUs',
                    color='red', fontsize=10, verticalalignment='bottom', horizontalalignment='left',
                    bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', boxstyle='round,pad=0.2'))

    # Set title and labels with consistent formatting
    current_ax.set_title('Daily Requested vs Configured CPU Capacity', fontsize=16)
    current_ax.set_xlabel('Date', fontsize=12)
    current_ax.set_ylabel('Capacity (CPUs)', fontsize=12)

    # Rotate x-axis labels for better readability of dates
    current_ax.tick_params(axis='x', rotation=45) # Use rotation only
    current_ax.tick_params(axis='y', labelsize=10)

    # Add a grid for easier reading of values
    current_ax.grid(True, linestyle='--', alpha=0.7)

    # Set the y-axis limit based on configured capacity and requested data
    max_y_cpu = max(daily_cpu_capacity['TotalRequestedCPUs'].max(), configured_capacity_cpus)
    current_ax.set_ylim(bottom=0, top=max_y_cpu * 1.1)

    # Place legend below the plot
    current_ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2),
                      ncol=1,
                      fontsize=10,
                      frameon=False)

    # Adjust layout to prevent overlapping elements, make space for the legend below
    plt.tight_layout(rect=[0, 0.1, 1, 1])

    plt.show()
    
plot_capacity_cpus_daily(df, weeks=104, requested_cpu_threshold=200000) # Example for last 104 weeks (2 years)

In [ ]:
def plot_capacity_cpus_daily(df: pd.DataFrame, weeks: int = 1, requested_cpu_threshold: float = 20000, partitions: Optional[List[str]] = None):
    '''
    Plot the total available and requested CPU capacity daily for selected partitions in subplots.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data.
                         Expected to have 'Start' (datetime), 'AllocCPUS', and 'Partition' columns.
    - weeks (int): The number of past weeks for which to plot the data. Defaults to 1.
    - requested_cpu_threshold (float): The maximum allowed TotalRequestedCPUs for a day.
                                      Days exceeding this will be dropped.
    - partitions (Optional[List[str]]): A list of partition names to plot.
                                          If None, requested capacity for all *unique* partitions will be plotted
                                          in separate subplots.
    '''
    # Define configured CPU capacity for each known partition
    # Based on the sum 7296 + 1280 = 9088 CPUs
    partition_configured_capacity_cpus = {
        'parallel': 256*28+250/2,   # Example: Assuming 'cpu' partition has this capacity
        'gpu': 256*3,    # Example: Assuming 'gpu' partition has this capacity
        'longrun': 256*2,  # Example: Assuming 'short' partition has this capacity
        'quicktest': 256*4,
        'bigmem': 128*4,
        # Add other partitions and their capacities as needed.
        # A fallback for unknown partitions could be added, e.g., 0 or the total system capacity.
    }
    # Fallback for partitions not explicitly defined, or a sensible default
    default_configured_capacity_cpus = 8052+1280 # Using the sum as a generic fallback if a partition isn't mapped

    # Filter df for the specified number of weeks
    filter_date = pd.Timestamp.now() - pd.DateOffset(weeks=weeks)
    df_filtered_time = df[df['Start'] >= filter_date].copy()

    # Determine which partitions to plot
    if partitions:
        partitions_to_plot = [p for p in partitions if p in df_filtered_time['Partition'].unique()]
        if not partitions_to_plot:
            print(f"No data found for the specified partitions: {', '.join(partitions)} in the last {weeks} weeks after time filtering.")
            return
    else:
        partitions_to_plot = df_filtered_time['Partition'].unique().tolist()
        if not partitions_to_plot:
            print(f"No partitions found in the data for the last {weeks} weeks.")
            return

    # Set up subplots
    num_partitions = len(partitions_to_plot)
    if num_partitions == 0:
        print("No partitions to plot.")
        return

    n_cols = min(3, num_partitions) # Max 3 columns
    n_rows = (num_partitions + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 8, n_rows * 6), squeeze=False)
    axes = axes.flatten()

    # Iterate through each partition and create a subplot
    for i, partition_name in enumerate(partitions_to_plot):
        ax = axes[i]

        # Get the configured CPU capacity for the current partition, with a fallback
        current_configured_capacity_cpus = partition_configured_capacity_cpus.get(partition_name, default_configured_capacity_cpus)

        # Filter data for the current partition
        df_partition = df_filtered_time[df_filtered_time['Partition'] == partition_name].copy()

        if df_partition.empty:
            ax.set_title(f'No Data for {partition_name}')
            ax.set_xlabel('Date')
            ax.set_ylabel('Capacity (CPUs)')
            ax.tick_params(axis='x', rotation=45, ha='right')
            ax.text(0.5, 0.5, 'No data', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
            continue

        # Group by date and calculate total requested CPU capacity for this partition
        daily_cpu_capacity_partition = df_partition.groupby(df_partition['Start'].dt.date)['AllocCPUS'].sum().reset_index(name='TotalRequestedCPUs')
        daily_cpu_capacity_partition.rename(columns={'Start': 'Date'}, inplace=True)
        daily_cpu_capacity_partition['Date'] = pd.to_datetime(daily_cpu_capacity_partition['Date'])

        # Filter out data points where TotalRequestedCPUs is more than the threshold for the current partition
        original_rows_part = len(daily_cpu_capacity_partition)
        daily_cpu_capacity_partition = daily_cpu_capacity_partition[daily_cpu_capacity_partition['TotalRequestedCPUs'] <= requested_cpu_threshold].copy()
        filtered_rows_part = len(daily_cpu_capacity_partition)

        if original_rows_part > filtered_rows_part:
            print(f"Filtered out {original_rows_part - filtered_rows_part} days for partition '{partition_name}' where TotalRequestedCPUs exceeded {requested_cpu_threshold:.0f} CPUs.")

        if daily_cpu_capacity_partition.empty:
            ax.set_title(f'No Plottable Data for {partition_name}')
            ax.set_xlabel('Date')
            ax.set_ylabel('Capacity (CPUs)')
            ax.tick_params(axis='x', rotation=0)#, ha='right')
            ax.text(0.5, 0.5, 'No plottable data', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
            continue

        # Plot requested CPU capacity for the current partition
        sns.lineplot(data=daily_cpu_capacity_partition, x='Date', y='TotalRequestedCPUs',
                     label=f'Requested CPUs ({partition_name})', linewidth=2, ax=ax)

        # Plot a horizontal line for the partition's configured CPU capacity
        ax.axhline(y=current_configured_capacity_cpus, color='red', linestyle='--', label=f'Configured Total ({current_configured_capacity_cpus} CPUs)', linewidth=2)

        # Add text label for the horizontal line (adjusted for subplot)
        text_x_position = daily_cpu_capacity_partition['Date'].min()
        text_y_position = current_configured_capacity_cpus * 1.01
        ax.text(text_x_position, text_y_position, f'Configured: {current_configured_capacity_cpus} CPUs',
                color='red', fontsize=8, verticalalignment='bottom', horizontalalignment='left',
                bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', boxstyle='round,pad=0.2'))

        # Set subplot title and labels
        ax.set_title(f'Daily CPU Capacity for {partition_name}', fontsize=12)
        ax.set_xlabel('Date', fontsize=10)
        ax.set_ylabel('Capacity (CPUs)', fontsize=10)

        # Rotate x-axis labels
        ax.tick_params(axis='x', rotation=0)#, ha='right')
        ax.tick_params(axis='y', labelsize=8)

        # Add a grid
        ax.grid(True, linestyle='--', alpha=0.7)

        # Set the y-axis limit
        max_y_partition = max(daily_cpu_capacity_partition['TotalRequestedCPUs'].max(), current_configured_capacity_cpus)
        ax.set_ylim(bottom=0, top=max_y_partition * 1.1)

        # Place legend on each subplot
        ax.legend(loc='upper left', fontsize=8, frameon=False)

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # Overall title for the figure
    overall_title_suffix = f" for selected partitions: {', '.join(partitions_to_plot)}" if partitions else " for All Partitions"
    fig.suptitle(f'Daily Requested CPU Capacity Per Partition{overall_title_suffix}', fontsize=18, y=1.02)

    plt.tight_layout(rect=[0, 0, 1, 0.98])
    file_name = f'daily_requested_cpu_capacity_per_partition{overall_title_suffix.replace(" ", "_").replace(",", "").lower()}.png'
    plt.show()
    # plt.savefig(file_name)
    plt.close()
    print(f"Plot '{file_name}' saved successfully.")
    
plot_capacity_cpus_daily(df, weeks=104) # Call the function with 104 weeks (2 years) of data


In [ ]:
def plot_monthly_usage(df):
    '''
    Plots the monthly job usage by user account as a stacked area chart.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data,
                         expected to have 'YearMonth' and 'Account' columns.
    '''
    if df.empty:
        print("Cannot plot monthly usage: DataFrame is empty.")
        return

    # Convert 'Year' and 'Month' to integers and then to a period for easier grouping/plotting
    df['Year'] = df['Year'].fillna(0).astype(int) # Handle potential NaN values
    df['Month'] = df['Month'].fillna(0).astype(int) # Handle potential NaN values
    df['YearMonth'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-01', errors='coerce')

    # Drop rows where 'YearMonth' could not be parsed (e.g., if Year or Month was invalid)
    df.dropna(subset=['YearMonth'], inplace=True)
    
    # Filter data for the last two years (assuming current year is 2025 based on sample data)
    # Adjust this based on the actual current year when the code is run
    current_year = pd.Timestamp.now().year
    start_date_two_years_ago = pd.Timestamp(year=current_year - 2, month=pd.Timestamp.now().month, day=1)
    df_recent = df[df['YearMonth'] >= start_date_two_years_ago].copy()

    # Basic data validation
    if 'YearMonth' not in df.columns:
        print("Error: 'YearMonth' column not found. Please ensure data preprocessing has been run.")
        return
    if 'Account' not in df.columns:
        print("Error: 'Account' column not found. Please ensure data preprocessing has been run.")
        return

    # Group by 'YearMonth' and 'Account' to count jobs
    monthly_usage = df.groupby(['YearMonth', 'Account']).size().unstack(fill_value=0)

    # Convert 'YearMonth' PeriodIndex to string for better x-axis labeling with rotation
    monthly_usage.index = monthly_usage.index.astype(str)

    fig, current_ax = plt.subplots(figsize=(14, 9)) # Consistent figure size

    # Plot as stacked area chart
    monthly_usage.plot(kind='area', stacked=True, ax=current_ax, cmap='viridis', alpha=0.8) # Adjusted alpha for visibility

    # Apply consistent formatting
    current_ax.set_title('Monthly Job Usage by Account', fontsize=16, fontweight='bold')
    current_ax.set_xlabel('Month (YYYY-MM)', fontsize=12)
    current_ax.set_ylabel('Number of Jobs', fontsize=12)

    # Rotate x-axis labels for better readability
    current_ax.tick_params(axis='x', rotation=0, labelsize=10)
    current_ax.tick_params(axis='y', labelsize=10)

    # Add a grid for easier reading of values
    current_ax.grid(True, linestyle='--', alpha=0.7)

    # Place legend below the plot, consistent with other plots
    current_ax.legend(title='Account', loc='upper center', bbox_to_anchor=(0.5, -0.4),
                      ncol=min(4, len(monthly_usage.columns)), # Adjust number of columns based on accounts
                      fontsize=10,
                      frameon=False) # Remove frame around legend

    # Adjust layout to prevent overlapping elements, make space for the legend below
    plt.tight_layout(rect=[0, 0.1, 1, 1])

    plt.show()
    
plot_monthly_usage(df)

In [ ]:
def plot_monthly_usage(df):
    '''
    Plots the monthly job usage by user account as a stacked bar chart.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data,
                         expected to have 'YearMonth' and 'Account' columns.
    '''
    if df.empty:
        print("Cannot plot monthly usage: DataFrame is empty.")
        return

    # Convert 'Year' and 'Month' to integers and then to a period for easier grouping/plotting
    df['Year'] = df['Year'].fillna(0).astype(int) # Handle potential NaN values
    df['Month'] = df['Month'].fillna(0).astype(int) # Handle potential NaN values
    df['YearMonth'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-01', errors='coerce')

    # Drop rows where 'YearMonth' could not be parsed (e.g., if Year or Month was invalid)
    df.dropna(subset=['YearMonth'], inplace=True)
    
    # Filter data for the last two years (assuming current year is 2025 based on sample data)
    # Adjust this based on the actual current year when the code is run
    current_year = pd.Timestamp.now().year
    start_date_two_years_ago = pd.Timestamp(year=current_year - 2, month=pd.Timestamp.now().month, day=1)
    df_recent = df[df['YearMonth'] >= start_date_two_years_ago].copy()

    # Basic data validation
    if 'YearMonth' not in df.columns:
        print("Error: 'YearMonth' column not found. Please ensure data preprocessing has been run.")
        return
    if 'Account' not in df.columns:
        print("Error: 'Account' column not found. Please ensure data preprocessing has been run.")
        return

    # Group by 'YearMonth' and 'Account' to count jobs
    monthly_usage = df.groupby(['YearMonth', 'Account']).size().unstack(fill_value=0)

    # Convert 'YearMonth' PeriodIndex to string for better x-axis labeling with rotation
    monthly_usage.index = monthly_usage.index.astype(str)

    fig, current_ax = plt.subplots(figsize=(14, 10)) # Consistent figure size

    # Use a stacked bar chart instead of a stacked area chart
    # Use a different colormap to ensure distinct colors for each account
    monthly_usage.plot(kind='bar', stacked=True, ax=current_ax, cmap='tab20_r', alpha=0.9)
    
    # yscale is log
    # current_ax.set_yscale('log')  # Use logarithmic scale for better visibility of smaller values

    # Apply consistent formatting
    current_ax.set_title('Monthly Job Usage by Account', fontsize=16, fontweight='regular')
    current_ax.set_xlabel('Month (YYYY-MM)', fontsize=12)
    current_ax.set_ylabel('Number of Jobs', fontsize=12)

    # Rotate x-axis labels for better readability
    current_ax.tick_params(axis='x', rotation=45, labelsize=10)
    current_ax.tick_params(axis='y', labelsize=10)

    # Add a grid for easier reading of values
    current_ax.grid(True, linestyle='--', alpha=0.7, axis='y') # Grid on y-axis for bar chart

    # Place legend below the plot, consistent with other plots
    current_ax.legend(title='Account', loc='upper center', bbox_to_anchor=(0.5, -0.4),
                      ncol=min(4, len(monthly_usage.columns)), # Adjust number of columns based on accounts
                      fontsize=10,
                      frameon=False) # Remove frame around legend

    # Adjust layout to prevent overlapping elements, make space for the legend below
    plt.tight_layout(rect=[0, 0.1, 1, 1])

    plt.show()
    
plot_monthly_usage(df)

In [ ]:
def plot_monthly_usage(df):
    '''
    Plots the monthly job usage by user account as a stacked bar chart.
    The accounts in the legend are sorted by their total job count in ascending order,
    and the legend displays the total job count for each account.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data,
                         expected to have 'Year' (int), 'Month' (int), and 'Account' columns.
    '''
    if df.empty:
        print("Cannot plot monthly usage: DataFrame is empty.")
        return

    # Convert 'Year' and 'Month' to integers and then to a period for easier grouping/plotting
    # Ensure 'Year' and 'Month' are treated as numeric for fillna
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').fillna(0).astype(int)
    df['Month'] = pd.to_numeric(df['Month'], errors='coerce').fillna(0).astype(int)
    
    # Create 'YearMonth' column
    df['YearMonth'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-01', errors='coerce')

    # Drop rows where 'YearMonth' could not be parsed (e.g., if Year or Month was invalid)
    df.dropna(subset=['YearMonth'], inplace=True)
    
    # Filter data for the last two years (assuming current year is 2025)
    current_year = pd.Timestamp.now().year
    start_date_two_years_ago = pd.Timestamp(year=current_year - 2, month=pd.Timestamp.now().month, day=1)
    df_recent = df[df['YearMonth'] >= start_date_two_years_ago].copy()

    # Basic data validation for filtered DataFrame
    if df_recent.empty:
        print("No recent data available for plotting (last two years).")
        return
    if 'YearMonth' not in df_recent.columns:
        print("Error: 'YearMonth' column not found in recent data. Please ensure data preprocessing has been run.")
        return
    if 'Account' not in df_recent.columns:
        print("Error: 'Account' column not found in recent data. Please ensure data preprocessing has been run.")
        return

    # Group by 'YearMonth' and 'Account' to count jobs
    monthly_usage = df_recent.groupby(['YearMonth', 'Account']).size().unstack(fill_value=0)

    # Calculate the total jobs per account for sorting
    account_totals = monthly_usage.sum().sort_values(ascending=True)

    # Reindex the columns of monthly_usage based on the sorted totals
    monthly_usage = monthly_usage[account_totals.index]

    # Convert 'YearMonth' PeriodIndex to string for better x-axis labeling with rotation
    monthly_usage.index = monthly_usage.index.strftime('%Y-%m') # Format as YYYY-MM

    fig, current_ax = plt.subplots(figsize=(14, 10))

    # Use a stacked bar chart
    # Use a reversed tab20 colormap for distinct and professional colors
    monthly_usage.plot(kind='bar', stacked=True, ax=current_ax, cmap='tab20_r', alpha=0.9)
    
    # Apply consistent formatting
    current_ax.set_title('Monthly Job Usage by Account (Last Two Years)', fontsize=16, fontweight='bold')
    current_ax.set_xlabel('Month (YYYY-MM)', fontsize=12)
    current_ax.set_ylabel('Number of Jobs', fontsize=12)

    # Rotate x-axis labels for better readability
    current_ax.tick_params(axis='x', rotation=90, labelsize=10)
    current_ax.tick_params(axis='y', labelsize=10)

    # Add a grid for easier reading of values
    current_ax.grid(True, linestyle='--', alpha=0.7, axis='y')

    # --- Update Legend to Display Job Counts ---
    # Create custom legend handles and labels with counts
    legend_labels = []
    handles, labels = current_ax.get_legend_handles_labels()
    
    # Sort handles and labels based on the sorted account_totals
    sorted_labels = account_totals.index.tolist()
    sorted_handles = [handles[labels.index(lbl)] for lbl in sorted_labels]

    for label in sorted_labels:
        total_jobs = account_totals[label]
        legend_labels.append(f"{label} ({total_jobs:,})") # Add comma for thousands separator

    current_ax.legend(sorted_handles, legend_labels, title='Account (Total Jobs)', loc='upper center', bbox_to_anchor=(0.5, -0.2),
                      ncol=min(4, len(monthly_usage.columns)),
                      fontsize=10,
                      frameon=False) # Remove frame around legend

    # Adjust layout to prevent overlapping elements, make space for the legend below
    plt.tight_layout(rect=[0, 0.15, 1, 0.95]) # Adjust rect bottom to give more space for legend

    plt.show()
    
    
plot_monthly_usage(df)  # Call the function to plot monthly usage

In [ ]:
def plot_monthly_unique_users(df):
    '''
    Plots the monthly number of unique users by user account as a stacked bar chart.
    The accounts in the legend are sorted by their total unique user count in ascending order,
    and the legend displays the total unique user count for each account.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data,
                         expected to have 'Year' (int), 'Month' (int), 'Account',
                         and a 'User' (or similar) column.
    '''
    if df.empty:
        print("Cannot plot monthly unique user usage: DataFrame is empty.")
        return

    # --- Data Preprocessing (similar to job usage plot) ---
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').fillna(0).astype(int)
    df['Month'] = pd.to_numeric(df['Month'], errors='coerce').fillna(0).astype(int)
    
    df['YearMonth'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-01', errors='coerce')
    df.dropna(subset=['YearMonth'], inplace=True)
    
    current_year = pd.Timestamp.now().year
    start_date_two_years_ago = pd.Timestamp(year=current_year - 2, month=pd.Timestamp.now().month, day=1)
    df_recent = df[df['YearMonth'] >= start_date_two_years_ago].copy()

    # --- Basic Data Validation ---
    if df_recent.empty:
        print("No recent data available for plotting unique users (last two years).")
        return
    if 'YearMonth' not in df_recent.columns:
        print("Error: 'YearMonth' column not found in recent data. Please ensure data preprocessing has been run.")
        return
    if 'Account' not in df_recent.columns:
        print("Error: 'Account' column not found in recent data. Please ensure data preprocessing has been run.")
        return
    if 'User' not in df_recent.columns: # <--- CRITICAL: Check for User column
        print("Error: 'User' column not found. Cannot plot unique users without user identification.")
        print("Please ensure your DataFrame has a 'User' column, or update the code to use the correct user identifier column.")
        return

    # --- Key Change: Group by 'User' and count unique users ---
    monthly_unique_users = df_recent.groupby(['YearMonth', 'Account'])['User'].nunique().unstack(fill_value=0)

    # Calculate total unique users per account for sorting the legend
    account_unique_user_totals = monthly_unique_users.sum().sort_values(ascending=True)

    # Reindex the columns of monthly_unique_users based on the sorted totals
    monthly_unique_users = monthly_unique_users[account_unique_user_totals.index]

    # Convert 'YearMonth' to string for better x-axis labeling
    monthly_unique_users.index = monthly_unique_users.index.strftime('%Y-%m')

    # --- Plotting ---
    fig, current_ax = plt.subplots(figsize=(14, 10))

    # Using a reversed 'tab20' colormap for a professional and distinct look
    monthly_unique_users.plot(kind='bar', stacked=True, ax=current_ax, cmap='tab20_r', alpha=0.9)
    
    # --- Apply Consistent Formatting ---
    current_ax.set_title('Monthly Unique User Activity by Account (Last Two Years)', fontsize=16, fontweight='bold')
    current_ax.set_xlabel('Month (YYYY-MM)', fontsize=12)
    current_ax.set_ylabel('Number of Unique Users', fontsize=12) # <--- Updated Y-axis label

    current_ax.tick_params(axis='x', rotation=90, labelsize=10)
    current_ax.tick_params(axis='y', labelsize=10)
    current_ax.grid(True, linestyle='--', alpha=0.7, axis='y')

    # --- Update Legend to Display Unique User Counts ---
    legend_labels = []
    handles, labels = current_ax.get_legend_handles_labels()
    
    # Sort handles and labels based on the sorted account_unique_user_totals
    sorted_labels = account_unique_user_totals.index.tolist()
    sorted_handles = [handles[labels.index(lbl)] for lbl in sorted_labels]

    for label in sorted_labels:
        total_unique_users = account_unique_user_totals[label]
        legend_labels.append(f"{label} ({total_unique_users:,})") # Format with thousands separator

    current_ax.legend(sorted_handles, legend_labels, title='Account (Total Unique Users)', loc='upper center', bbox_to_anchor=(0.5, -0.2),
                      ncol=min(4, len(monthly_unique_users.columns)),
                      fontsize=10,
                      frameon=False)

    plt.tight_layout(rect=[0, 0.15, 1, 0.95]) # Adjust layout for legend
    plt.show()


plot_monthly_unique_users(df)  # Call the function to plot monthly unique user usage

In [ ]:
import numpy as np
def plot_yearly_unique_users_subplots(df):
    '''
    Plots the monthly number of unique users by user account for each year
    as stacked bar charts in separate subplots. Accounts in each subplot's legend
    are sorted by their total unique user count for that specific year,
    and the legend displays the total unique user count.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data,
                         expected to have 'Year' (int), 'Month' (int), 'Account',
                         and a 'User' (or similar) column.
    '''
    if df.empty:
        print("Cannot plot yearly unique user usage: DataFrame is empty.")
        return

    # --- Data Preprocessing ---
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').fillna(0).astype(int)
    df['Month'] = pd.to_numeric(df['Month'], errors='coerce').fillna(0).astype(int)
    
    df['YearMonth'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-01', errors='coerce')
    df.dropna(subset=['YearMonth'], inplace=True)
    
    # Filter data for the last two years (or any relevant period if you want all years)
    # For year-wise subplots, let's get all years present in the data for flexibility
    # You can uncomment the current_year filtering if you only want the most recent years
    
    # current_year = pd.Timestamp.now().year
    # start_date_two_years_ago = pd.Timestamp(year=current_year - 2, month=pd.Timestamp.now().month, day=1)
    # df_relevant_years = df[df['YearMonth'] >= start_date_two_years_ago].copy()
    
    # For plotting all available years in subplots:
    df_relevant_years = df.copy()

    # --- Basic Data Validation for User ---
    if df_relevant_years.empty:
        print("No relevant data available for plotting unique users.")
        return
    if 'User' not in df_relevant_years.columns:
        print("Error: 'User' column not found. Cannot plot unique users without user identification.")
        print("Please ensure your DataFrame has a 'User' column, or update the code to use the correct user identifier column.")
        return
    if 'Account' not in df_relevant_years.columns:
        print("Error: 'Account' column not found.")
        return

    # Get unique sorted years for subplot creation
    unique_years = sorted(df_relevant_years['Year'].unique())
    if not unique_years:
        print("No valid years found in the DataFrame to plot.")
        return

    # Determine subplot grid dimensions
    num_years = len(unique_years)
    # Adjust cols and rows for better layout (e.g., max 2 or 3 columns)
    ncols = min(3, num_years)
    nrows = int(np.ceil(num_years / ncols))

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(8 * ncols, 6 * nrows), squeeze=False) # squeeze=False ensures axes is always 2D
    axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

    # Determine a common Y-axis limit for better comparison across years
    # This involves calculating max unique users across ALL months and accounts for ALL years
    max_y_lim = 0
    temp_grouped_data = df_relevant_years.groupby(['YearMonth', 'Account'])['User'].nunique().unstack(fill_value=0)
    if not temp_grouped_data.empty:
        max_y_lim = temp_grouped_data.sum(axis=1).max() # Max of summed unique users per month
        if max_y_lim == 0: # Avoid issues if all counts are zero
            max_y_lim = 1

    for i, year in enumerate(unique_years):
        ax = axes[i]
        df_year = df_relevant_years[df_relevant_years['Year'] == year].copy()

        if df_year.empty:
            ax.set_title(f'No Data for {year}', fontsize=14, fontweight='bold')
            ax.axis('off') # Hide axes if no data
            continue

        # Group by 'YearMonth' (which will effectively be just 'Month' for each year) and 'Account'
        monthly_unique_users_year = df_year.groupby(['YearMonth', 'Account'])['User'].nunique().unstack(fill_value=0)

        # Calculate total unique users per account for sorting the legend for THIS YEAR
        account_unique_user_totals_year = monthly_unique_users_year.sum().sort_values(ascending=True)

        # Reindex the columns for sorting in the plot
        monthly_unique_users_year = monthly_unique_users_year[account_unique_user_totals_year.index]

        # Convert 'YearMonth' to string, showing only month for x-axis
        monthly_unique_users_year.index = monthly_unique_users_year.index.strftime('%b') # E.g., Jan, Feb

        # --- Plotting on the current subplot ---
        monthly_unique_users_year.plot(kind='bar', stacked=True, ax=ax, cmap='tab20_r', alpha=0.9)
        
        # --- Apply Subplot Specific Formatting ---
        ax.set_title(f'Unique Users - {year}', fontsize=14, fontweight='bold')
        ax.set_xlabel('Month', fontsize=11)
        ax.set_ylabel('Number of Unique Users', fontsize=11)
        ax.set_ylim(0, max_y_lim * 1.1) # Apply consistent Y-axis limit
        ax.tick_params(axis='x', rotation=90, labelsize=9)
        ax.tick_params(axis='y', labelsize=9)
        ax.grid(True, linestyle='--', alpha=0.7, axis='y')

        # --- Update Legend for the current subplot ---
        legend_labels = []
        handles, labels = ax.get_legend_handles_labels()
        
        # Sort handles and labels based on the sorted account_unique_user_totals_year
        sorted_labels = account_unique_user_totals_year.index.tolist()
        sorted_handles = [handles[labels.index(lbl)] for lbl in sorted_labels]

        for label in sorted_labels:
            total_unique_users = account_unique_user_totals_year[label]
            legend_labels.append(f"{label} ({total_unique_users:,})")

        ax.legend(sorted_handles, legend_labels, title='Account (Total Unique Users)', 
                  loc='upper left', bbox_to_anchor=(1.02, 1), # Place legend outside the plot area
                  ncol=1, # One column for legend for cleaner look next to subplot
                  fontsize=9,
                  frameon=False)
        
    # --- Clean up any unused subplots if num_years is not a perfect multiple of ncols ---
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j]) # Remove empty subplots

    # --- Adjust overall layout ---
    fig.suptitle('Monthly Unique User Activity by Account - Yearly Breakdown', fontsize=18, fontweight='bold', y=1.02) # Main title
    plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Adjust overall figure layout, leave space for suptitle
    plt.show()


plot_yearly_unique_users_subplots(df)  # Call the function to plot yearly unique user usage in subplots

In [ ]:
def plot_total_unique_users_monthly_trend(df: pd.DataFrame):
    '''
    Plots the total number of unique users per month as a line trend.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame containing job data,
                         expected to have 'Year', 'Month', and 'User' columns.
    '''
    if df.empty:
        print("Cannot plot total unique users monthly trend: DataFrame is empty.")
        return
    
    
    # Ensure 'Start' column is datetime type for filtering
    df['Start'] = pd.to_datetime(df['Start'])
    
    # Filter data for the last two years using 'Start' datetime column
    current_date = pd.Timestamp.now()
    start_date_two_years_ago = pd.Timestamp(year=current_date.year - 2, month=current_date.month, day=1)
    df_recent = df[df['Start'] >= start_date_two_years_ago].copy()

    if df_recent.empty:
        print("No recent data available for total unique users monthly trend after filtering for the last 2 years.")
        return

    # Group by 'Year' and 'Month' and count unique Users
    total_unique_users_monthly = df_recent.groupby(['Year', 'Month'])['User'].nunique().reset_index(name='UniqueUserCount')

    # Create a datetime column from 'Year' and 'Month' for plotting
    # Explicitly convert 'Year' and 'Month' to int first
    total_unique_users_monthly['PlotDate'] = pd.to_datetime(
        total_unique_users_monthly['Year'].astype(int).astype(str) + '-' +
        total_unique_users_monthly['Month'].astype(int).astype(str) + '-01'
    )
    print(total_unique_users_monthly.head()) # Debugging line to check the data

    fig, current_ax = plt.subplots(figsize=(14, 7)) # Consistent figure size

    # Plot as a line chart using common_plot style
    common_plot(total_unique_users_monthly, 'PlotDate', 'UniqueUserCount',
                'Total Unique Users Per Month (Last 2 Years)', # Updated title for clarity
                'Month (YYYY-MM)', 'Number of Unique Users',
                color='teal', # Distinct color for this plot
                kind='bar',
                ax=current_ax)

    # # Format x-axis ticks to show month and year clearly
    # current_ax.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m'))
    # current_ax.tick_params(axis='x', rotation=45, labelsize=10)
    # current_ax.tick_params(axis='y', labelsize=10)

    # Add a grid for easier reading of values
    current_ax.grid(True, linestyle='--', alpha=0.7)
    
    # super title
    fig.suptitle('Total Unique Users Monthly Trend (Last 2 Years)', fontsize=16, fontweight='bold', y=1.02)

    # Adjust layout
    plt.tight_layout()
    plt.show()
    
plot_total_unique_users_monthly_trend(df)

In [ ]:
# Aggregate data for jobs per partition
jobs_per_partition = df['Partition'].value_counts().reset_index()
jobs_per_partition.columns = ['Partition', 'JobCount']

# Plotting jobs per partition
common_plot(
    df=jobs_per_partition,
    x='Partition',
    y='JobCount',
    title='Number of Jobs per Partition',
    xlabel='Partition',
    ylabel='Number of Jobs',
    color='skyblue',
    kind='bar'
)

# Aggregate data for total allocated CPUs per partition
alloc_cpus_per_partition = df.groupby('Partition')['AllocCPUS'].sum().reset_index()
alloc_cpus_per_partition.columns = ['Partition', 'TotalAllocCPUs']

# Plotting total allocated CPUs per partition
common_plot(
    df=alloc_cpus_per_partition,
    x='Partition',
    y='TotalAllocCPUs',
    title='Total Allocated CPUs per Partition',
    xlabel='Partition',
    ylabel='Total Allocated CPUs',
    color='lightcoral',
    kind='bar'
)

# Aggregate data for total requested memory per partition
req_mem_per_partition = df.groupby('Partition')['ReqMem'].sum().reset_index()
req_mem_per_partition.columns = ['Partition', 'TotalReqMem']

# Plotting total requested memory per partition
common_plot(
    df=req_mem_per_partition,
    x='Partition',
    y='TotalReqMem',
    title='Total Requested Memory (MB) per Partition',
    xlabel='Partition',
    ylabel='Total Requested Memory (MB)',
    color='lightgreen',
    kind='bar'
)

In [ ]:
def plot_partition_usage_over_time(df: pd.DataFrame, partition_name: str):
    """
    Plots the usage metrics (number of jobs, total allocated CPUs,
    total requested memory) for a specific partition over time, aggregated by month.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing job data.
        partition_name (str): The name of the partition to plot.
    """

    # Filter data for the specific partition
    partition_df = df[df['Partition'] == partition_name].copy()

    # Create a 'MonthYear' column for time aggregation
    partition_df['MonthYear'] = partition_df['Start'].dt.to_period('M')
    
    # Convert ReqMem from MB to GB for consistency in memory usage
    if 'ReqMem' in partition_df.columns and partition_df['ReqMem'].max() > 2048:
        partition_df['ReqMem'] = partition_df['ReqMem'] / 1024  # Convert MB to GB
    else:
        partition_df['ReqMem'] = partition_df['ReqMem']

    # Aggregate data by MonthYear
    monthly_usage = partition_df.groupby('MonthYear').agg(
        JobCount=('JobID', 'count'),
        TotalAllocCPUs=('AllocCPUS', 'sum'),
        TotalReqMem=('ReqMem', 'sum')
    ).reset_index()

    # Convert 'MonthYear' back to datetime for plotting consistency
    monthly_usage['MonthYear'] = monthly_usage['MonthYear'].dt.to_timestamp()

    # Plotting Number of Jobs over time for the partition
    common_plot(
        df=monthly_usage,
        x='MonthYear',
        y='JobCount',
        title=f'Number of Jobs on {partition_name} Partition Over Time',
        xlabel='Month',
        ylabel='Number of Jobs',
        color='blue',
        kind='bar',
    )

    # Plotting Total Allocated CPUs over time for the partition
    common_plot(
        df=monthly_usage,
        x='MonthYear',
        y='TotalAllocCPUs',
        title=f'Total Allocated CPUs on {partition_name} Partition Over Time',
        xlabel='Month',
        ylabel='Total Allocated CPUs',
        color='orange',
        kind='bar',
    )

    # Plotting Total Requested Memory over time for the partition
    common_plot(
        df=monthly_usage,
        x='MonthYear',
        y='TotalReqMem',
        title=f'Total Requested Memory (GB) on {partition_name} Partition Over Time',
        xlabel='Month',
        ylabel='Total Requested Memory (GB)',
        color='green',
        kind='bar',
    )

# Example usage:
partitions = ['parallel', 'gpu', 'bigmem', 'longrun', 'quicktest']
partitions = ['parallel']
for partition in partitions:
    plot_partition_usage_over_time(df, partition_name=partition)

In [ ]:
def plot_yearly_partition_job_usage_subplots(df: pd.DataFrame):
    """
    Plots the number of jobs per partition for each year as stacked bar charts in subplots.
    Each subplot represents a unique year, with bars stacked by partition.
    Legends within each subplot are sorted by total job count for that year's partitions,
    and display the total job count for each partition.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing job data,
                           expected to have 'Start' (datetime), 'Partition', and 'JobID' columns.
                           'Year' and 'Month' will be derived from 'Start'.
    """

    if df.empty:
        print("Cannot plot partition usage: DataFrame is empty.")
        return

    # --- Data Preprocessing ---
    # Ensure 'Start' column is datetime
    if not pd.api.types.is_datetime64_any_dtype(df['Start']):
        print("Warning: 'Start' column is not datetime. Attempting conversion.")
        df['Start'] = pd.to_datetime(df['Start'], errors='coerce')
    
    # Drop rows where 'Start' could not be parsed
    df.dropna(subset=['Start'], inplace=True)

    if df.empty:
        print("After dropping rows with invalid 'Start' times, the DataFrame is empty.")
        return

    df['Year'] = df['Start'].dt.year
    df['Month'] = df['Start'].dt.month
    df['YearMonth'] = df['Start'].dt.to_period('M') # Use Period for accurate monthly grouping

    # Drop rows with missing critical data AFTER deriving YearMonth
    df.dropna(subset=['YearMonth', 'Partition', 'JobID'], inplace=True) 

    if df.empty:
        print("After preprocessing and cleaning, the DataFrame is empty for plotting.")
        return
    
    # For plotting all available years in subplots (no 'last two years' filter here unless desired)
    df_relevant_years = df.copy()

    # --- Basic Data Validation ---
    if 'Partition' not in df_relevant_years.columns:
        print("Error: 'Partition' column not found. Cannot plot partition usage.")
        return
    if 'JobID' not in df_relevant_years.columns:
        print("Error: 'JobID' column not found. Cannot count jobs.")
        return

    # --- Aggregate job counts by YearMonth and Partition ---
    # This creates a DataFrame with 'YearMonth' as index and 'Partition' as columns
    monthly_partition_jobs = df_relevant_years.groupby(['YearMonth', 'Partition'])['JobID'].count().unstack(fill_value=0)

    # Get unique sorted years for subplot creation from the aggregated data's index
    unique_years = sorted(monthly_partition_jobs.index.map(lambda x: x.year).unique())
    if not unique_years:
        print("No valid years found in the DataFrame to plot.")
        return

    # Determine subplot grid dimensions
    num_years = len(unique_years)
    ncols = min(3, num_years) # Max 3 columns for subplots for readability
    nrows = int(np.ceil(num_years / ncols))

    # Adjust figsize to give more room, especially for legends
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(6 * ncols + 5, 7 * nrows), squeeze=False)
    axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

    # --- Calculate overall max job count for consistent Y-axis limits across all subplots ---
    max_job_count = 0
    if not monthly_partition_jobs.empty:
        # Sum rows (job counts for all partitions for each month) and find the maximum
        max_job_count = monthly_partition_jobs.sum(axis=1).max()
    if max_job_count == 0:
        max_job_count = 1 # Avoid issues with empty or zero data

    for i, year in enumerate(unique_years):
        ax = axes[i]
        
        # Filter monthly_partition_jobs for the current year
        monthly_jobs_for_year = monthly_partition_jobs[monthly_partition_jobs.index.map(lambda x: x.year) == year]

        if monthly_jobs_for_year.empty:
            ax.set_title(f'Job Usage - {year} (No Data)', fontsize=14, fontweight='bold')
            ax.axis('off') # Hide axes if no data for the year
            continue

        # Calculate total jobs per partition for sorting the legend for THIS YEAR
        partition_totals_year = monthly_jobs_for_year.sum().sort_values(ascending=True)

        # Reindex the columns of monthly_jobs_for_year based on the sorted totals
        # This ensures the stacking order and legend order match the sorted totals
        # Filter out partitions that might have 0 total jobs for this year to avoid them in legend
        partitions_to_plot = partition_totals_year[partition_totals_year > 0].index
        monthly_jobs_for_year = monthly_jobs_for_year[partitions_to_plot]


        # Convert 'MonthYear' PeriodIndex to string, showing only month for x-axis
        # Convert to timestamp first to use strftime for month abbreviation
        monthly_jobs_for_year.index = monthly_jobs_for_year.index.to_timestamp().strftime('%b') # E.g., Jan, Feb

        # --- Plotting on the current subplot ---
        monthly_jobs_for_year.plot(kind='bar', stacked=True, ax=ax, cmap='tab20_r', alpha=0.9)
        
        # --- Apply Subplot Specific Formatting ---
        ax.set_title(f'Job Usage - {year}', fontsize=14, fontweight='bold')
        ax.set_xlabel('Month', fontsize=11)
        ax.set_ylabel('Number of Jobs', fontsize=11)
        ax.set_ylim(0, max_job_count * 1.1) # Apply consistent Y-axis limit across all subplots
        ax.tick_params(axis='x', rotation=45, labelsize=9)
        ax.tick_params(axis='y', labelsize=9)
        ax.grid(True, linestyle='--', alpha=0.7, axis='y')
        

        # --- Update Legend for the current subplot ---
        legend_labels = []
        handles, labels = ax.get_legend_handles_labels()
        
        # Ensure we only consider labels that actually have data in the current year's plot
        # and re-sort them based on partition_totals_year
        current_plot_labels_sorted = [lbl for lbl in partition_totals_year.index if lbl in labels]
        
        # Map labels to their corresponding handles for correct sorting
        label_to_handle = dict(zip(labels, handles))
        sorted_handles = [label_to_handle[lbl] for lbl in current_plot_labels_sorted]


        for label in current_plot_labels_sorted:
            total_jobs = partition_totals_year.get(label, 0) # Use .get() for robustness
            legend_labels.append(f"{label} ({total_jobs:,})")

        ax.legend(sorted_handles, legend_labels, title='Partition (Total Jobs)', 
                  loc='upper left', bbox_to_anchor=(1.02, 1), # Place legend outside the plot area
                  ncol=1, # One column for legend for cleaner look next to subplot
                  fontsize=9,
                  frameon=False)
        
    # --- Clean up any unused subplots if num_years is not a perfect multiple of ncols ---
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j]) # Remove empty subplots

    # --- Adjust overall layout ---
    fig.suptitle('Monthly Job Usage by Partition - Yearly Breakdown', fontsize=18, fontweight='bold', y=1.02) # Main title
    # Adjust rect to ensure enough space for titles and legends
    plt.tight_layout(rect=[0, 0.03, 1, 0.98]) 
    plt.show()
    
    
plot_yearly_partition_job_usage_subplots(df)  # Call the function to plot yearly partition job usage in subplots

In [ ]:
def plot_job_type_usage_over_time(df: pd.DataFrame, metric: str, metric_label: str, time_granularity: str = 'M'):
    """
    Plots the usage of a specified metric over time, separated by 'Single Job' and 'Array Job'.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing job data with 'JobType' column.
        metric (str): The column name for the usage metric to plot (e.g., 'TotalAllocCPUs', 'ReqMem_GB').
        metric_label (str): The human-readable label for the metric (e.g., 'Total Allocated CPUs', 'Requested Memory (GB)').
        time_granularity (str): The pandas time series frequency for aggregation (e.g., 'M' for monthly).
    """

    # Ensure 'MonthYear' column exists for aggregation
    df['MonthYear'] = df['Start'].dt.to_period(time_granularity)

    job_types = df['JobType'].unique()

    for job_type in job_types:
        # Filter for the specific job type
        job_type_df = df[df['JobType'] == job_type].copy()

        # Aggregate data by MonthYear for the current job type
        monthly_usage = job_type_df.groupby('MonthYear')[metric].sum().reset_index()

        # Convert 'MonthYear' back to datetime for plotting consistency
        monthly_usage['MonthYear'] = monthly_usage['MonthYear'].dt.to_timestamp()

        # Plotting the metric over time for the current job type
        common_plot(
            df=monthly_usage,
            x='MonthYear',
            y=metric,
            title=f'{metric_label} for {job_type}s Over Time',
            xlabel=f'Time ({time_granularity})',
            ylabel=metric_label,
            color='purple' if job_type == 'Array Job' else 'teal', # Different colors for clarity
            kind='bar'
        )

# Convert ReqMem from MB to GB for plotting consistency
df['ReqMem_GB'] = df['ReqMem'] / 1024
# --- NEW: Classify JobType based on JobID string ---
# Convert JobID to string to check for substrings
df['JobID_str'] = df['JobID'].astype(str)
df['JobType'] = df['JobID_str'].apply(lambda x: 'Array Job' if '_' in x or '.' in x else 'Single Job')

print(f"Number of Single Jobs: {df[df['JobType'] == 'Single Job'].shape[0]}")
print(f"Number of Array Jobs: {df[df['JobType'] == 'Array Job'].shape[0]}")

# Plotting usage for single jobs vs. array jobs
# Example: Plotting Total Allocated CPUs over time for each job type
plot_job_type_usage_over_time(df, 'AllocCPUS', 'Total Allocated CPUs')

# Example: Plotting Total Requested Memory (GB) over time for each job type
plot_job_type_usage_over_time(df, 'ReqMem_GB', 'Total Requested Memory (GB)')

In [ ]:
import numpy as np
def plot_yearly_job_type_usage_subplots(df: pd.DataFrame, metric: str, metric_label: str, threshold_value: Optional[float] = None):
    """
    Plots the usage of a specified metric (e.g., allocated CPUs, requested memory)
    over time, separated by 'Single Job' and 'Array Job', in year-wise subplots.
    Each subplot shows monthly stacked bars. Legends within each subplot are sorted
    by the total metric value for each job type in that year, displaying the total.
    Optionally, filters out individual data points where the metric value exceeds a given threshold.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing job data with
                           'Start', 'JobType' (derived from JobID), and the specified metric column.
        metric (str): The column name for the usage metric to plot (e.g., 'AllocCPUS', 'ReqMem_GB').
        metric_label (str): The human-readable label for the metric (e.g., 'Total Allocated CPUs', 'Requested Memory (GB)').
        threshold_value (Optional[float]): If provided, individual job records where the 'metric'
                                           is greater than this value will be filtered out before aggregation.
    """

    if df.empty:
        print(f"Cannot plot yearly job type usage for {metric_label}: DataFrame is empty.")
        return

    # --- Data Preprocessing ---
    # Ensure 'Start' column is datetime
    if not pd.api.types.is_datetime64_any_dtype(df['Start']):
        print("Warning: 'Start' column is not datetime. Attempting conversion.")
        df['Start'] = pd.to_datetime(df['Start'], errors='coerce')
    
    df.dropna(subset=['Start'], inplace=True) # Drop rows where 'Start' could not be parsed

    if df.empty:
        print(f"After cleaning invalid 'Start' times, the DataFrame is empty for {metric_label}.")
        return

    # Derive Year, Month, YearMonth
    df['Year'] = df['Start'].dt.year
    df['Month'] = df['Start'].dt.month
    df['YearMonth'] = df['Start'].dt.to_period('M') # Use Period for accurate monthly grouping

    # Ensure JobType is classified (as per your example usage)
    if 'JobID' in df.columns and 'JobType' not in df.columns:
        print("Classifying 'JobType' based on 'JobID' for plotting.")
        df['JobID_str'] = df['JobID'].astype(str)
        df['JobType'] = df['JobID_str'].apply(lambda x: 'Array Job' if '_' in x or '.' in x else 'Single Job')
        df.drop(columns=['JobID_str'], inplace=True)
    elif 'JobType' not in df.columns:
        print("Error: 'JobType' column not found and cannot be derived from 'JobID'. Cannot plot.")
        return

    # Drop rows with missing critical data AFTER deriving JobType
    df.dropna(subset=['YearMonth', 'JobType', metric], inplace=True) 

    if df.empty:
        print(f"After preprocessing and cleaning, the DataFrame is empty for {metric_label} plotting.")
        return
    
    df_relevant_years = df.copy() # Use a copy for operations

    # --- Apply Threshold Filtering (NEW ADDITION) ---
    if threshold_value is not None:
        if metric not in df_relevant_years.columns:
            print(f"Error: Metric '{metric}' not found for thresholding. Skipping threshold filter.")
        else:
            original_rows_count = len(df_relevant_years)
            df_relevant_years = df_relevant_years[df_relevant_years[metric] <= threshold_value].copy()
            filtered_rows_count = len(df_relevant_years)
            
            if original_rows_count > filtered_rows_count:
                print(f"Filtered {original_rows_count - filtered_rows_count} rows ({((original_rows_count - filtered_rows_count) / original_rows_count):.1%}) "
                      f"where '{metric}' was greater than {threshold_value}.")
            
            if df_relevant_years.empty:
                print(f"DataFrame became empty after applying threshold {threshold_value} on '{metric}'. No plot will be generated.")
                return

    # --- Aggregate the specified metric by YearMonth and JobType ---
    monthly_job_type_metric = df_relevant_years.groupby(['YearMonth', 'JobType'])[metric].sum().unstack(fill_value=0)

    # Get unique sorted years for subplot creation
    unique_years = sorted(monthly_job_type_metric.index.map(lambda x: x.year).unique())
    if not unique_years:
        print("No valid years found in the DataFrame to plot.")
        return

    # Determine subplot grid dimensions
    num_years = len(unique_years)
    ncols = min(3, num_years)
    nrows = int(np.ceil(num_years / ncols))

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(6 * ncols + 5, 7 * nrows), squeeze=False)
    axes = axes.flatten()

    # --- Calculate overall max metric value for consistent Y-axis limits across all subplots ---
    max_metric_value = 0
    if not monthly_job_type_metric.empty:
        max_metric_value = monthly_job_type_metric.sum(axis=1).max()
    if max_metric_value == 0:
        max_metric_value = 1

    for i, year in enumerate(unique_years):
        ax = axes[i]
        
        monthly_metric_for_year = monthly_job_type_metric[monthly_job_type_metric.index.map(lambda x: x.year) == year]

        if monthly_metric_for_year.empty:
            ax.set_title(f'{metric_label} - {year} (No Data)', fontsize=14, fontweight='bold')
            ax.axis('off')
            continue

        job_type_totals_year = monthly_metric_for_year.sum().sort_values(ascending=True)

        job_types_to_plot = job_type_totals_year[job_type_totals_year > 0].index
        monthly_metric_for_year = monthly_metric_for_year[job_types_to_plot]

        monthly_metric_for_year.index = monthly_metric_for_year.index.to_timestamp().strftime('%b')

        # --- Plotting on the current subplot ---
        monthly_metric_for_year.plot(kind='bar', stacked=True, ax=ax, cmap='tab20_r', alpha=0.9)
        
        # --- Apply Subplot Specific Formatting ---
        ax.set_title(f'{metric_label} - {year}', fontsize=14, fontweight='bold')
        ax.set_xlabel('Month', fontsize=11)
        ax.set_ylabel(metric_label, fontsize=11)
        ax.set_ylim(0, max_metric_value * 1.1)
        ax.tick_params(axis='x', rotation=45, labelsize=9)
        ax.tick_params(axis='y', labelsize=9)
        ax.grid(True, linestyle='--', alpha=0.7, axis='y')

        # --- Update Legend for the current subplot ---
        legend_labels = []
        handles, labels = ax.get_legend_handles_labels()
        
        current_plot_labels_sorted = [lbl for lbl in job_type_totals_year.index if lbl in labels]
        
        label_to_handle = dict(zip(labels, handles))
        sorted_handles = [label_to_handle[lbl] for lbl in current_plot_labels_sorted]

        for label in current_plot_labels_sorted:
            total_value = job_type_totals_year.get(label, 0)
            legend_labels.append(f"{label} ({total_value:,.0f})")

        ax.legend(sorted_handles, legend_labels, title='Job Type (Total)', 
                  loc='upper left', bbox_to_anchor=(1.02, 1),
                  ncol=1,
                  fontsize=9,
                  frameon=False)
        
    # --- Clean up any unused subplots ---
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # --- Adjust overall layout ---
    fig.suptitle(f'Monthly {metric_label} by Job Type - Yearly Breakdown', fontsize=18, fontweight='bold', y=1.02)
    plt.tight_layout(rect=[0, 0.03, 1, 0.98]) 
    plt.show()

    
    
    
plot_yearly_job_type_usage_subplots(df, 'AllocCPUS', 'Total Allocated CPUs', threshold_value=20000)  # Example with threshold
plot_yearly_job_type_usage_subplots(df, 'ReqMem', 'Total Requested Memory (GB)', threshold_value=200000)  # Example with threshold


In [ ]:
def plot_partition_job_type_usage(df: pd.DataFrame, metric: str, metric_label: str, time_granularity: str = 'M'):
    """
    Plots the usage of a specified metric over time for single and array jobs,
    separately for each unique partition.
    Excludes partitions containing multiple values (e.g., 'gpu,bigmem') and plots as bar charts.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing job data with 'JobType' and 'Partition' columns.
        metric (str): The column name for the usage metric to plot (e.g., 'TotalAllocCPUs', 'ReqMem_GB').
        metric_label (str): The human-readable label for the metric (e.g., 'Total Allocated CPUs', 'Requested Memory (GB)').
        time_granularity (str): The pandas time series frequency for aggregation (e.g., 'M' for monthly).
    """
    unique_partitions = df['Partition'].unique()
    job_types = df['JobType'].unique()

    for partition in unique_partitions:
        # Filter out partitions that contain commas or spaces, indicating multiple values
        # e.g., "gpu,bigmem" or "partition1 partition2"
        if ',' in partition or ' ' in partition:
            print(f"Skipping multi-value partition: {partition}")
            continue

        print(f"\n--- Plotting for Partition: {partition} ---")
        partition_df = df[df['Partition'] == partition].copy()

        # Ensure 'MonthYear' column exists for aggregation within this partition's data
        partition_df['MonthYear'] = partition_df['Start'].dt.to_period(time_granularity)

        for job_type in job_types:
            # Filter for the specific job type within the current partition
            job_type_partition_df = partition_df[partition_df['JobType'] == job_type].copy()

            if not job_type_partition_df.empty:
                # Aggregate data by MonthYear for the current job type and partition
                monthly_usage = job_type_partition_df.groupby('MonthYear')[metric].sum().reset_index()

                # Convert 'MonthYear' back to datetime for plotting consistency
                monthly_usage['MonthYear'] = monthly_usage['MonthYear'].dt.to_timestamp()

                # Plotting the metric over time for the current job type and partition
                common_plot(
                    df=monthly_usage,
                    x='MonthYear',
                    y=metric,
                    title=f'{metric_label} for {job_type}s on {partition} Over Time',
                    xlabel=f'Time ({time_granularity})',
                    ylabel=metric_label,
                    color='purple' if job_type == 'Array Job' else 'teal', # Consistent colors
                    kind='bar' # Changed from 'line' to 'bar' as requested
                )
            else:
                print(f"No {job_type} data found for partition: {partition}")

                
# Convert ReqMem from MB to GB for plotting consistency
df['ReqMem_GB'] = df['ReqMem'] / 1024

# Classify JobType based on JobID string
# Convert JobID to string to check for substrings
df['JobID_str'] = df['JobID'].astype(str)
df['JobType'] = df['JobID_str'].apply(lambda x: 'Array Job' if '_' in x or '.' in x else 'Single Job')

print(f"Number of Single Jobs: {df[df['JobType'] == 'Single Job'].shape[0]}")
print(f"Number of Array Jobs: {df[df['JobType'] == 'Array Job'].shape[0]}")


# --- NEW: Plotting usage per partition for single jobs vs. array jobs ---
# Example: Plotting Total Allocated CPUs over time for each job type PER PARTITION
plot_partition_job_type_usage(df, 'AllocCPUS', 'Total Allocated CPUs')

# Example: Plotting Total Requested Memory (GB) over time for each job type PER PARTITION
plot_partition_job_type_usage(df, 'ReqMem_GB', 'Total Requested Memory (GB)')


In [ ]:
def plot_cores_requested_per_partition_over_time(df: pd.DataFrame, time_granularity: str = 'M'):
    """
    Plots the number of jobs requesting specific core counts (0-16, 16-32, 32-64, 64-128, and 128+ cores)
    over time, for each single-value partition.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing job data.
        time_granularity (str): The pandas time series frequency for aggregation (e.g., 'M' for monthly).
    """
    # Define core ranges (bins) and corresponding labels
    # Bins are defined such that pd.cut with right=True creates (lower, upper] intervals
    core_bins = [0, 16, 32, 64, 128, float('inf')]
    core_labels = ['0-16 cores', '16-32 cores', '32-64 cores', '64-128 cores', '128+ cores']

    # Categorize jobs by requested cores using the new bins and labels
    df['CoreCategory'] = pd.cut(df['ReqCPUS'], bins=core_bins, labels=core_labels, right=True, include_lowest=True)

    unique_partitions = df['Partition'].unique()

    for partition in unique_partitions:
        # Filter out partitions that contain commas or spaces, indicating multiple values
        if ',' in partition or ' ' in partition:
            print(f"Skipping multi-value partition for core analysis: {partition}")
            continue

        print(f"\n--- Plotting Core Request Trends for Partition: {partition} ---")
        partition_df = df[df['Partition'] == partition].copy()

        if partition_df.empty:
            print(f"No data for partition: {partition}")
            continue

        # Ensure 'MonthYear' column exists for aggregation within this partition's data
        partition_df['MonthYear'] = partition_df['Start'].dt.to_period(time_granularity)

        # Aggregate job counts by MonthYear and CoreCategory
        # Using .unstack() to pivot CoreCategory into columns for easier plotting
        core_usage_over_time = partition_df.groupby(['MonthYear', 'CoreCategory']).size().unstack(fill_value=0)
        # Convert PeriodIndex to TimestampIndex for plotting with matplotlib/seaborn
        core_usage_over_time.index = core_usage_over_time.index.to_timestamp()

        # To plot each core category separately using common_plot
        # Melt the DataFrame to long format suitable for iterating over categories
        plot_df = core_usage_over_time.reset_index().melt(
            id_vars='MonthYear',
            var_name='CoreCategory',
            value_name='JobCount'
        )

        # Ensure consistent order of CoreCategory in plots
        plot_df['CoreCategory'] = pd.Categorical(plot_df['CoreCategory'], categories=core_labels, ordered=True)
        plot_df = plot_df.sort_values(['MonthYear', 'CoreCategory'])

        # Plot each core category
        for core_category in core_labels:
            category_df = plot_df[plot_df['CoreCategory'] == core_category].copy()
            if not category_df.empty:
                common_plot(
                    df=category_df,
                    x='MonthYear',
                    y='JobCount',
                    title=f'Jobs Requesting {core_category} on {partition} Over Time',
                    xlabel=f'Time ({time_granularity})',
                    ylabel='Number of Jobs',
                    color=plt.cm.viridis(core_labels.index(core_category) / len(core_labels)), # Dynamic color
                    kind='bar' # Plotting as bar chart
                )
            else:
                print(f"No jobs found for '{core_category}' on partition '{partition}'.")



# --- NEW: Plotting jobs by requested cores per partition over time with updated bins ---
plot_cores_requested_per_partition_over_time(df)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math

def plot_cores_requested_per_partition_over_time(df: pd.DataFrame, time_granularity: str = 'M'):
    """
    Plots the number of jobs requesting specific core counts over time for each partition.
    Each figure contains up to 3x3 subplots, each representing a partition and the first year of data.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing job data.
        time_granularity (str): The pandas time series frequency for aggregation (e.g., 'M' for monthly).
    """
    if df.empty:
        print("Cannot plot core request breakdown: DataFrame is empty.")
        return

    # --- Robust Date Cleaning and Filtering ---
    df_clean = df.copy()
    df_clean['Start'] = pd.to_datetime(df_clean['Start'], errors='coerce')
    df_clean.dropna(subset=['Start', 'Partition', 'ReqCPUS'], inplace=True)

    if df_clean.empty:
        print("After initial cleaning (NaNs in Start, Partition, ReqCPUS), DataFrame is empty for core request analysis.")
        return

    current_year = pd.Timestamp.now().year
    df_clean = df_clean[
        (df_clean['Start'].dt.year >= 1970) &
        (df_clean['Start'].dt.year <= current_year + 1)
    ].copy()

    if df_clean.empty:
        print("After filtering 'Start' dates by year range (1970-CurrentYear+1), DataFrame is empty for core request analysis.")
        return
    # --- End Robust Date Cleaning ---

    # Define core ranges (bins) and corresponding labels
    core_bins = [0, 16, 32, 64, 128, float('inf')]
    core_labels = ['0-16 cores', '16-32 cores', '32-64 cores', '64-128 cores', '128+ cores']

    # Categorize jobs by requested cores using the new bins and labels
    df_clean['CoreCategory'] = pd.cut(df_clean['ReqCPUS'], bins=core_bins, labels=core_labels, right=True, include_lowest=True, ordered=True)
    df_clean.dropna(subset=['CoreCategory'], inplace=True)

    if df_clean.empty:
        print("After categorizing core requests, no jobs remain for plotting.")
        return

    # Ensure 'YearMonth' column exists for aggregation within this partition's data
    df_clean['YearMonth'] = df_clean['Start'].dt.to_period(time_granularity)

    # Aggregate job counts by Partition, YearMonth and CoreCategory
    # Using observed=False for CategoricalDtype to include all categories even if not present in a group
    core_usage_over_time = df_clean.groupby(['Partition', 'YearMonth', 'CoreCategory'], observed=False).size().unstack(fill_value=0)

    # Get unique partitions, filtering out multi-value or NaN partitions
    unique_partitions = []
    for p in df_clean['Partition'].unique():
        if pd.isna(p) or (isinstance(p, str) and (',' in p or ' ' in p)):
            print(f"Skipping multi-value or invalid partition for core request analysis: {p}")
            continue
        unique_partitions.append(p)
    unique_partitions = sorted(unique_partitions)

    if not unique_partitions:
        print("No single-value partitions found for plotting core requests.")
        return

    # Prepare a list of all (partition, year) combinations that need a subplot
    plots_to_generate = []
    for partition in unique_partitions:
        if partition not in core_usage_over_time.index.get_level_values('Partition'):
            continue # Skip if partition has no data after aggregation
        partition_data = core_usage_over_time.loc[partition]
        unique_years_for_partition = sorted(partition_data.index.get_level_values('YearMonth').year.unique())
        for year in unique_years_for_partition:
            plots_to_generate.append((partition, year))

    if not plots_to_generate:
        print("No valid (partition, year) combinations found to plot core requests.")
        return

    MAX_COLS = 3
    MAX_ROWS = 3
    SUBPLOTS_PER_FIGURE = MAX_COLS * MAX_ROWS

    # Iterate through plots in chunks of SUBPLOTS_PER_FIGURE
    for i in range(0, len(plots_to_generate), SUBPLOTS_PER_FIGURE):
        current_chunk = plots_to_generate[i:i + SUBPLOTS_PER_FIGURE]
        num_plots_in_chunk = len(current_chunk)
        current_nrows = math.ceil(num_plots_in_chunk / MAX_COLS)
        current_ncols = min(MAX_COLS, num_plots_in_chunk)

        fig, axes = plt.subplots(nrows=current_nrows, ncols=current_ncols,
                                 figsize=(10 * current_ncols + 2, 6 * current_nrows),
                                 squeeze=False)
        axes = axes.flatten()

        for j, (partition, year) in enumerate(current_chunk):
            ax = axes[j]
            print(f"Plotting Core Requests: Partition={partition}, Year={year}")

            current_year_data = core_usage_over_time.loc[(partition, slice(None)), :]
            current_year_data = current_year_data[current_year_data.index.get_level_values('YearMonth').year == year]

            if current_year_data.empty:
                ax.set_title(f'No Data: {partition} - {year}', fontsize=12)
                ax.axis('off')
                continue

            # Convert YearMonth to string for x-axis labels
            current_year_data.index = current_year_data.index.get_level_values('YearMonth').astype(str)

            # Ensure all core categories are present as columns, fill missing with 0
            current_plot_data = current_year_data.reindex(columns=core_labels, fill_value=0)

            # Dynamic Y-axis limit for this subplot
            current_subplot_max_y = current_plot_data.sum(axis=1).max()
            if current_subplot_max_y == 0:
                current_subplot_max_y = 1 # Avoid division by zero if all counts are zero

            current_plot_data.plot(kind='bar', stacked=True, ax=ax, colormap='plasma', alpha=0.9)

            ax.set_title(f'{partition} - {year}', fontsize=14, fontweight='bold')
            ax.set_xlabel('Month', fontsize=11)
            ax.set_ylabel('Number of Jobs', fontsize=11)
            ax.set_ylim(0, current_subplot_max_y * 1.1) # Dynamic Y-limit
            ax.tick_params(axis='x', rotation=45, labelsize=9)
            ax.tick_params(axis='y', labelsize=9)
            ax.grid(True, linestyle='--', alpha=0.7, axis='y')

            # Add legend to each subplot
            handles, labels = ax.get_legend_handles_labels()
            # Sort labels according to core_labels order for consistent legend
            sorted_labels_for_legend = [lbl for lbl in core_labels if lbl in labels]
            label_to_handle = dict(zip(labels, handles))
            sorted_handles = [label_to_handle[lbl] for lbl in sorted_labels_for_legend]

            legend_labels_with_counts = []
            for label in sorted_labels_for_legend:
                total_jobs = current_plot_data[label].sum()
                legend_labels_with_counts.append(f"{label} ({total_jobs:,})")

            ax.legend(sorted_handles, legend_labels_with_counts, title='Core Category (Total Jobs)',
                      loc='upper right', bbox_to_anchor=(1.3, 1), fontsize=9, frameon=False)

        # Hide any unused subplots in the current figure
        for k in range(j + 1, len(axes)):
            fig.delaxes(axes[k])

        fig.suptitle(f'Monthly Breakdown of Core Requests per Partition (Figure {i // SUBPLOTS_PER_FIGURE + 1})',
                     fontsize=18, fontweight='bold', y=1.02)
        plt.tight_layout(rect=[0, 0.03, 1, 0.98])
        plt.show()

    print("\n--- Finished plotting core request trends. ---")


# Example usage
plot_cores_requested_per_partition_over_time(df)


In [ ]:
def plot_yearly_partition_long_wait_breakdown(df: pd.DataFrame, partition: str = None):
    """
    Plots the breakdown of jobs with wait times > 4 hours, partitioned by year
    and individual partition in subplots. Each subplot shows monthly stacked bars,
    where stacks represent different wait time categories. Legends are sorted by
    total job count for each wait category within that specific (partition, year) subplot.
    Y-axis limits are dynamic for each subplot.
    Each figure will contain a maximum of 3x3 subplots, creating new figures as needed.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing job data with
                           'Start', 'Partition', 'JobID', and 'Wait' columns.
        partition (str, optional): If provided, plots only for this specific partition.
                                   Defaults to None, plotting all unique single-value partitions.
    """
    if df.empty:
        print("Cannot plot long wait time breakdown: DataFrame is empty.")
        return

    # --- Robust Date Cleaning and Filtering ---
    df_clean_wait = df.copy()
    df_clean_wait['Start'] = pd.to_datetime(df_clean_wait['Start'], errors='coerce')
    df_clean_wait.dropna(subset=['Wait', 'Start', 'Partition', 'JobID'], inplace=True)

    if df_clean_wait.empty:
        print("After initial cleaning (NaNs in Start, Wait, Partition, JobID), DataFrame is empty for wait time analysis.")
        return

    current_year = pd.Timestamp.now().year
    df_clean_wait = df_clean_wait[
        (df_clean_wait['Start'].dt.year >= 1970) &
        (df_clean_wait['Start'].dt.year <= current_year + 1)
    ].copy()

    if df_clean_wait.empty:
        print("After filtering 'Start' dates by year range (1970-CurrentYear+1), DataFrame is empty for wait time analysis.")
        return
    # --- End Robust Date Cleaning ---

    four_hours = pd.Timedelta(hours=4)
    long_wait_jobs_df = df_clean_wait[df_clean_wait['Wait'] > four_hours].copy()

    if long_wait_jobs_df.empty:
        print("No jobs found with wait times greater than 4 hours to plot breakdown.")
        return

    wait_bins = [
        four_hours,
        pd.Timedelta(hours=12),
        pd.Timedelta(hours=24),
        pd.Timedelta(hours=48),
        long_wait_jobs_df['Wait'].max() + pd.Timedelta(seconds=1)
    ]
    wait_labels = [
        '4-12 hours',
        '12-24 hours',
        '24-48 hours',
        '>48 hours'
    ]

    # Handle cases where max wait time might be less than 48 hours, reducing bins dynamically
    effective_bins = [b for b in wait_bins if b <= long_wait_jobs_df['Wait'].max() + pd.Timedelta(seconds=1)]
    effective_labels = wait_labels[:len(effective_bins) - 1]

    if not effective_labels:
        effective_bins = [four_hours, long_wait_jobs_df['Wait'].max() + pd.Timedelta(seconds=1)]
        effective_labels = ['4+ hours']

    long_wait_jobs_df['WaitCategory'] = pd.cut(long_wait_jobs_df['Wait'],
                                               bins=effective_bins,
                                               labels=effective_labels,
                                               right=True,
                                               include_lowest=True,
                                               ordered=True)

    long_wait_jobs_df.dropna(subset=['WaitCategory'], inplace=True)

    if long_wait_jobs_df.empty:
        print("After categorizing wait times, no jobs remain for plotting.")
        return

    long_wait_jobs_df['Year'] = long_wait_jobs_df['Start'].dt.year
    long_wait_jobs_df['YearMonth'] = long_wait_jobs_df['Start'].dt.to_period('M')

    # Aggregate data once
    monthly_partition_wait_counts_all = long_wait_jobs_df.groupby(
        ['YearMonth', 'Partition', 'WaitCategory'], observed=False
    )['JobID'].count().unstack(fill_value=0)

    # Determine partitions to plot
    if partition:
        unique_partitions_to_plot = [p for p in [partition] if p in monthly_partition_wait_counts_all.index.get_level_values('Partition')]
        if not unique_partitions_to_plot:
            print(f"Specified partition '{partition}' not found in long wait data or is invalid.")
            return
    else:
        # Filter for single-value partitions only
        all_partitions_in_data = monthly_partition_wait_counts_all.index.get_level_values('Partition').unique()
        unique_partitions_to_plot = []
        for p in all_partitions_in_data:
            if pd.isna(p) or (isinstance(p, str) and (',' in p or ' ' in p)):
                print(f"Skipping multi-value or invalid partition for wait time analysis: {p}")
                continue
            unique_partitions_to_plot.append(p)
        unique_partitions_to_plot = sorted(unique_partitions_to_plot)
        if not unique_partitions_to_plot:
            print("No single-value partitions found for plotting long wait times.")
            return

    # Prepare a list of all (partition, year) combinations that need a subplot
    plots_to_generate = []
    for current_partition in unique_partitions_to_plot:
        partition_data = monthly_partition_wait_counts_all.xs(current_partition, level='Partition', drop_level=False)
        if not partition_data.empty:
            unique_years_for_partition = sorted(partition_data.index.get_level_values('YearMonth').map(lambda x: x.year).unique())
            for year in unique_years_for_partition:
                plots_to_generate.append((current_partition, year))

    if not plots_to_generate:
        print("No valid (partition, year) combinations found to plot long wait times.")
        return

    MAX_COLS = 3
    MAX_ROWS = 3
    SUBPLOTS_PER_FIGURE = MAX_COLS * MAX_ROWS

    # Iterate through plots in chunks of SUBPLOTS_PER_FIGURE
    for i in range(0, len(plots_to_generate), SUBPLOTS_PER_FIGURE):
        current_chunk = plots_to_generate[i:i + SUBPLOTS_PER_FIGURE]
        num_plots_in_chunk = len(current_chunk)
        current_nrows = math.ceil(num_plots_in_chunk / MAX_COLS)
        current_ncols = min(MAX_COLS, num_plots_in_chunk)

        fig, axes = plt.subplots(nrows=current_nrows, ncols=current_ncols,
                                 figsize=(10 * current_ncols + 2, 6 * current_nrows),
                                 squeeze=False)
        axes = axes.flatten()
        for j, (current_partition, year) in enumerate(current_chunk):
            ax = axes[j]
            print(f"Plotting Long Waits: Partition={current_partition}, Year={year}")

            current_year_data = monthly_partition_wait_counts_all[
                monthly_partition_wait_counts_all.index.get_level_values('YearMonth').map(lambda x: x.year) == year
            ]
            if current_partition not in current_year_data.index.get_level_values('Partition'):
                current_plot_data = pd.DataFrame()
            else:
                current_plot_data = current_year_data.xs(current_partition, level='Partition', drop_level=False)
                current_plot_data = current_plot_data.droplevel('Partition')
                if current_plot_data.empty:
                    current_plot_data = pd.DataFrame()

            if current_plot_data.empty:
                ax.set_title(f'No Long Waits: {current_partition} - {year}', fontsize=12)
                ax.axis('off')
                continue

            current_plot_data.index = current_plot_data.index.map(lambda x: x.strftime('%b'))

            # Dynamic Y-axis limit for this subplot
            current_subplot_max_y = current_plot_data.sum(axis=1).max()
            if current_subplot_max_y == 0:
                current_subplot_max_y = 1  # Avoid division by zero

            # Ensure all wait categories are present as columns, fill missing with 0
            current_plot_data = current_plot_data.reindex(columns=effective_labels, fill_value=0)
            # Sort categories for consistent legend order if desired, here by total count
            wait_category_totals_subplot = current_plot_data.sum().sort_values(ascending=True)
            categories_to_plot = [lbl for lbl in effective_labels if lbl in wait_category_totals_subplot.index and wait_category_totals_subplot[lbl] > 0]
            current_plot_data = current_plot_data[categories_to_plot]  # Reorder columns for plotting

            current_plot_data.plot(kind='bar', stacked=True, ax=ax, cmap='viridis', alpha=0.9)
            ax.set_title(f'{current_partition} - {year}', fontsize=14, fontweight='bold')
            ax.set_xlabel('Month', fontsize=11)
            ax.set_ylabel('Number of Jobs', fontsize=11)
            ax.set_ylim(0, current_subplot_max_y * 1.1)  # Dynamic Y-limit
            ax.tick_params(axis='x', rotation=45, labelsize=9)
            ax.tick_params(axis='y', labelsize=9)
            ax.grid(True, linestyle='--', alpha=0.7, axis='y')

            # Add legend to each subplot
            handles, labels = ax.get_legend_handles_labels()
            # Create custom legend labels with total job counts for this subplot
            legend_labels_with_counts = []
            # Ensure labels are sorted by effective_labels order for consistency, then append count
            sorted_labels_for_legend = [lbl for lbl in effective_labels if lbl in labels]
            label_to_handle = dict(zip(labels, handles))
            sorted_handles = [label_to_handle[lbl] for lbl in sorted_labels_for_legend]

            for label in sorted_labels_for_legend:
                total_jobs = wait_category_totals_subplot.get(label, 0)
                legend_labels_with_counts.append(f"{label} ({total_jobs:,})")

            ax.legend(sorted_handles, legend_labels_with_counts, title='Wait Time Category (Total Jobs)',
                      loc='upper right', bbox_to_anchor=(1.3, 1), fontsize=9, frameon=False)

        # Hide any unused subplots in the current figure
        for k in range(j + 1, len(axes)):
            fig.delaxes(axes[k])

        fig.suptitle(f'Monthly Breakdown of Long Wait Times (>4h) (Figure {i // SUBPLOTS_PER_FIGURE + 1})',
                     fontsize=18, fontweight='bold', y=1.02)
        plt.tight_layout(rect=[0, 0.03, 1, 0.98])
        plt.show()

    print("\n--- Finished plotting long wait times. ---")

import math




plot_yearly_partition_long_wait_breakdown(df)

In [ ]:
def plot_core_category_comparison_over_time(df: pd.DataFrame, num_weeks: int = 1):
    """
    Plots the total number of jobs for each core category (0-16, 16-32, etc.)
    over a dynamic weekly period (e.g., 1 week, 2 weeks, etc.), allowing comparison across categories.
    The data is filtered to include only the last 'num_weeks' from the latest job start date.
    The total count for each core group is plotted as a single bar.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing job data.
        num_weeks (int): The number of recent weeks to filter data for.
    """
    # Define core ranges (bins) and corresponding labels
    core_bins = [0, 16, 32, 64, 128, float('inf')]
    core_labels = ['0-16 cores', '16-32 cores', '32-64 cores', '64-128 cores', '128+ cores']

    # Categorize jobs by requested cores
    df['CoreCategory'] = pd.cut(df['ReqCPUS'], bins=core_bins, labels=core_labels, right=True, include_lowest=True)

    # Determine the latest date in the dataset
    latest_date = df['Start'].max()
    # Calculate the cutoff date for filtering
    cutoff_date = latest_date - pd.Timedelta(weeks=num_weeks)

    # Filter data for the specified number of recent weeks
    filtered_df = df[df['Start'] >= cutoff_date].copy()

    if filtered_df.empty:
        print(f"No data available for the last {num_weeks} weeks.")
        return

    # Aggregate total job counts by CoreCategory for the filtered period
    aggregated_data = filtered_df.groupby('CoreCategory').size().reset_index(name='TotalJobCount')

    # Ensure consistent order of CoreCategory for plotting
    aggregated_data['CoreCategory'] = pd.Categorical(aggregated_data['CoreCategory'], categories=core_labels, ordered=True)
    aggregated_data = aggregated_data.sort_values('CoreCategory')

    # Plot the total for each core group as a single bar
    print(f"\n--- Plotting Total Jobs per Core Category for the Last {num_weeks} Weeks ---")
    common_plot(
        df=aggregated_data,
        x='CoreCategory',
        y='TotalJobCount',
        title=f'Total Jobs by Core Category (Last {num_weeks} Weeks)',
        xlabel='Core Category',
        ylabel='Total Number of Jobs',
        color='skyblue', # A single color for the comparison bar chart
        kind='bar',
        ylim=(0, aggregated_data['TotalJobCount'].max() * 1.1) # Set y-limit dynamically based on max count
    )



# Example: Plotting with 4-week periods
plot_core_category_comparison_over_time(df, num_weeks=104)